Generating plots that explore the basic, static LOPC model.

In [ ]:
# computation
import lumapi
import numpy as np
import warnings
import xarray as xr
import xyzpy as xyz
from xyzpy.gen.combo_runner import multi_concat
from multilayer_simulator.lumerical_classes import LumericalOscillator, format_stackrt, format_stackfield
from multilayer_simulator.helpers.mixins import convert_wavelength_and_frequency
import dask
from functools import partial
import pandas as pd
from tqdm import tqdm
# plotting
import hvplot.xarray
import holoviews as hv
from holoviews import dim, opts
import panel as pn
import panel.widgets as pnw

In [ ]:
hv.extension("bokeh", "matplotlib", case_sensitive_completion=True)
pn.config.throttled = True

xarray_engine='h5netcdf'

In [ ]:
# A hacky way to import a script

import os
import sys

dir2 = os.path.abspath("")
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path:
    sys.path.append(dir1)
from LOPC import LOPC
from LOPC.helpers import (
    assign_derived_attrs,
    restack,
    enhancement_factor,
    combo_length,
    estimate_combo_run_time,
    linewidth_calculator,
    lopc_data,
    spectrum,
    normalise_over_dim,
    integrate_da,
    sel_or_integrate,
    find_optimum_coords,
    plot_secondary,
    pre_process_for_plots,
    vlines,
    coordinate_string,
    plot_da,
    plot_var,
    plot_optimum_over_dim,
    plot_field,
    complex_elements,
)

Turn on auto-archiving of cells and Holoviews outputs. See the user guide [here](https://holoviews.org/user_guide/Exporting_and_Archiving.html).

Might need to install `ipympl`.

In [ ]:
# hv.archive.auto() # This breaks the DynamicMap call below due to something to do with pickling

run_number = 2

Useful variables for plotting later:

In [ ]:
blue = hv.Cycle.default_cycles['default_colors'][0]
red = hv.Cycle.default_cycles['default_colors'][1]
yellow = hv.Cycle.default_cycles['default_colors'][2]
green = hv.Cycle.default_cycles['default_colors'][3]

In [ ]:
wavelengths_in_nanometres = np.linspace(480, 880, 256)
wavelengths = wavelengths_in_nanometres * 1e-9
frequencies = convert_wavelength_and_frequency(wavelengths)
angles = np.linspace(0, 86, 64)

In [ ]:
default_oscillator_params = {
    "N": 1e26,
    "permittivity": 2.2,
    "lorentz_resonance_wavelength": 680,
    "lorentz_linewidth": 7.5e13,
}

# Setup

In [ ]:
fdtd = lumapi.FDTD()

In [ ]:
oscillator = LumericalOscillator(session=fdtd)

In [ ]:
formatter = format_stackrt(
    output_format="xarray_dataset",
    **{
        "variables": ["Rs", "Rp", "Ts", "Tp"],
        "add_absorption": False,
    }
)

In [ ]:
resources = {
    "lumerical_session": fdtd,
    "oscillator": oscillator,
    "simulation_mode": "stackrt",
    "formatter": formatter,
    "frequencies": frequencies,
    "angles": angles,
}

In [ ]:
constants = {
    "length_scale": 1e-9
}

In [ ]:
r = xyz.Runner(lopc_data, var_names=None, constants=constants, resources=resources)

## LOPC

In [ ]:
h_lopc = xyz.Harvester(runner=r, data_name=f"data/run_{run_number}/LOPC.nc")

In [ ]:
%%timeit
# check how long one run takes
lopc_data(**resources)

## Reference slabs

In [ ]:
h_ref = xyz.Harvester(runner=r, data_name=f"data/run_{run_number}/ref.nc")

# Data generation

In [ ]:
# combos = {
#     "passive_RI": [1.35],
#     "incident_medium_RI": [1.35],
#     "exit_medium_RI": [1.35],
#     "N": [1e26],
#     "permittivity": [2.2],
#     "lorentz_resonance_wavelength": [680],
#     "lorentz_linewidth": [7.5e13],
#     "num_periods": np.union1d(np.arange(1, 30, 1), np.arange(30, 51, 10)),
#     "passive_layer_thickness": np.arange(0, 311, 10),
#     "excitonic_layer_thickness": np.arange(0, 151, 10),
#     "remove_last_layer": [0], # True/False are not compatible with netCDF format
# }

## Layer thicknesses

### LOPC simulation

In [ ]:
for k, v in combos.items():
    print(f"{k}: length {len(v)}")

In [ ]:
estimate_combo_run_time(1.2, combos)/3600

In [ ]:
h_lopc.harvest_combos(combos)

In [ ]:
h_lopc.save_full_ds()

In [ ]:
# ds = h_lopc.full_ds.copy()

### Load LOPC dataset

In [ ]:
# # chunks for per-angle plots
# chunks = {
#     "frequency": 256,
#     "excitonic_layer_thickness": 16,
#     "passive_layer_thickness": 32,
#     "theta": 1,
#     "num_periods": 16,
# }

In [ ]:
# chunks for plotting or integrating over angle
chunks = {
    "frequency": 256,
    "excitonic_layer_thickness": 16,
    "passive_layer_thickness": 32,
    "theta": 16,
    "num_periods": 1,
}

#### Maths to work out good chunk sizes

In [ ]:
def chunk_size(chunks):
    return np.prod(list(chunks.values()))

In [ ]:
# Number of chunks
(combo_length(combos)*256*64) / chunk_size(chunks)

In [ ]:
# Do chunks contain at least a million data points?
chunk_size(chunks)/1e6

####

In [ ]:
ds = xr.open_mfdataset(
    f"data/run_{run_number}/LOPC.nc",
    engine=xarray_engine,
    lock=False,
    chunks=chunks,
)

# ds.equals(h_lopc.full_ds)

In [ ]:
# add derived attrs
ds = assign_derived_attrs(ds, per_oscillator=["Rs", "Rp", "R", "Ts", "Tp", "T", "As", "Ap", "A"])
# ds

In [ ]:
restack_plt_to_period = partial(
    restack,
    start_idxs=["passive_layer_thickness", "excitonic_layer_thickness"],
    end_idxs=["period", "excitonic_layer_thickness"],
)

In [ ]:
restacked_ds = restack_plt_to_period(ds)

### Reference slab simulation

In [ ]:
# useful variables
total_excitonic_thicknesses = np.unique(ds.total_excitonic_thickness)
total_passive_thicknesses = np.unique(ds.total_passive_thickness)
total_thicknesses = np.unique(ds.total_thickness)

In [ ]:
# compressed reference slab with the passive layer at the end
reference_slab_combos_1 = combos|{
    "num_periods": [1],
    "passive_layer_thickness": total_passive_thicknesses,
    "excitonic_layer_thickness": total_excitonic_thicknesses,
    "remove_last_layer": [0]
}

In [ ]:
# compressed reference slab without passive layer, AND filled reference slabs
reference_slab_combos_2 = combos|{
    "num_periods": [1],
    "passive_layer_thickness": [0],
    "excitonic_layer_thickness": np.union1d(total_excitonic_thicknesses, total_thicknesses),
    "remove_last_layer": [1]
}

In [ ]:
print(f"reference_slab_combos_1: {estimate_combo_run_time(1.2, reference_slab_combos_1)/3600} hours")
print(f"reference_slab_combos_2: {estimate_combo_run_time(1.2, reference_slab_combos_2)/3600} hours")

Compressed reference slab 0 takes too long because of the huge number of combinations. Also when the passive RI is equal to the incident/exit it makes no difference at all, therefore don't bother with it.

In [ ]:
# h_ref.harvest_combos(reference_slab_combos_1)
h_ref.harvest_combos(reference_slab_combos_2)

### Load reference slab dataset

In [ ]:
# useful variables
total_excitonic_thicknesses = np.unique(ds.total_excitonic_thickness)
total_passive_thicknesses = np.unique(ds.total_passive_thickness)
total_thicknesses = np.unique(ds.total_thickness)

In [ ]:
ref = xr.open_mfdataset(
    f"data/run_{run_number}/ref.nc",
    engine=xarray_engine,
    lock=False,
    # chunks={'theta': 16},
)

Note: `period=False` is an important option because otherwise it ends up a coordinate of `total_excitonic_thickness` and causes a conflict after binary operations with `ds`.

In [ ]:
ref = assign_derived_attrs(ref, period=False, total_excitonic_thickness=False, total_passive_thickness=False, total_thickness=False)
# ref

In [ ]:
# compressed reference slab without passive layer
crs_1 = (
    ref.sel(
        remove_last_layer=1,
        passive_layer_thickness=0,
        excitonic_layer_thickness=total_excitonic_thicknesses,
        drop=True,
    )
    .squeeze(drop=True)
    .rename(excitonic_layer_thickness="total_excitonic_thickness")
)

# # compressed reference slab with the passive layer at the end
# crs_0 = (
#     ref.sel(
#         remove_last_layer=0,
#         passive_layer_thickness=total_passive_thicknesses,
#         excitonic_layer_thickness=total_excitonic_thicknesses,
#         drop=True,
#     )
#     .squeeze(drop=True)
#     .rename(
#         excitonic_layer_thickness="total_excitonic_thickness",
#         passive_layer_thickness="total_passive_thickness",
#     )
# )

In [ ]:
# filled reference slab
frs_1 = (
    ref.sel(
        remove_last_layer=1,
        passive_layer_thickness=0,
        excitonic_layer_thickness=total_thicknesses,
        drop=True,
    )
    .squeeze(drop=True)
    .rename(excitonic_layer_thickness="total_thickness")
)

### Calculate derived variables

In [ ]:
polarised_attrs = ['Rs', 'Rp', 'Ts', 'Tp', 'As', 'Ap']

In [ ]:
absorptances = ['As', 'Ap', 'A', 'A_per_oscillator']

#### Normalise by CRS

Two ways to do this, either groupby:

In [ ]:
gb_tet = ds[absorptances[:3]].groupby('total_excitonic_thickness')

In [ ]:
# 'biology' absorptance enhancement factor: normalise by reference slab type 1: compressed reference slab w/o passive layer
norm_1 = gb_tet/crs_1

or reindex the reference slab to share coordinates with the LOPC:

In [ ]:
crs_1_like_ds = crs_1.sel(total_excitonic_thickness = ds.total_excitonic_thickness)

In [ ]:
norm_1.equals(ds/crs_1_like_ds)

Both methods can be accessed by the `enhancement_factor` helper function:

In [ ]:
norm_1 = enhancement_factor(ds, ref=crs_1, common_dim="total_excitonic_thickness", method="groupby")

Either way:

In [ ]:
# save/load to use disk memory
# norm_1.to_netcdf(f'data/run_{run_number}/norm_1.nc')

In [ ]:
# load from new kernel here
norm_1 = xr.open_mfdataset(
    f"data/run_{run_number}/norm_1.nc",
    chunks=chunks,
)

In [ ]:
restacked_norm_1 = restack_plt_to_period(norm_1)

#### Normalise by FRS

In [ ]:
gb_tt = ds[absorptances].groupby('total_thickness')

In [ ]:
# 'stuffed' absorptance enhancement factor: normalise by reference slab type 2: filled reference slab
with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    norm_2 = gb_tt/frs_1

In [ ]:
# # save/load to use disk memory
# norm_2.to_netcdf(f"data/run_{run_number}/norm_2.nc", engine=xarray_engine)

In [ ]:
# load from new kernel here
norm_2 = xr.open_mfdataset(
    f"data/run_{run_number}/norm_2.nc",
    chunks=chunks,
    engine=xarray_engine,
    lock=False,
)

In [ ]:
norm_2 = assign_derived_attrs(norm_2, absorption=False, unpolarised=False, per_oscillator=False)

In [ ]:
restacked_norm_2 = norm_2.stack(multiperiod=['passive_layer_thickness', 'excitonic_layer_thickness']).set_index(multiperiod=['period', 'excitonic_layer_thickness']).unstack()

In [ ]:
xs = {
    "theta": pnw.DiscreteSlider,
    "passive_layer_thickness": pnw.DiscreteSlider,
    "num_periods": pnw.DiscreteSlider,
}

In [ ]:
norm_2_int = norm_2.squeeze(drop=True).interactive

In [ ]:
norm_2_int.sel(**xs).A.hvplot(
    x="wavelength",
    by="excitonic_layer_thickness",
) * hv.HLine(1).opts(opts.HLine(line_dash='dotted'))

In [ ]:
norm_2_int.sel(**xs).A_per_oscillator.hvplot(
    x="wavelength",
    by="excitonic_layer_thickness",
    ylim=(0, 3),
) * hv.HLine(1).opts(opts.HLine(line_dash="dotted"))

In [ ]:
restacked_xs = xs
restacked_xs["period"] = restacked_xs.pop("passive_layer_thickness")

In [ ]:
thicknesses = [10, 20, 50, 100, 150]

restacked_norm_2.interactive.squeeze().sel(excitonic_layer_thickness=thicknesses, **restacked_xs).A_per_oscillator.hvplot(
    x="wavelength",
    by="excitonic_layer_thickness",
    ylim=(0, 5),
) * hv.HLine(1).opts(opts.HLine(line_dash='dotted'))

#### Difference with CRS

In [ ]:
gb_tet = ds[polarised_attrs].groupby('total_excitonic_thickness')

In [ ]:
# 'biology' absorptance enhancement factor: normalise by reference slab type 1: compressed reference slab w/o passive layer
diff_1 = gb_tet - crs_1

In [ ]:
# # save/load to use disk memory
# # actually this doesn't seem to be necessary because the visualisation are fast enough without
# diff_1.to_netcdf(f'data/run_{run_number}/diff_1.nc')

In [ ]:
# # load from new kernel here
# diff_1 = xr.open_mfdataset(
#     f"data/run_{run_number}/diff_1.nc",
#     chunks=chunks,
# )

In [ ]:
diff_1 = assign_derived_attrs(
    dataset=diff_1,
    unpolarised=True,
    absorption=False,
    period=False,
    total_excitonic_thickness=False,
    total_passive_thickness=False,
    total_thickness=False,
    N_tot=False,
    per_oscillator=["Rs", "Rp", "R", "Ts", "Tp", "T", "As", "Ap", "A"],
)

In [ ]:
restacked_diff_1 = diff_1.stack(multiperiod=['passive_layer_thickness', 'excitonic_layer_thickness']).set_index(multiperiod=['period', 'excitonic_layer_thickness']).unstack()

#### Difference with FRS

#### Integration over flat spectrum

In [ ]:
# Flat intensity profile

def constant(value=1):
    return np.vectorize(lambda x: value)

flat_spectrum = spectrum(constant(), ds.frequency, normalisation=1, array_name='Intensity')

flat_spectrum.hvplot('frequency')

In [ ]:
domain = (ds.stack(**{'dummy_idx': ['frequency', 'theta']})['dummy_idx'].unstack('dummy_idx'))

domain

In [ ]:
flat_spectrum_2D = spectrum(constant(), domain, normalisation=1, array_name='Intensity')

flat_spectrum_2D.hvplot(kind='quadmesh', x='frequency', y='theta')

In [ ]:
flat_spectrum_2D.integrate('frequency').integrate('theta')

##### Generate the data

In [ ]:
ds_flat_spectrum = (ds * flat_spectrum).integrate("frequency")
ds_flat_spectrum.to_netcdf(
    f"data/run_{run_number}/LOPC_flat_spectrum.nc", engine=xarray_engine
)

In [ ]:
ref_flat_spectrum = (ref * flat_spectrum).integrate("frequency")
ref_flat_spectrum.to_netcdf(
    f"data/run_{run_number}/ref_flat_spectrum.nc", engine=xarray_engine
)

###### Note: it is VERY IMPORTANT to integrate by frequency, not wavelength, because the normalisation is by frequency!

In [ ]:
temp = (ds*flat_spectrum).integrate('frequency').interactive.sel(theta=0, excitonic_layer_thickness=150, passive_layer_thickness=0).squeeze()

In [ ]:
temp.A.plot()

In [ ]:
temp_2 = (ds*flat_spectrum).integrate('wavelength').interactive.sel(theta=0, excitonic_layer_thickness=150, passive_layer_thickness=0).squeeze()

In [ ]:
temp_2.A.plot()

##### Load the data and define subsets

In [ ]:
ds_flat_spectrum = xr.open_dataset(
    f"data/run_{run_number}/LOPC_flat_spectrum.nc",
    engine=xarray_engine,
    lock=False,
)
ds_flat_spectrum = assign_derived_attrs(ds_flat_spectrum)

In [ ]:
restacked_ds_flat_spectrum = (
    ds_flat_spectrum.stack(
        multiperiod=["passive_layer_thickness", "excitonic_layer_thickness"]
    )
    .set_index(multiperiod=["period", "excitonic_layer_thickness"])
    .unstack()
)

In [ ]:
ref_flat_spectrum = xr.open_dataset(
    f"data/run_{run_number}/ref_flat_spectrum.nc",
    engine=xarray_engine,
    lock=False,
)
ref_flat_spectrum = assign_derived_attrs(ref_flat_spectrum, period=False, total_excitonic_thickness=False, total_passive_thickness=False, total_thickness=False)

In [ ]:
# Compressed reference slab without passive layer
crs_1_flat_spectrum = (
    ref_flat_spectrum.sel(
        remove_last_layer=1,
        passive_layer_thickness=0,
        excitonic_layer_thickness=total_excitonic_thicknesses,
        drop=True
    )
    .squeeze(drop=True)
    .rename(excitonic_layer_thickness="total_excitonic_thickness")
)

In [ ]:
norm_flat_spectrum = (ds_flat_spectrum.groupby('total_excitonic_thickness')/crs_1_flat_spectrum)#.drop_sel(excitonic_layer_thickness=0)

In [ ]:
restacked_norm_flat_spectrum = (
    norm_flat_spectrum.stack(
        multiperiod=["passive_layer_thickness", "excitonic_layer_thickness"]
    )
    .set_index(multiperiod=["period", "excitonic_layer_thickness"])
    .unstack()
)

In [ ]:
crs_1_fs_like_ds = crs_1_flat_spectrum.sel(
    total_excitonic_thickness=ds.total_excitonic_thickness
)

In [ ]:
norm_flat_spectrum2 = (ds_flat_spectrum.drop_sel(excitonic_layer_thickness=0)/crs_1_fs_like_ds.drop_sel(excitonic_layer_thickness=0))

In [ ]:
norm_flat_spectrum2 = (ds_flat_spectrum/crs_1_fs_like_ds).drop_sel(excitonic_layer_thickness=0)

#### Gaussian spectrum

In [ ]:
# Gaussian intensity profile

from scipy.stats import norm

spectrum(norm(loc=680, scale=100).pdf, ds.wavelength, normalisation=1, array_name='Intensity').hvplot()

#### Demo of a spectrum over wavelength AND angle

In [ ]:
def test_spectral_func(domain):
    return np.sin(domain.frequency*1e-14) * np.cos(domain.theta*np.pi/90)

In [ ]:
test_spectral_func(ds.stack(midx=["frequency", "theta"]).midx).unstack('midx').plot(x='frequency', y='theta')

In [ ]:
test_spectral_func(ds[["frequency", "theta"]]).plot(x='frequency', y='theta')

In [ ]:
spectrum(test_spectral_func, domain, normalisation=1).plot(x='frequency', y='theta')

In [ ]:
integrate_da(spectrum(test_spectral_func, domain, normalisation=1), ['frequency', 'theta'])

#### Checking the normalisation method is at least self-consistent

In [ ]:
xs = {"passive_layer_thickness": 100, "excitonic_layer_thickness": 10, "num_periods": 50}

In [ ]:
normalise_over_dim(ds.R, ['frequency', 'theta'], method=1).sel(xs).integrate(['frequency', 'theta']).compute()

In [ ]:
normalise_over_dim(ds.R, ['frequency', 'theta'], method=2).sel(xs).integrate(['frequency', 'theta']).compute()

### Load derived variables

In [ ]:
polarised_attrs = ['Rs', 'Rp', 'Ts', 'Tp', 'As', 'Ap']

In [ ]:
absorptances = ['As', 'Ap', 'A', 'A_per_oscillator']

In [ ]:
norm_1 = xr.open_mfdataset(
    f"data/run_{run_number}/norm_1.nc",
    chunks=chunks,
)

In [ ]:
restacked_norm_1 = restack_plt_to_period(norm_1)

In [ ]:
norm_2 = xr.open_mfdataset(
    f"data/run_{run_number}/norm_2.nc",
    chunks=chunks,
    engine=xarray_engine,
    lock=False,
)

In [ ]:
norm_2 = assign_derived_attrs(norm_2, absorption=False, unpolarised=False, per_oscillator=False)

In [ ]:
restacked_norm_2 = restack_plt_to_period(norm_2)

In [ ]:
gb_tet = ds[polarised_attrs].groupby('total_excitonic_thickness')

# 'biology' absorptance enhancement factor: normalise by reference slab type 1: compressed reference slab w/o passive layer
diff_1 = gb_tet - crs_1

diff_1 = assign_derived_attrs(
    dataset=diff_1,
    unpolarised=True,
    absorption=False,
    period=False,
    total_excitonic_thickness=False,
    total_passive_thickness=False,
    total_thickness=False,
    N_tot=False,
    per_oscillator=["Rs", "Rp", "R", "Ts", "Tp", "T", "As", "Ap", "A"],
)

restacked_diff_1 = restack_plt_to_period(diff_1)

In [ ]:
gb_tt = ds[polarised_attrs].groupby("total_thickness")

# 'stuffed' difference factor: difference with reference slab type 2: filled reference slab
diff_2 = gb_tt - frs_1

diff_2 = assign_derived_attrs(
    dataset=diff_2,
    unpolarised=True,
    absorption=False,
    period=True,  # reset period to only depend on two dims
    total_excitonic_thickness=False,
    total_passive_thickness=False,
    total_thickness=False,
    N_tot=False,
    per_oscillator=["Rs", "Rp", "R", "Ts", "Tp", "T", "As", "Ap", "A"],
)

# restacked_diff_2 = restack(
#     ds=diff_2,
#     start_idxs=[
#         "passive_layer_thickness",
#         "excitonic_layer_thickness",
#         "num_periods",
#         "remove_last_layer",
#     ],
#     end_idxs=[
#         "period",
#         "excitonic_layer_thickness",
#         "num_periods",
#         "remove_last_layer",
#     ],
# )

restacked_diff_2 = restack_plt_to_period(diff_2)

In [ ]:
ds_flat_spectrum = xr.open_dataset(
    f"data/run_{run_number}/LOPC_flat_spectrum.nc",
    engine=xarray_engine,
    lock=False,
)
ds_flat_spectrum = assign_derived_attrs(ds_flat_spectrum)

In [ ]:
restacked_ds_flat_spectrum = restack_plt_to_period(ds_flat_spectrum)

In [ ]:
ref_flat_spectrum = xr.open_dataset(
    f"data/run_{run_number}/ref_flat_spectrum.nc",
    engine=xarray_engine,
    lock=False,
)
ref_flat_spectrum = assign_derived_attrs(ref_flat_spectrum, period=False, total_excitonic_thickness=False, total_passive_thickness=False, total_thickness=False)

In [ ]:
# Compressed reference slab without passive layer
crs_1_flat_spectrum = (
    ref_flat_spectrum.sel(
        remove_last_layer=1,
        passive_layer_thickness=0,
        excitonic_layer_thickness=total_excitonic_thicknesses,
        drop=True
    )
    .squeeze(drop=True)
    .rename(excitonic_layer_thickness="total_excitonic_thickness")
)

In [ ]:
norm_flat_spectrum = (ds_flat_spectrum.groupby('total_excitonic_thickness')/crs_1_flat_spectrum)#.drop_sel(excitonic_layer_thickness=0)

In [ ]:
restacked_norm_flat_spectrum = restack_plt_to_period(norm_flat_spectrum)

#### Angle chunking for faster plots (unnecessary with new chunk scheme)

In [ ]:
angle_chunks = {
    "frequency": 256,
    "excitonic_layer_thickness": 4,
    "passive_layer_thickness": 4,
    "theta": 64, # loading the whole theta dimension at once means theta/wavelength plots are much faster
    "num_periods": 8,
}

In [ ]:
angle_ds = xr.open_mfdataset(
    f"data/run_{run_number}/LOPC.nc",
    engine=xarray_engine,
    lock=False,
    chunks=angle_chunks,
)

In [ ]:
# add derived attrs
angle_ds = assign_derived_attrs(angle_ds, per_oscillator=["Rs", "Rp", "R", "Ts", "Tp", "T", "As", "Ap", "A"])
# ds

In [ ]:
angle_gb_tet = angle_ds.groupby('total_excitonic_thickness')

In [ ]:
# 'biology' absorptance enhancement factor: normalise by reference slab type 1: compressed reference slab w/o passive layer
angle_norm_1 = angle_gb_tet/crs_1

# Plots

In [ ]:
unpolarised_RTA = ['R', 'T', 'A']
s_polarised_RTA = ['Rs', 'Ts', 'As']
unpolarised_po = ["R_per_oscillator", "T_per_oscillator", "A_per_oscillator"]
unpolarised_vars = unpolarised_RTA + unpolarised_po

In [ ]:
# WARNING: all these datasets will be fundamentally changed after this cell, to the extent that it can't be run twice
# For consistency, keep important calculations in the preceding section!
ds = pre_process_for_plots(ds)
restacked_ds = pre_process_for_plots(restacked_ds)
ref = pre_process_for_plots(ref)
crs_1 = pre_process_for_plots(crs_1)
frs_1 = pre_process_for_plots(frs_1)
norm_1 = pre_process_for_plots(norm_1)
restacked_norm_1 = pre_process_for_plots(restacked_norm_1)
norm_2 = pre_process_for_plots(norm_2)
restacked_norm_2 = pre_process_for_plots(restacked_norm_2)
diff_1 = pre_process_for_plots(diff_1)
restacked_diff_1 = pre_process_for_plots(restacked_diff_1)
diff_2 = pre_process_for_plots(diff_2)
restacked_diff_2 = pre_process_for_plots(restacked_diff_2)
ds_flat_spectrum = pre_process_for_plots(ds_flat_spectrum)
restacked_ds_flat_spectrum = pre_process_for_plots(restacked_ds_flat_spectrum)
ref_flat_spectrum = pre_process_for_plots(ref_flat_spectrum)
crs_1_flat_spectrum = pre_process_for_plots(crs_1_flat_spectrum)
norm_flat_spectrum = pre_process_for_plots(norm_flat_spectrum)
restacked_norm_flat_spectrum = pre_process_for_plots(restacked_norm_flat_spectrum)
# angle_ds = pre_process_for_plots(angle_ds)
# angle_norm_1 = pre_process_for_plots(angle_norm_1)

In [ ]:
period_dim = hv.Dimension("period", label="Λ", unit="nm")
wavelength_dim = hv.Dimension("wavelength", label="λ", unit="nm")
real_index_dim = hv.Dimension("n")
imag_index_dim = hv.Dimension("k", label="ϰ")

### Useful lines

#### Function to do linewidth maths

In [ ]:
def linewidth_calculator_factory(centre, linewidth):
    return partial(linewidth_calculator, centre=centre, linewidth=linewidth)

#### Lorentz lines

I want some sort of metric for 'near the resonance' and 'far from the resonance'. The natural unit of distance in this instance is the linewidth. The linewidth is given in rad/s so there need to be some conversions to get the equivalent lines in the plots by wavelength, but they are roughly symmetrical around the peak wavelength.

Based on the plots of the refractive index below, I think I will consider 'near' to be 'within two linewidths', and 'far' to be 'at least four linewidths away'.

In [ ]:
# resonance_line = hv.VLine(680, label='LO resonance wavelength').opts(line_dash='dotted')

# Convert from rad/s to Hz
lorentz_linewidth_frequency = default_oscillator_params["lorentz_linewidth"] / (2*np.pi)

In [ ]:
lorentz_line_frequency = linewidth_calculator_factory(convert_wavelength_and_frequency(680e-9), lorentz_linewidth_frequency)

def lorentz_line_wavelength(x=None):
    x = -x if x is not None else x
    return convert_wavelength_and_frequency(lorentz_line_frequency(x))

In [ ]:
def lorentz_vlines(x=0, scale=1, mode='wavelength', **kwargs):
    if mode == 'wavelength':
        line_func = lorentz_line_wavelength
    elif mode == 'frequency':
        line_func = lorentz_line_frequency
    else:
        raise TypeError(f"mode should be 'wavelength' or 'frequency', not {mode}")
        
    match x:
        case [*xs]:
            line_pos = [line_func(x)/scale for x in xs]
        case x:
            line_pos = line_func(x)/scale
            
    return vlines(line_pos, **kwargs)

### Useful functions

#### Select a wavelength or wavelength range based on the distance from the resonance in linewidths.

In [ ]:
def select_lorentz_line(da, lorentz_line=0, window_radius=0):
    if window_radius == 0:
        wavelength = lorentz_line_wavelength(lorentz_line) * 1e9
        wavelength_sel_method = "nearest"
    else:
        wavelength = slice(
            lorentz_line_wavelength(lorentz_line - window_radius) * 1e9,
            lorentz_line_wavelength(lorentz_line + window_radius) * 1e9,
        )
        wavelength_sel_method = None
    da = da.sel(wavelength=wavelength, method=wavelength_sel_method)
    
    return da

#### Plot a comparison of the reflectance and absorptance of the LOPC with that of the reference slab.

In [ ]:
opts_R = [opts.Curve(color=blue, ylim=(0,1)), opts.Image(cmap='viridis', clim=(0,1)), opts.QuadMesh(cmap='viridis', clim=(0,1))]

def plot_R(variable="R", dataset=None, label_field="long_name", label_append=None, **hvplot_kwargs):
    plot = plot_var(variable, dataset, label_field, label_append, **hvplot_kwargs)
    plot.opts(*opts_R)
    return plot

# # test
# plot_R(dataset=restacked_ds.sel(period=200, excitonic_layer_thickness=20, num_periods=10).squeeze(), x="wavelength", y="theta").opts(cmap="cividis", clim=(None, None))

In [ ]:
opts_T = [opts.Curve(color=yellow, ylim=(0,1)), opts.Image(cmap='cividis', clim=(0,1)), opts.QuadMesh(cmap='cividis', clim=(0,1))]

def plot_T(variable="T", dataset=None, label_field="long_name", label_append=None, **hvplot_kwargs):
    plot = plot_var(variable, dataset, label_field, label_append, **hvplot_kwargs)
    plot.opts(*opts_T)
    return plot

In [ ]:
opts_A = [opts.Curve(color=red, ylim=(0,1)), opts.Image(cmap='inferno', clim=(0,1)), opts.QuadMesh(cmap='inferno', clim=(0,1))]

def plot_A(variable="A", dataset=None, label_field="long_name", label_append=None, **hvplot_kwargs):
    plot = plot_var(variable, dataset, label_field, label_append, **hvplot_kwargs)
    plot.opts(*opts_A)
    return plot

In [ ]:
def plot_vars_to_funcs(plot_vars):
    var_func_mapping = {
        "R": plot_R,
        "T": plot_T,
        "A": plot_A,
        "Rs": partial(plot_R, variable="Rs"),
        "Ts": partial(plot_T, variable="Ts"),
        "As": partial(plot_A, variable="As"),
        "Rp": partial(plot_R, variable="Rp"),
        "Tp": partial(plot_T, variable="Tp"),
        "Ap": partial(plot_A, variable="Ap"),
        "R_per_oscillator": partial(plot_R, variable="R_per_oscillator"),
        "T_per_oscillator": partial(plot_T, variable="T_per_oscillator"),
        "A_per_oscillator": partial(plot_A, variable="A_per_oscillator"),
        "Rs_per_oscillator": partial(plot_R, variable="Rs_per_oscillator"),
        "Ts_per_oscillator": partial(plot_T, variable="Ts_per_oscillator"),
        "As_per_oscillator": partial(plot_A, variable="As_per_oscillator"),
        "Rp_per_oscillator": partial(plot_R, variable="Rp_per_oscillator"),
        "Tp_per_oscillator": partial(plot_T, variable="Tp_per_oscillator"),
        "Ap_per_oscillator": partial(plot_A, variable="Ap_per_oscillator"),
    }

    plot_funcs = []
    for var in plot_vars:
        try:
            func = var_func_mapping[var]
        except KeyError:
            func = partial(plot_var, variable=var)
        plot_funcs.append(func)

    return plot_funcs

In [ ]:
# new version
def plot_RTA(
    period,
    excitonic_layer_thickness,
    num_periods,
    theta,
    title="",
    include=["LOPC", "CRS_1"],
    plot_vars=["R", "T", "A"],
    label_override=None,
    label_append=None,
):
    label_field = None  # for debugging
    label_append = "" if label_append is None else label_append

    P = period
    t = excitonic_layer_thickness
    N = num_periods

    plot_funcs = [
        partial(func, x="wavelength", label_field=label_field)
        for func in plot_vars_to_funcs(plot_vars)
    ]
    curves = []
    if "LOPC" in include:
        lopc_label = " (LOPC)" if label_override is None else label_override
        lopc_label += label_append
        lopc_sel = restacked_ds.sel(
            period=P, excitonic_layer_thickness=t, num_periods=N
        ).squeeze()
        lopc_sel = sel_or_integrate(lopc_sel, "theta", theta, normalisation=1)
        lopc_curves = [
            plot_func(dataset=lopc_sel, label_append=lopc_label).opts(line_dash="solid")
            for plot_func in plot_funcs
        ]
        curves += lopc_curves
    if "CRS_1" in include:
        crs_1_label = " (CRS)" if label_override is None else label_override
        crs_1_label += label_append
        crs_1_sel = crs_1.sel(total_excitonic_thickness=t * N).squeeze()
        crs_1_sel = sel_or_integrate(crs_1_sel, "theta", theta, normalisation=1)
        crs_1_curves = [
            plot_func(dataset=crs_1_sel, label_append=crs_1_label).opts(
                line_dash="dashed"
            )
            for plot_func in plot_funcs
        ]
        curves += crs_1_curves
    if "FRS_1" in include:
        frs_1_label = " (FRS)" if label_override is None else label_override
        frs_1_label += label_append
        frs_1_sel = frs_1.sel(total_thickness=(P + t) * N).squeeze()
        frs_1_sel = sel_or_integrate(frs_1_sel, "theta", theta, normalisation=1)
        frs_1_curves = [
            plot_func(dataset=frs_1_sel, label_append=frs_1_label).opts(
                line_dash="dotted"
            )
            for plot_func in plot_funcs
        ]
        curves += frs_1_curves

    overlay = hv.Overlay(curves).opts(
        opts.Curve(
            ylim=(0, 1),
            ylabel="Intensity",
            title=f"{title}{coordinate_string(period=P, excitonic_layer_thickness=t, num_periods=N, theta=theta)}",
        ),
    )

    return overlay


# # test
# display(
#     plot_RTA(200, 40, 20, 0, "test\n", include=["LOPC", "CRS_1", "FRS_1"]).opts(
#         legend_position="right"
#     )
# )

# display(
#     plot_RTA(
#         200,
#         40,
#         20,
#         (10, 50),
#         "test RA only\n",
#         include=["LOPC", "CRS_1", "FRS_1"],
#         plot_vars=["R", "A"],
#     ).opts(opts.Overlay(legend_position="right"))
# )

# display(
#     plot_RTA(
#         200,
#         40,
#         20,
#         75,
#         "test\n",
#         include=["LOPC"],
#         plot_vars=["R_per_oscillator", "A_per_oscillator"],
#         label_append=" test",
#         label_override="OVERRIDDEN",
#     ).opts(opts.Curve(ylim=(None, None)), opts.Overlay(legend_position="right"))
# )

#### Plot a comparison of normal incidence to integrated

In [ ]:
def plot_comparison(*comparison_params: tuple[dict, list["opts"]], plot_func=plot_RTA, **shared_params):
    param_opts = [(shared_params|comp_params, comp_opts) for comp_params, comp_opts in comparison_params]
    plots = [plot_func(**comp_params).opts(*comp_opts) for comp_params, comp_opts in param_opts]
    return plots

In [ ]:
def compare_RTA(*args, opts_cycle=None, plot_func=plot_RTA, **shared_params):
    default_opts = [[opts.Curve(line_dash=style)] for style in ["solid", "dashed", "dotted", "dotdash", "dashdot"]]
    opts_cycle = default_opts if opts_cycle is None else opts_cycle
    
    # comparison_params = list(zip(args, opts_cycle))
    
    plots = plot_comparison(*zip(args, opts_cycle), plot_func=plot_func, **shared_params)
    overlay = hv.Overlay(plots).opts(opts.Overlay(legend_position="right"))
                                     
    return overlay

# # test
# shared_params = {
#     "period": 250,
#     "excitonic_layer_thickness": 70,
#     "num_periods": 30,
#     "include": ["LOPC"],
# }
# compare_RTA({"theta": (0, 75), "label_override": " (integrated)"}, {"theta": 0, "label_override": " (θ = 0)"}, **shared_params)

In [ ]:
compare_RTA_normal_vs_integrated = partial(compare_RTA, {"theta": (0, 45), "label_override": " (integrated)"}, {"theta": 0, "label_override": " (θ = 0)"}, include= ["LOPC"],)

# # test
# shared_params = {
#     "period": 250,
#     "excitonic_layer_thickness": 70,
#     "num_periods": 30,
# }
# compare_RTA_normal_vs_integrated(**shared_params)

#### Plot the RTA of the structures in 2D

In [ ]:
def plot_RTA_2D(
    period,
    excitonic_layer_thickness,
    num_periods,
    theta=(0, 75),
    title="",
    include=["LOPC", "CRS_1"],
):
    P = period
    t = excitonic_layer_thickness
    N = num_periods

    plots = []
    if "LOPC" in include:
        lopc_sel = restacked_ds.sel(
            period=P, excitonic_layer_thickness=t, num_periods=N
        ).squeeze()
        lopc_sel = lopc_sel.sel(theta=slice(*theta))
        plots.append(
            lopc_sel["R"]
            .hvplot(kind="image", x="wavelength", y="theta", title="Reflectance (LOPC)")
            .opts(opts.Image(cmap="viridis"))
        )
        plots.append(
            lopc_sel["T"]
            .hvplot(kind="image", x="wavelength", y="theta", title="Transmittance (LOPC)")
            .opts(opts.Image(cmap="cividis"))
        )
        plots.append(
            lopc_sel["A"]
            .hvplot(kind="image", x="wavelength", y="theta", title="Absorptance (LOPC)")
            .opts(opts.Image(cmap="inferno"))
        )
    if "CRS_1" in include:
        crs_1_sel = crs_1.sel(total_excitonic_thickness=t * N).squeeze()
        crs_1_sel = crs_1_sel.sel(theta=slice(*theta))
        plots.append(
            crs_1_sel["R"]
            .hvplot(kind="image", x="wavelength", y="theta", title="Reflectance (CRS)")
            .opts(opts.Image(cmap="viridis"))
        )
        plots.append(
            crs_1_sel["T"]
            .hvplot(kind="image", x="wavelength", y="theta", title="Transmittance (CRS)")
            .opts(opts.Image(cmap="cividis"))
        )
        plots.append(
            crs_1_sel["A"]
            .hvplot(kind="image", x="wavelength", y="theta", title="Absorptance (CRS)")
            .opts(opts.Image(cmap="inferno"))
        )
    if "FRS_1" in include:
        frs_1_sel = frs_1.sel(total_thickness=(P + t) * N).squeeze()
        frs_1_sel = frs_1_sel.sel(theta=slice(*theta))
        plots.append(
            frs_1_sel["R"]
            .hvplot(kind="image", x="wavelength", y="theta", title="Reflectance (FRS)")
            .opts(opts.Image(cmap="viridis"))
        )
        plots.append(
            frs_1_sel["T"]
            .hvplot(kind="image", x="wavelength", y="theta", title="Transmittance (FRS)")
            .opts(opts.Image(cmap="cividis"))
        )
        plots.append(
            frs_1_sel["A"]
            .hvplot(kind="image", x="wavelength", y="theta", title="Absorptance (FRS)")
            .opts(opts.Image(cmap="inferno"))
        )

    layout = hv.Layout(plots).opts(
        opts.Image(
            clim=(0, 1),
            clabel="Intensity",
        ),
        opts.Layout(
            title=f"{title}{coordinate_string(period=P, excitonic_layer_thickness=t, num_periods=N)}",
        ),
    )

    return layout

# # test
# display(plot_RTA_2D(200, 40, 20, (0, 90), "test\n", include=["LOPC", "CRS_1", "FRS_1"]).opts(opts.Image(frame_width=200)).cols(3))

# display(plot_RTA_2D(200, 40, 20, (10, 50), "test\n", include=["LOPC", "CRS_1", "FRS_1"]).opts(opts.Image(frame_width=200)).cols(3))

#### Plot an enhancement factor.

In [ ]:
def plot_ef(
    variable,
    dataset,
    sel=None,
    sel_method=None,
    title="",
    *,
    x="wavelength",
    y=None,
):
    sel = {} if sel is None else sel
    da = dataset[variable].sel(**sel, method=sel_method).squeeze()
    if y is None:
        plot = da.hvplot(x=x, label=f"{variable} enhancement factor")
        plot *= hv.HLine(1).opts(line_dash="dotted")
    else:
        plot = da.hvplot(
            kind="image",
            x=x,
            y=y,
            label=f"{variable} enhancement factor",
            clim=(0.5, 1.5),
        )
    plot = plot.opts(
        opts.Curve(
            title=f"{title}{coordinate_string(**sel)}",
        ),
        opts.Overlay(
            title=f"{title}{coordinate_string(**sel)}",
        ),
    )

    return plot


# # test
# sel_1 = {"period": 200, "excitonic_layer_thickness": 40, "num_periods": 10, "theta": 30}
# sel_2 = {"period": 200, "excitonic_layer_thickness": 40, "num_periods": 10, "theta": 0}
# sel_3 = {"period": 200, "excitonic_layer_thickness": 40, "num_periods": 10}
# display(
#     (
#         plot_ef("As", restacked_norm_1, sel_1, "nearest", "test\n")
#         + plot_ef("As", restacked_norm_2, sel_2, title="test2\n")
#     ).cols(1)
# )
# display(
#     plot_ef("As", restacked_norm_1, sel_3, title="test3\n", x="theta", y="wavelength").opts(clim=(0, 2), cmap="RdBu_r")
# )

#### Test plot_optimum_over_dim

In [ ]:
foo, bar = plot_optimum_over_dim(restacked_ds.A.sel(theta=0, wavelength=660, method="nearest"), "period", "excitonic_layer_thickness", "num_periods", "max")

In [ ]:
foo, bar = plot_optimum_over_dim(integrate_da(restacked_ds.A, "theta", normalisation=1).sel(wavelength=660, method="nearest"), "period", "excitonic_layer_thickness", "num_periods", "max")

In [ ]:
foo

In [ ]:
bar

#### Find and plot the min or max over any dimension.

In [ ]:
def wrapped_2D_plot(
    variable,
    dataset,
    optimise="max",
    lorentz_line=0,
    window_radius=0,
    theta=0,
    cmap="viridis",
    period_start=None,
    period_stop=None,
    integrate_angle=None,
    extra_plots=["RTA_normal", "RTA_int", "norm_1_normal", "norm_1_int"],
    dim=None,  # automatically assign if dataset recognised
):
    plots = []

    if str(dataset) == "restacked_ds":
        # the drop_sel is important for avoiding the most common degenerate cases
        dataset = restacked_ds.drop_sel({"excitonic_layer_thickness": 0})
        dim = "period"

    if str(dataset) == "restacked_norm_1":
        # the drop_sel is important for avoiding the most common degenerate cases
        dataset = restacked_norm_1.drop_sel({"excitonic_layer_thickness": 0})
        dim = "period"

    if str(dataset) == "restacked_diff_1":
        # the drop_sel is important for avoiding the most common degenerate cases
        dataset = restacked_diff_1.drop_sel({"excitonic_layer_thickness": 0})
        dim = "period"

    da = dataset[variable]

    if not integrate_angle:
        da = da.sel(theta=theta, method="nearest")
    else:  # integrate_angle must be a float, so that (theta, integrate_angle) is a slice syntax
        da = da.sel(theta=slice(theta, integrate_angle))
        da = integrate_da(da, "theta", weighting=1, normalisation=1)

    if period_start < period_stop:
        da = da.sel(period=slice(period_start, period_stop))
    else:  # otherwise no data is selected and everything breaks
        da = da.sel(period=slice(period_start, None))
    da = select_lorentz_line(da, lorentz_line=lorentz_line, window_radius=window_radius)

    vline_locs = [0]

    if window_radius == 0:
        wavelength = float(da.wavelength)
        title = f"{optimise.capitalize()}imum {variable} at {wavelength:.0f} nm"
        if lorentz_line != 0:  # don't put two lines over the resonance
            vline_locs.append(lorentz_line)
    else:
        wavelength_start = float(da.wavelength[0])
        wavelength_stop = float(da.wavelength[-1])
        # make it easier to compare values
        da = integrate_da(
            da, "wavelength", weighting=1, normalisation=1
        )  # replaces the below two lines
        # da = normalise_over_dim(da, "wavelength", 1)
        # da = da.integrate("wavelength")
        title = f"{optimise.capitalize()}imum integrated {variable} between {wavelength_start:.0f} and {wavelength_stop:.0f} nm"
        vline_locs.append(lorentz_line - window_radius)
        vline_locs.append(lorentz_line + window_radius)

    plot_1, optimum_coords = plot_optimum_over_dim(
        da,
        dim=dim,
        x="excitonic_layer_thickness",
        y="num_periods",
        optimise=optimise,
    )

    P = float(optimum_coords["period"])
    t = float(optimum_coords["excitonic_layer_thickness"])
    N = float(optimum_coords["num_periods"])
    try:  # this should work if not integrating over theta
        th = float(optimum_coords["theta"])
    except:  # probably the problem is that theta doesn't exist because I integrated over it already
        th = (theta, integrate_angle)
    lorentz_lines = lorentz_vlines(vline_locs, scale=1e-9, mode="wavelength").opts(
        opts.VLine(line_color=green, line_dash="dotted"),
    )

    # give the resonance line a special colour
    lorentz_lines.VLine.I.opts(opts.VLine(line_color=yellow))

    plot_1.opts(
        opts.QuadMesh(cmap=cmap),
        opts.Points(color="red"),
        opts.Overlay(title=f"{title}\nOptimal period: {P:.0f}"),
    )

    plots.append(plot_1)

    if "RTA_normal" in extra_plots:  # plot RTA at theta=0
        new_plot = plot_RTA(
            period=P, excitonic_layer_thickness=t, num_periods=N, theta=0
        )
        new_plot *= lorentz_lines

        plots.append(new_plot)

    if "RTA_int" in extra_plots:  # plot RTA at theta OR integrating over theta
        new_plot = plot_RTA(
            period=P, excitonic_layer_thickness=t, num_periods=N, theta=th
        )
        new_plot *= lorentz_lines
        plots.append(new_plot)

    if "norm_1_normal" in extra_plots:  # plot enhancement factor at theta=0
        sel = {
            "period": P,
            "excitonic_layer_thickness": t,
            "num_periods": N,
            "theta": 0,
        }
        new_plot = plot_ef(variable="A", dataset=restacked_norm_1, sel=sel)
        new_plot *= lorentz_lines

        plots.append(new_plot)

    if (
        "norm_1_int" in extra_plots
    ):  # plot enhancement factor at theta OR integrating over theta
        try:  # this should work if not integrating over theta
            sel = {
                "period": P,
                "excitonic_layer_thickness": t,
                "num_periods": N,
                "theta": th,
            }
            new_plot = plot_ef(variable="A", dataset=restacked_norm_1, sel=sel)
            new_plot *= lorentz_lines
        except:  # if integrating, we need to do the integral *before* normalising
            ds_int = sel_or_integrate(ds, dim="theta", val=th)
            crs_1_int = sel_or_integrate(crs_1, dim="theta", val=th)
            norm = enhancement_factor(
                ds_int,
                ref=crs_1_int,
                common_dim="total_excitonic_thickness",
                method="groupby",
            )
            restacked_norm = restack_plt_to_period(norm)
            # replaces the lines below
            #             # this should all get separated out into its own function
            #             crs_1_like_ds = crs_1.sel(
            #                 total_excitonic_thickness=ds.total_excitonic_thickness
            #             )

            #             ds_int = sel_or_integrate(ds, dim="theta", val=th)
            #             crs_1_int = sel_or_integrate(crs_1_like_ds, dim="theta", val=th)
            #             norm = ds_int / crs_1_int
            #             restacked_norm = norm.stack(multiperiod=['passive_layer_thickness', 'excitonic_layer_thickness']).set_index(multiperiod=['period', 'excitonic_layer_thickness']).unstack()

            sel = {"period": P, "excitonic_layer_thickness": t, "num_periods": N}
            new_plot = plot_ef(variable="A", dataset=restacked_norm, sel=sel)
            new_plot *= lorentz_lines
            sel["theta"] = th
            new_plot = new_plot.opts(opts.Overlay(title=f"{coordinate_string(**sel)}"))
        plots.append(new_plot)

    return hv.Layout(plots).cols(1)

#### Plot the E-field, overlayed with the refractive index profile and layer boundaries.

In [ ]:
# sometimes this errors on the first call for some reason
fdtd = lumapi.FDTD()

In [ ]:
oscillator = LumericalOscillator(fdtd)

In [ ]:
plot_field(
    680,
    lumerical_session=fdtd,
    oscillator=oscillator,
    ri_lower=1.35,
    ri_upper=1.6,
    excitonic_layer_thickness=30,
    passive_layer_thickness=210,
    num_periods=10,
).opts(opts.VSpan(color='gray'))

In [ ]:
def wrap_plot_field(
    wavelength, excitonic_layer_thickness, passive_layer_thickness, num_periods
):
    coords = {
        "λ": wavelength,
        "Excitonic layer thickness": excitonic_layer_thickness,
        "Passive layer thickness": passive_layer_thickness,
        "Number of periods": num_periods,
    }

    title = f"{coordinate_string(**coords)}"

    return plot_field(
        wavelength=wavelength,
        lumerical_session=fdtd,
        oscillator=oscillator,
        ri_lower=1.35,
        ri_upper=1.6,
        excitonic_layer_thickness=excitonic_layer_thickness,
        passive_layer_thickness=passive_layer_thickness,
        num_periods=num_periods,
    ).opts(opts.Curve(title=title, ylim=(0,None)), opts.VSpan(color="gray"))

In [ ]:
# an example of what this can do
pn.interact(
    wrap_plot_field,
    wavelength=(480, 880),
    excitonic_layer_thickness=(10, 200),
    passive_layer_thickness=(0, 300),
    num_periods=(1, 50),
)

## Refractive index

In [ ]:
# first run cells defining fdtd, oscillator, default_oscillator_params
with lumapi.FDTD() as fdtd:
    oscillator = LumericalOscillator(session=fdtd)
    oscillator_index = LOPC.LOPC(lumerical_session=fdtd, oscillator=oscillator, **default_oscillator_params).oscillator.index(frequencies)

#### Facts about the RI

In [ ]:
oscillator_index.min()

In [ ]:
oscillator_index.max()

In [ ]:
oscillator_index.min() - 1.35

In [ ]:
oscillator_index.max() - 1.35

In [ ]:
np.argmax(np.abs(oscillator_index))

In [ ]:
np.argmin(np.abs(oscillator_index))

In [ ]:
# wavelength range between max and min
wavelengths_in_nanometres[np.argmin(oscillator_index):np.argmax(oscillator_index)]

#### Basic plots

Plot the refractive index of the Lorentz oscillator against wavelength.

In [ ]:
(
    (
        hv.Layout(
            complex_elements(
                (wavelengths_in_nanometres, oscillator_index),
                wavelength_dim,
                element=hv.Curve,
                auto_label="group",
                label="Lorentz Oscillator",
            ).values()
        )
        * lorentz_vlines([-4, -2, -1, 0, 1, 2, 4], scale=1e-9)
    )
    .redim("Curve.Real", y=real_index_dim)
    .redim("Curve.Imaginary", y=imag_index_dim)
    .opts(opts.Curve(width=600), opts.VLine(line_dash="dotted"))
    .cols(1)
)

Plot the refractive index of the Lorentz oscillator against frequency.

In [ ]:
(
    (
        hv.Layout(
            complex_elements(
                (frequencies, oscillator_index),
                "f (Hz)",
                element=hv.Curve,
                auto_label="group",
                label="Lorentz Oscillator",
            ).values()
        )
        * lorentz_vlines([-4, -2, -1, 0, 1, 2, 4], mode='frequency')
    )
    .redim("Curve.Real", y=real_index_dim)
    .redim("Curve.Imaginary", y=imag_index_dim)
    .opts(opts.Curve(width=600), opts.VLine(line_dash="dotted"))
    .cols(1)
)

Plot the refractive index of the Lorentz oscillator against wavelength on one axis.

In [ ]:
(
    hv.Overlay(
        complex_elements(
            (wavelengths_in_nanometres, oscillator_index),
            wavelength_dim,
            element=hv.Curve,
            auto_label="label",
            group="Lorentz Oscillator",
        ).values()
    )
    .redim('Curve.Lorentz_Oscillator.Real', y=real_index_dim)
    .redim('Curve.Lorentz_Oscillator.Imag', y=imag_index_dim)
    .opts(opts.Curve(width=600))
)

Plot the refractive index of the Lorentz oscillator against wavelength on two axes.

In [ ]:
(
    hv.Overlay(
        complex_elements(
            (wavelengths_in_nanometres, oscillator_index),
            wavelength_dim,
            element=hv.Curve,
            auto_label="label",
            group="Lorentz Oscillator",
        ).values()
    )
    .redim('Curve.Lorentz_Oscillator.Real', y=real_index_dim)
    .redim('Curve.Lorentz_Oscillator.Imag', y=imag_index_dim)
    .opts(opts.Curve(width=600), opts.Curve('Imaginary', hooks=[plot_secondary]))
)

#### Measured thylakoid membrane data

In [ ]:
# Get real and imaginary refractive index data of thylakoid membrance
thyl_real_data = np.loadtxt("data/Chl_real.dat", delimiter="\t", dtype="f")
thyl_imag_data = np.loadtxt("data/Chl_img.dat", delimiter="\t", dtype="f")

In [ ]:
osc_n_curve = hv.Curve(
    (wavelengths_in_nanometres, oscillator_index.real),
    kdims=[wavelength_dim],
    vdims=[real_index_dim],
    # group="Lorentz Oscillator",
    label="Lorentz Oscillator, Real",
)
osc_k_curve = hv.Curve(
    (wavelengths_in_nanometres, oscillator_index.imag),
    kdims=[wavelength_dim],
    vdims=[imag_index_dim],
    # group="Lorentz Oscillator",
    label="Lorentz Oscillator, Imaginary",
)#.opts(hooks=[plot_secondary])

In [ ]:
thyl_n_curve = hv.Curve(
    (thyl_real_data[:, 0], thyl_real_data[:, 1]),
    kdims=[wavelength_dim],
    vdims=[real_index_dim],
    # group="Thylakoid Membrane",
    label="Thylakoid Membrane, Real",
)
thyl_k_curve = hv.Curve(
    (thyl_imag_data[:, 0], thyl_imag_data[:, 1]),
    kdims=[wavelength_dim],
    vdims=[imag_index_dim],
    # group="Thylakoid Membrane",
    label="Thylakoid Membrane, Imaginary",
).opts(hooks=[plot_secondary])

In [ ]:
n_curves = (osc_n_curve * thyl_n_curve)
k_curves = (osc_k_curve * thyl_k_curve).opts(opts.Curve(line_dash='dashed', hooks=[plot_secondary]))

In [ ]:
# Even though these symbols aren't used, the options are applied to the underlying curves!
lo_curves = (osc_n_curve * osc_k_curve).opts(opts.Curve(color=blue))

In [ ]:
tm_curves = (thyl_n_curve * thyl_k_curve).opts(opts.Curve(color=red))

In [ ]:
tm_curves.opts(
    opts.Curve(width=800),
    opts.Overlay(
        legend_position="top_right", legend_opts={"background_fill_alpha": 0.5}
    ),
    clone=True,
)

In [ ]:
(n_curves * k_curves).opts(
    opts.Curve(width=800),
    opts.Overlay(
        legend_position="right", legend_opts={"background_fill_alpha": 0.5}
    ),
    clone=True,
)

#### Interpolated thylakoid membrane data

In [ ]:
# Interpolate the refractive index data
thyl_real = np.interp(wavelengths_in_nanometres, thyl_real_data[:,0], thyl_real_data[:,1])
thyl_imag = np.interp(wavelengths_in_nanometres, thyl_imag_data[:,0], thyl_imag_data[:,1])

# Combine into complex refractive index of the membrane
n_M = thyl_real + 1j*thyl_imag

In [ ]:
interp_curves = complex_elements(
    (wavelengths_in_nanometres, n_M),
    auto_label="group",
    label="Thylakoid membrane (interpolated)",
)

(
    interp_curves["Real"]
    * hv.Scatter(thyl_n_curve)
    * (
        interp_curves["Imaginary"].opts(hooks=[plot_secondary])
        * hv.Scatter(thyl_k_curve).opts(hooks=[plot_secondary])
    )
).opts(
    opts.Curve(width=800, color=blue),
    opts.Scatter(color=red),
    opts.Overlay(legend_position="right"),
)

In [ ]:
# Interpolate using InterpolatedIndex class
from multilayer_simulator.material import InterpolatedIndex

In [ ]:
from scipy import interpolate

In [ ]:
real_data_frequencies = convert_wavelength_and_frequency(thyl_real_data[::-1][:,0]*1e-9)
real_data_indexes = thyl_real_data[::-1][:,1]

thyl_real_index = InterpolatedIndex(real_data_frequencies, real_data_indexes, interpolate.interp1d, interp_kwargs={'bounds_error': False, 'fill_value': (real_data_indexes[0], real_data_indexes[-1])})

In [ ]:
hv.Curve((thyl_real_data[:,0], thyl_real_index._index_function_real(real_data_frequencies[::-1])))#.opts(ylim=(0, 1.65))

In [ ]:
imag_data_frequencies = convert_wavelength_and_frequency(thyl_imag_data[::-1][:,0]*1e-9)
imag_data_indexes = thyl_imag_data[::-1][:,1]

thyl_imag_index = InterpolatedIndex.from_scipy_method(imag_data_frequencies, imag_data_indexes, 'interp1d', interp_kwargs={'bounds_error': False, 'fill_value': (imag_data_indexes[0], imag_data_indexes[-1])})

In [ ]:
hv.Curve((thyl_imag_data[:,0], thyl_imag_index._index_function_real(imag_data_frequencies[::-1])))#.opts(ylim=(0, 1.65))

In [ ]:
def thyl_complex_index(frequencies, component, **kwargs):
    return thyl_real_index._index_function_real(frequencies) + 1j*thyl_imag_index._index_function_real(frequencies)

In [ ]:
from multilayer_simulator.material import CallableIndex

thyl_index = CallableIndex(thyl_complex_index)

In [ ]:
(
    hv.Overlay(
        complex_elements(
            (wavelengths_in_nanometres, thyl_index.index(frequencies)),
            auto_label="label"
        ).values()
    ).opts(opts.Curve(width=600))
)

In [ ]:
index_plot = hv.NdLayout(
    complex_elements(
        (wavelengths_in_nanometres, thyl_index.index(frequencies)), label="Interpolated"
    )
) * hv.NdLayout(
    {
        "Real": hv.Scatter(
            (thyl_real_data[:, 0], thyl_real_data[:, 1]), label="Measured"
        ),
        "Imaginary": hv.Scatter(
            (thyl_imag_data[:, 0], thyl_imag_data[:, 1]), label="Measured"
        ),
    }
)

In [ ]:
index_plot.opts(opts.Curve(width=600), opts.Scatter(color=red), opts.Overlay(legend_position='right')).cols(1)

## Emergence of PBG (1D plots of RTA)

### Interactive plot of RTA

In [ ]:
xs_1D = {
    "excitonic_layer_thickness": pnw.DiscreteSlider,
    "theta": pnw.DiscreteSlider,
    "num_periods": pnw.DiscreteSlider,
    "passive_layer_thickness": pnw.DiscreteSlider,
}

ds_sel = ds[s_polarised_RTA].interactive(loc="left").sel(**xs_1D).squeeze(drop=True)

In [ ]:
# Get a handle on the widgets
widgets = {k: w for k, w in zip(xs_1D, ds_sel.widgets())}

In [ ]:
(
    ds_sel.hvplot(x="wavelength") * lorentz_vlines([-4, -2, -1, 0, 1, 2, 4], scale=1e-9)
).opts(opts.VLine(line_dash="dotted"))

In [ ]:
print({k: v.value for k, v in widgets.items()})

Interesting parameter sets:
* {'excitonic_layer_thickness': 30, 'theta': 0.0, 'num_periods': 10, 'passive_layer_thickness': 210}
    * See what happens on increasing excitonic_layer_thickness to 40 or 50 - 'paradoxical' reduction in absorption!

### Interactive plot of RTA (by period rather than passive layer thickness)

In [ ]:
restacked_xs_1D = {
    "excitonic_layer_thickness": pnw.DiscreteSlider,
    "theta": pnw.DiscreteSlider,
    "num_periods": pnw.DiscreteSlider,
    "period": pnw.DiscreteSlider,
}

restacked_ds_sel = restacked_ds[s_polarised_RTA].interactive(loc="left").sel(**restacked_xs_1D).squeeze(drop=True)

In [ ]:
# Get a handle on the widgets
restacked_widgets = {k: w for k, w in zip(restacked_xs_1D, restacked_ds_sel.widgets())}

In [ ]:
(
    restacked_ds_sel.hvplot(x="wavelength") * lorentz_vlines([-4, -2, -1, 0, 1, 2, 4], scale=1e-9)
).opts(
    opts.VLine(line_dash="dotted"))

In [ ]:
print({k: v.value for k, v in restacked_widgets.items()})

Interesting parameter sets:
* {'excitonic_layer_thickness': 30, 'theta': 0.0, 'num_periods': 10, 'period': 240}
    * The reduction in absorption above disappears when the period is fixed.

In [ ]:
def wrapped_plot(*args, **kwargs):
    return plot_RTA(*args, **kwargs).opts(legend_position='right')
bound_plot = pn.interact(wrapped_plot, period=(10,460,10), excitonic_layer_thickness=(10, 150,10), num_periods=(1, 50), theta=0)

In [ ]:
bound_plot

### Interactive plot of RTA (faceted by num_periods)

In [ ]:
xs_1D_np = {
    "num_periods": [1, 5, 10, 20, 50],
    "excitonic_layer_thickness": pnw.DiscreteSlider,
    "theta": pnw.DiscreteSlider,
    "passive_layer_thickness": pnw.DiscreteSlider,
}

ds_sel_np = ds[s_polarised_RTA].interactive(loc="left").sel(**xs_1D_np).squeeze(drop=True)

In [ ]:
# Get a handle on the widgets
widgets_np = {k: w for k, w in zip(["excitonic_layer_thickness", "theta", "passive_layer_thickness"], ds_sel_np.widgets())}

In [ ]:
(
    (
        ds_sel_np.Rs.hvplot(x="wavelength", by="num_periods")
        + ds_sel_np.Ts.hvplot(x="wavelength", by="num_periods")
        + ds_sel_np.As.hvplot(x="wavelength", by="num_periods")
    )
    * lorentz_vlines([-4, -2, -1, 0, 1, 2, 4], scale=1e-9)
).cols(1).opts(opts.VLine(line_dash="dotted"), opts.Overlay(title=""))

In [ ]:
print({k: v.value for k, v in widgets_np.items()})

Interesting parameter sets:
* {'excitonic_layer_thickness': 30, 'theta': 0.0, 'passive_layer_thickness': 210}
    * Reflectance around the oscillator resonance wavelength is significantly stronger for an intermediate (~10) number of periods!
* {'excitonic_layer_thickness': 40, 'theta': 0.0, 'passive_layer_thickness': 210}
    * Absorptance near the oscillator wavelength is slightly higher for an intermediate (~10) number of periods!

### Absorption can decrease with increasing excitonic layer thickness

The complicated interaction between the dispersive refractive index of the Lorentz oscillator and the photonic band gap can lead to seemingly paradoxical behaviour, such as the reduction of absorption around the oscillator resonance wavelength as the thickness of the excitonic layer increases by 33% from 30 nm to 40 nm. This is due to the increase in excitonic layer thickness causing both the central wavelength and strength of the PBG to increase.

In [ ]:
sel = {"theta": 0.0, "num_periods": 10, "passive_layer_thickness": 210}
sel_elt = {"excitonic_layer_thickness": slice(20, 50)}

In [ ]:
(
    ds.A.sel(**sel)
    .sel(**sel_elt)
    .squeeze()
    .hvplot(x="wavelength", by="excitonic_layer_thickness")
    * lorentz_vlines([-2, 0, 2], scale=1e-9)
).opts(
    opts.Curve(ylim=(0,1)),
    opts.VLine(line_color=green, line_dash='dotted', line_alpha=0.5),
    opts.Overlay(title=coordinate_string(**sel), legend_position="top_right"),
)

In [ ]:
(
    ds.R.sel(**sel)
    .sel(**sel_elt)
    .squeeze()
    .hvplot(x="wavelength", by="excitonic_layer_thickness")
    * lorentz_vlines([-2, 0, 2], scale=1e-9)
).opts(
    opts.Curve(ylim=(0,1)),
    opts.VLine(line_color=green, line_dash='dotted', line_alpha=0.5),
    opts.Overlay(title=coordinate_string(**sel), legend_position="top_right"),
)

This case also provides an example of the wavelength-specificity of photonic effects on absorption. If considering only a window of two linewidths around the resonance wavelength, the integrated absorptivity of the structure is highest when the excitonic layer thickness is 30 nm. If considering a window of four linewidths, the same metric increases monotonically over the same range.

In [ ]:
ds_flat_spectrum.A.sel(**sel).sel(**sel_elt).squeeze()

In [ ]:
select_lorentz_line(ds.A, 0, 2).sel(**sel).sel(**sel_elt).integrate('wavelength').compute()

In [ ]:
select_lorentz_line(ds.A, 0, 4).sel(**sel).sel(**sel_elt).integrate('wavelength').compute()

In [ ]:
curve_2 = (
    select_lorentz_line(ds.A, 0, 2)
    .sel(**sel)
    .sel(**sel_elt)
    .integrate("wavelength")
    .squeeze()
    .hvplot(x="excitonic_layer_thickness", label='Window of 2 Γ')
)
curve_4 = (
    select_lorentz_line(ds.A, 0, 4)
    .sel(**sel)
    .sel(**sel_elt)
    .integrate("wavelength")
    .squeeze()
    .hvplot(x="excitonic_layer_thickness", label='Window of 4 Γ')
)

In [ ]:
(curve_2*curve_4).opts(title=coordinate_string(**sel), legend_position="top_left", ylabel="Integrated absorptivity (arbitrary units)")

In [ ]:
da = ds.A.sel(**sel)

curves = {0: select_lorentz_line(da, 0, 0).squeeze().hvplot(x='excitonic_layer_thickness', label=f'Resonance wavelength')}

for x in range (1, 6):
    da_sel = select_lorentz_line(da, 0, x)
    normalisation = spectrum(np.ones_like, da_sel.wavelength, normalisation=1)
    curves.update({x: (da_sel*normalisation).integrate('wavelength').rename('A').squeeze().hvplot(x='excitonic_layer_thickness', label=f'Window of {x} Γ')})

In [ ]:
da = ds.A.sel(**sel)

curves = {0: select_lorentz_line(da, 0, 0).squeeze().hvplot(x='excitonic_layer_thickness', label=f'Resonance wavelength')}

for x in range (1, 6):
    da_sel = select_lorentz_line(da, 0, x)
    da_sel = normalise_over_dim(da_sel, "wavelength")
    curves.update({x: (da_sel).integrate('wavelength').rename('A').squeeze().hvplot(x='excitonic_layer_thickness', label=f'Window of {x} Γ')})

In [ ]:
hv.NdOverlay(curves, kdims=["Window size (Γ)"]).opts(
    title="Average absorptance over a window around the resonance wavelength\n"
    + coordinate_string(**sel),
    legend_position="right",
)

When fixing the period, which approximately fixes the position of the PBG, the 'paradoxical' effect disappears and absorptance across the spectrum increases monotonically with excitonic layer thickness.

In [ ]:
sel = {"theta": 0.0, "num_periods": 10}
sel_elt = {"excitonic_layer_thickness": slice(20, 50), "period": slice(200, 270)}
(
    restacked_ds.A.sel(**sel)
    .sel(**sel_elt)
    .squeeze()
    .hvplot(x="wavelength", by="excitonic_layer_thickness", groupby=['period'])
    * lorentz_vlines([-2, 0, 2], scale=1e-9)
).opts(
    opts.Curve(ylim=(0,1)),
    opts.VLine(line_color=green, line_dash='dotted', line_alpha=0.5),
    opts.Overlay(title=coordinate_string(**sel), legend_position="top_right"),
)

In [ ]:
da = restacked_ds.A.sel(**sel)
periods = range(200,271, 10)

curves = {(0,P): select_lorentz_line(da, 0, 0).sel(period=P).squeeze().hvplot(x='excitonic_layer_thickness', label=f'Resonance wavelength') for P in periods}

for x in range (1, 6):
    for P in periods:
        da_sel = select_lorentz_line(da, 0, x).sel(period=P)
        normalisation = spectrum(np.ones_like, da_sel.wavelength, normalisation=1)
        curves.update({(x,P): (da_sel*normalisation).integrate('wavelength').rename('A').squeeze().hvplot(x='excitonic_layer_thickness', label=f'Window of {x} Γ')})

In [ ]:
hmap = hv.HoloMap(curves, kdims=["Window size (Γ)", "Λ"])

In [ ]:
hmap.overlay("Window size (Γ)").opts(
    opts.Curve(
    title="Average absorptance over a window around the resonance wavelength\n"
    + coordinate_string(**sel),)
)

### Reflectance can decrease with increasing number of periods

In general, the PBG narrows with increasing number of periods, and this can cause the reflectance near the PBG wavelength to decrease. This effect is particularly stark near the resonance wavelength though, and especially when it is on the red side of the PBG. This could be a double-PBG effect - need to solve the dispersion relation to know for sure.

In [ ]:
sel = {"excitonic_layer_thickness": 30, "theta": 0.0, "period": 240}
sel_np = {"num_periods": [1, 5, 10, 15, 20, 30, 50]}
(
    restacked_ds.R.sel(**sel)
    .sel(**sel_np)
    .squeeze()
    .hvplot(x="wavelength", by="num_periods")
    * lorentz_vlines([0], scale=1e-9, label='L0')
    * lorentz_vlines([-1], scale=1e-9, label='L1')
    * lorentz_vlines([-2], scale=1e-9, label='L2')
).opts(
    # opts.Curve(ylim=(0,1)),
    opts.VLine('VLine.L2', line_color=blue, line_dash="dotted", line_alpha=0.5),
    opts.VLine('VLine.L1', line_color=red, line_dash="dotted", line_alpha=0.5),
    opts.VLine('VLine.L0', line_color=yellow, line_dash="dotted", line_alpha=0.5),
    opts.Overlay(title=coordinate_string(**sel), legend_position="top_right"),
)

In [ ]:
sel = {"excitonic_layer_thickness": 30, "theta": 0.0, "period": 250}
sel_np = {"num_periods": [1, 5, 10, 15, 20, 30, 50]}
(
    restacked_ds.R.sel(**sel)
    .sel(**sel_np)
    .squeeze()
    .hvplot(x="wavelength", by="num_periods")
    * lorentz_vlines([0], scale=1e-9)
).opts(
    # opts.Curve(ylim=(0,1)),
    opts.VLine(line_color=green, line_dash="dotted", line_alpha=0.5),
    opts.Overlay(title=coordinate_string(**sel), legend_position="top_right"),
)

In [ ]:
sel = {"excitonic_layer_thickness": 70, "theta": 0.0, "period": 210}
sel_np = {"num_periods": [1, 5, 10, 15, 20, 30, 50]}
(
    restacked_ds.R.sel(**sel)
    .sel(**sel_np)
    .squeeze()
    .hvplot(x="wavelength", by="num_periods")
    * lorentz_vlines([0], scale=1e-9, label='L0')
    * lorentz_vlines([-7], scale=1e-9, label='L7')
).opts(
    # opts.Curve(ylim=(0,1)),
    opts.VLine('VLine.L7', line_color=green, line_dash="dotted", line_alpha=0.5),
    opts.VLine('VLine.L0', line_color=yellow, line_dash="dotted", line_alpha=0.5),
    opts.Overlay(title=coordinate_string(**sel), legend_position="top_right"),
)

In [ ]:
sel = {"excitonic_layer_thickness": 30, "theta": 0.0, "period": 240}
da = restacked_ds.R.sel(**sel)

curves = {0: select_lorentz_line(da, 0, 0).squeeze().hvplot(x='num_periods', label=f'Resonance wavelength')}

for x in range (1, 6):
    da_sel = select_lorentz_line(da, 0, x)
    normalisation = spectrum(np.ones_like, da_sel.wavelength, normalisation=1)
    curves.update({x: (da_sel*normalisation).integrate('wavelength').rename('R').squeeze().hvplot(x='num_periods', label=f'Window of {x} Γ')})

hv.NdOverlay(curves, kdims=["Window size (Γ)"]).opts(
    title="Average reflectance over a window around the resonance wavelength\n"
    + coordinate_string(**sel),
    legend_position="right",
)

In [ ]:
sel = {"excitonic_layer_thickness": 30, "theta": 0.0, "period": 240}
da = restacked_ds.R.sel(**sel)

curves = {0: select_lorentz_line(da, -1, 0).squeeze().hvplot(x='num_periods', label='-1 Lorentz line')}

for x in range (1, 6):
    da_sel = select_lorentz_line(da, 0, x)
    normalisation = spectrum(np.ones_like, da_sel.wavelength, normalisation=1)
    curves.update({x: (da_sel*normalisation).integrate('wavelength').rename('R').squeeze().hvplot(x='num_periods', label=f'Window of {x} Γ')})

hv.NdOverlay(curves, kdims=["Window size (Γ)"]).opts(
    title="Average reflectance over a window around the -1 Lorentz line\n"
    + coordinate_string(**sel),
    legend_position="right",
)

In [ ]:
sel = {"excitonic_layer_thickness": 30, "theta": 0.0, "period": 240}
da = restacked_ds.R.sel(**sel)

curves = {0: select_lorentz_line(da, -2, 0).squeeze().hvplot(x='num_periods', label='-2 Lorentz line')}

for x in range (1, 6):
    da_sel = select_lorentz_line(da, 0, x)
    normalisation = spectrum(np.ones_like, da_sel.wavelength, normalisation=1)
    curves.update({x: (da_sel*normalisation).integrate('wavelength').rename('R').squeeze().hvplot(x='num_periods', label=f'Window of {x} Γ')})

hv.NdOverlay(curves, kdims=["Window size (Γ)"]).opts(
    title="Average reflectance over a window around the -2 Lorentz line\n"
    + coordinate_string(**sel),
    legend_position="right",
)

In [ ]:
sel = {"excitonic_layer_thickness": 60, "theta": 0.0, "period": 210}
da = restacked_ds.R.sel(**sel)

curves = {0: select_lorentz_line(da, -7, 0).squeeze().hvplot(x='num_periods', label='-7 Lorentz line')}

for x in np.arange(1, 6)/2:
    da_sel = select_lorentz_line(da, 0, x)
    curve = integrate_da(da_sel, dim="wavelength", normalisation=1).squeeze().hvplot(x='num_periods', label=f'Window of {x} Γ')
    curves.update({x: curve})

overlay = hv.NdOverlay(curves, kdims=["Window size (Γ)"]).opts(
    title="Average reflectance over a window around the -7 Lorentz line\n"
    + coordinate_string(**sel),
    legend_position="right",
)
overlay

In [ ]:
overlay[0.5:]

In [ ]:
overlay.select(num_periods=slice(0, 10))

### Absorptance can decrease with increasing number of periods

It seems that in some cases, adding more layers can increase the reflectance at the expense of the absorption, causing the latter to increase with increasing number of periods.

In [ ]:
sel = {
    "excitonic_layer_thickness": 40,
    "theta": 0.0,
    # 'period': 250,
}
sel_np = {"num_periods": [1, 5, 10, 15, 20, 30, 50]}
(
    restacked_ds.A.sel(**sel)
    .sel(**sel_np)
    .squeeze()
    .hvplot(x="wavelength", by="num_periods", groupby=["period"])
    * lorentz_vlines([-1, 0, 1], scale=1e-9)
).opts(
    # opts.Curve(ylim=(0,1)),
    opts.VLine(line_color=green, line_dash="dotted", line_alpha=0.5),
    opts.Overlay(title=coordinate_string(**sel), legend_position="top_right"),
)

In [ ]:
sel = {
    "excitonic_layer_thickness": 40,
    "theta": 0.0,
    'period': 250,
}
sel_wl = {"wavelength": slice(674, 681)}
(
    restacked_ds.A.sel(**sel)
    .sel(**sel_wl)
    .squeeze()
    .hvplot(x="num_periods", by="wavelength", groupby=["period"])
    * lorentz_vlines([-1, 0, 1], scale=1e-9)
).opts(
    # opts.Curve(ylim=(0,1)),
    opts.VLine(line_color=green, line_dash="dotted", line_alpha=0.5),
    opts.Overlay(title=coordinate_string(**sel), legend_position="right"),
)

Does the transmittance ever go up with number of periods?

In [ ]:
sel = {
    "excitonic_layer_thickness": 40,
    "theta": 0.0,
    # 'period': 250,
}
sel_np = {"num_periods": [1, 5, 10, 15, 20, 30, 50]}
(
    restacked_ds.T.sel(**sel)
    .sel(**sel_np)
    .squeeze()
    .hvplot(x="wavelength", by="num_periods", groupby=["period"])
    * lorentz_vlines([-1, 0, 1], scale=1e-9)
).opts(
    # opts.Curve(ylim=(0,1)),
    opts.VLine(line_color=green, line_dash="dotted", line_alpha=0.5),
    opts.Overlay(title=coordinate_string(**sel), legend_position="top_right"),
)

In [ ]:
sel = {
    "excitonic_layer_thickness": 40,
    "theta": 0.0,
    'period': 250,
}
sel_wl = {"wavelength": slice(674, 681)}
(
    restacked_ds.T.sel(**sel)
    .sel(**sel_wl)
    .squeeze()
    .hvplot(x="num_periods", by="wavelength", groupby=["period"])
    * lorentz_vlines([-1, 0, 1], scale=1e-9)
).opts(
    # opts.Curve(ylim=(0,1)),
    opts.VLine(line_color=green, line_dash="dotted", line_alpha=0.5),
    opts.Overlay(title=coordinate_string(**sel), legend_position="right"),
)

Huh... what about reflectance and transmittance in this regime?

In [ ]:
sel = {
    "excitonic_layer_thickness": 40,
    "theta": 0.0,
    # 'period': 250,
}
sel_np = {"num_periods": [1, 5, 10, 15, 20, 30, 50]}
(
    restacked_ds.R.sel(**sel)
    .sel(**sel_np)
    .squeeze()
    .hvplot(x="wavelength", by="num_periods", groupby=["period"])
    * lorentz_vlines([-1, 0, 1], scale=1e-9)
).opts(
    # opts.Curve(ylim=(0,1)),
    opts.VLine(line_color=green, line_dash="dotted", line_alpha=0.5),
    opts.Overlay(title=coordinate_string(**sel), legend_position="top_right"),
)

In [ ]:
sel = {
    "excitonic_layer_thickness": 40,
    "theta": 0.0,
    # 'period': 250,
}
sel_np = {"num_periods": [1, 5, 10, 15, 20, 30, 50]}
(
    restacked_ds.T.sel(**sel)
    .sel(**sel_np)
    .squeeze()
    .hvplot(x="wavelength", by="num_periods", groupby=["period"])
    * lorentz_vlines([-1, 0, 1], scale=1e-9)
).opts(
    # opts.Curve(ylim=(0,1)),
    opts.VLine(line_color=green, line_dash="dotted", line_alpha=0.5),
    opts.Overlay(title=coordinate_string(**sel), legend_position="top_right"),
)

In [ ]:
sel = {
    "excitonic_layer_thickness": 40,
    "theta": 0.0,
    'period': 250,
}
sel_wl = {"wavelength": slice(674, 681)}
(
    restacked_ds.R.sel(**sel)
    .sel(**sel_wl)
    .squeeze()
    .hvplot(x="num_periods", by="wavelength", groupby=["period"])
    * lorentz_vlines([-1, 0, 1], scale=1e-9)
).opts(
    # opts.Curve(ylim=(0,1)),
    opts.VLine(line_color=green, line_dash="dotted", line_alpha=0.5),
    opts.Overlay(title=coordinate_string(**sel), legend_position="right"),
)

In [ ]:
da = restacked_ds.A.sel(**sel)

curves = {0: select_lorentz_line(da, 0, 0).squeeze().hvplot(x='num_periods', label=f'Resonance wavelength')}

for x in range (1, 6):
    da_sel = select_lorentz_line(da, 0, x)
    normalisation = spectrum(np.ones_like, da_sel.wavelength, normalisation=1)
    curves.update({x: (da_sel*normalisation).integrate('wavelength').rename('R').squeeze().hvplot(x='num_periods', label=f'Window of {x} Γ')})

In [ ]:
hv.NdOverlay(curves, kdims=["Window size (Γ)"]).opts(
    title="Average absorptance over a window around the resonance wavelength\n"
    + coordinate_string(**sel),
    legend_position="right",
)

Probably need to do some E field plots to make sense of this... That could be a bit of work.

In [ ]:
pn.interact(
    wrap_plot_field,
    wavelength=(480, 880),
    excitonic_layer_thickness=(10, 200, 10),
    passive_layer_thickness=(0, 300, 10),
    num_periods=(1, 50),
)

### Grid plot

This grid plot demonstrates the shift and growth in PBG with period and N.

In [ ]:
coords = {"excitonic_layer_thickness": 30, "theta": 0}

In [ ]:
curves = {(P,N): restacked_ds[unpolarised_RTA].sel(num_periods=N, period=P, **coords).squeeze().hvplot.line(x="wavelength")*lorentz_vlines(0, scale=1e-9).opts(line_dash='dotted') for N in [50, 20, 10] for P in [200, 250, 300]}

In [ ]:
grid = hv.GridSpace(curves, kdims=[period_dim, "Number of periods"])

In [ ]:
grid

In [ ]:
# hacky workaround to bugs in hvplot/holoviews
for p in grid:
    p.opts(
        opts.Curve(
            color=hv.Cycle([blue, yellow, red]),  # set R/T/A->blue/yellow/red
            frame_width=200,
        ),
        opts.Overlay(show_legend=False,show_grid=True,),
    )  # remove the excess legends

grid[200, 50].opts(
    opts.Overlay(show_legend=True, legend_position="left")
)  # add the legend back in on a free side

grid.opts(opts.GridSpace(show_legend=False, shared_xaxis=True, shared_yaxis=True))

This is the code I would have liked to have used to save this image.

Unfortunately the `toolbar=False` argument doesn't work so I have to actually *use* the toolbar above to save it!

... Except this ALSO doesn't work, because the save button saves 9 individual images?! So I guess I have to use the below code and manually crop or something stupid.

In [ ]:
hv.save(grid, f"data/run_{run_number}/figures/N_Lambda_grid", fmt="png", toolbar=False)

## 2D plots of RTA

In [ ]:
# example params
default_LOPC_params = {
    "period": 300,
    "num_periods": 10,
    "excitonic_layer_thickness": 30,
}

default_LOPC_params_ds = {
    "passive_layer_thickness": default_LOPC_params["period"] - default_LOPC_params["excitonic_layer_thickness"],
    "num_periods": default_LOPC_params["num_periods"],
    "excitonic_layer_thickness": default_LOPC_params["excitonic_layer_thickness"],
}

default_crs_1_params = {
    "total_excitonic_thickness": default_LOPC_params["excitonic_layer_thickness"]*default_LOPC_params["num_periods"],
}

### Angle-dependency of PBG

In [ ]:
# # This is super slow and I'm not sure why - chunks over theta I guess?
# (
#     ds.R.sel(
#         excitonic_layer_thickness=20,
#         passive_layer_thickness=[200, 300],
#         num_periods=20,
#         theta=slice(0, 60),
#     )
#     .squeeze(drop=True)
#     .hvplot(x="wavelength", y="theta", by="passive_layer_thickness", subplots=True)
# ).opts(opts.Image(cmap="viridis")).cols(1)

In [ ]:
ds.R.drop_sel(num_periods=0).sel(
    excitonic_layer_thickness=20,
    passive_layer_thickness=slice(0, 501, 4),
    num_periods=20,
    theta=slice(0, 60),
).squeeze(drop=True).plot(
    x="wavelength",
    y="theta",
    col="passive_layer_thickness",
)

### Plots using 'angle_ds' flow

In [ ]:
coords = {"excitonic_layer_thickness": 30,
          "passive_layer_thickness": 280,
          "num_periods": 50,
}

angle_plots = [angle_ds[var].sel(**coords).squeeze().hvplot(x="wavelength", y="theta") for var in ["Rs", "Rp", "R"]]

layout = hv.Layout(angle_plots).opts(opts.Image(title=coordinate_string(**coords), cmap="viridis", clim=(0, 1)))

(layout*lorentz_vlines(0, scale=1e-9).opts(line_dash='dotted', color="white")).cols(1)

In [ ]:
angle_plots = [angle_ds[var].sel(**coords).squeeze().hvplot(x="wavelength", y="theta") for var in ["Ts", "Tp", "T"]]

layout = hv.Layout(angle_plots).opts(opts.Image(title=coordinate_string(**coords), cmap="cividis", clim=(0, 1)))

(layout*lorentz_vlines(0, scale=1e-9).opts(line_dash='dotted', color="white")).cols(1)

In [ ]:
angle_plots = [angle_ds[var].sel(**coords).squeeze().hvplot(x="wavelength", y="theta") for var in ["As", "Ap", "A"]]

layout = hv.Layout(angle_plots).opts(opts.Image(title=coordinate_string(**coords), cmap="inferno", clim=(0, 1)))

(layout*lorentz_vlines(0, scale=1e-9).opts(line_dash='dotted', color="black")).cols(1)

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

angle_plots = [angle_norm_1[var].sel(**coords).squeeze().hvplot(x="wavelength", y="theta") for var in ["As", "Ap", "A"]]

layout = hv.Layout(angle_plots).opts(opts.Image(title=coordinate_string(**coords), cmap="RdBu_r", clim=(0.5, 1.5)))

(layout*lorentz_vlines(0, scale=1e-9).opts(line_dash='dotted', color="white")).cols(1)

### Plots using differently chunked ds

In [ ]:
angle_plots = [restacked_ds[var].sel(**default_LOPC_params).squeeze().hvplot(x="wavelength", y="theta") for var in ["Rs", "Rp", "R"]]

layout = hv.Layout(angle_plots).opts(opts.Image(title=coordinate_string(**default_LOPC_params), cmap="viridis", clim=(0, 1)))

(layout*lorentz_vlines(0, scale=1e-9).opts(line_dash='dotted', color="white")).cols(1)

In [ ]:
angle_plots = [restacked_ds[var].sel(**default_LOPC_params).squeeze().hvplot(x="wavelength", y="theta") for var in ["Ts", "Tp", "T"]]

layout = hv.Layout(angle_plots).opts(opts.Image(title=coordinate_string(**default_LOPC_params), cmap="cividis", clim=(0, 1)))

(layout*lorentz_vlines(0, scale=1e-9).opts(line_dash='dotted', color="white")).cols(1)

In [ ]:
angle_plots = [restacked_ds[var].sel(**default_LOPC_params).squeeze().hvplot(x="wavelength", y="theta") for var in ["As", "Ap", "A"]]

layout = hv.Layout(angle_plots).opts(opts.Image(title=coordinate_string(**default_LOPC_params), cmap="inferno", clim=(0, 1)))

(layout*lorentz_vlines(0, scale=1e-9).opts(line_dash='dotted', color="black")).cols(1)

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

angle_plots = [restacked_norm_1[var].sel(**default_LOPC_params).squeeze().hvplot(x="wavelength", y="theta") for var in ["As", "Ap", "A"]]

layout = hv.Layout(angle_plots).opts(opts.Image(title=coordinate_string(**default_LOPC_params), cmap="RdBu_r", clim=(0.5, 1.5)))

(layout*lorentz_vlines(0, scale=1e-9).opts(line_dash='dotted', color="white")).cols(1)

In [ ]:
angle_plots = [crs_1[var].sel(**default_crs_1_params).squeeze().hvplot(x="wavelength", y="theta") for var in ["Rs", "Rp", "R"]]

layout = hv.Layout(angle_plots).opts(opts.Image(title=coordinate_string(**default_crs_1_params), cmap="viridis", clim=(0, 1)))

(layout*lorentz_vlines(0, scale=1e-9).opts(line_dash='dotted', color="white")).cols(1)

In [ ]:
angle_plots = [crs_1[var].sel(**default_crs_1_params).squeeze().hvplot(x="wavelength", y="theta") for var in ["As", "Ap", "A"]]

layout = hv.Layout(angle_plots).opts(opts.Image(title=coordinate_string(**default_crs_1_params), cmap="inferno", clim=(0, 1)))

(layout*lorentz_vlines(0, scale=1e-9).opts(line_dash='dotted', color="black")).cols(1)

### Period-dependency of PBG

In [ ]:
xs = {
    "excitonic_layer_thickness": pnw.DiscreteSlider,
    "theta": pnw.DiscreteSlider,
    "num_periods": pnw.DiscreteSlider,
}

temp_sel = restacked_ds[unpolarised_RTA].squeeze(drop=True).interactive(loc="left").sel(**xs)

(
    (
        temp_sel.sel(period=200).hvplot(x="wavelength")
        + temp_sel.sel(period=250).hvplot(x="wavelength")
        + temp_sel.sel(period=300).hvplot(x="wavelength")
    )
    * lorentz_vlines([-4, -2, -1, 0, 1, 2, 4], scale=1e-9).opts(opts.VLine(line_dash="dotted"))
).cols(1)

In [ ]:
# I don't know why temp_sel.R fails unless you try dictionary access first, but it does!
try:
    temp_sel['R']
except:
    pass

In [ ]:
# xs = {
#     "excitonic_layer_thickness": pnw.DiscreteSlider,
#     "theta": pnw.DiscreteSlider,
#     "num_periods": pnw.DiscreteSlider,
# }

# temp_sel = restacked_ds[unpolarised_RTA].squeeze(drop=True).interactive.sel(**xs)

(
    (
        temp_sel.R.dropna("period").hvplot(x="wavelength", y="period").opts(cmap='viridis')
        + temp_sel.T.dropna("period").hvplot(x="wavelength", y="period").opts(cmap='cividis')
        + temp_sel.A.dropna("period").hvplot(x="wavelength", y="period").opts(cmap='inferno')
    )
    * lorentz_vlines([-4, -2, -1, 0, 1, 2, 4], scale=1e-9).opts(opts.VLine(line_dash="dotted"))
).cols(1)

### Period plots for thesis

In [ ]:
temp_params = {
    "theta": 0,
    "num_periods": 10,
    "excitonic_layer_thickness": 30,
}

In [ ]:
period_plots = [
    restacked_ds[var]
    .sel(**temp_params)
    .squeeze()
    .dropna("period")
    .hvplot(x="wavelength", y="period")
    for var in ["R"]
]

layout = hv.Layout(period_plots).opts(
    opts.Image(title=coordinate_string(**temp_params), cmap="viridis", clim=(0, 0.31))
)

(layout * lorentz_vlines(0, scale=1e-9)).opts(
    opts.Image(xlabel=layout[0].kdims[0].label, ylabel=layout[0].kdims[1].label),
    opts.VLine(line_dash="dotted", color="white"),
).cols(1)

In [ ]:
period_plots = [
    restacked_norm_1[var]
    .sel(**temp_params)
    .squeeze()
    .dropna("period")
    .hvplot(x="wavelength", y="period")
    for var in ["A"]
]

layout = hv.Layout(period_plots).opts(
    opts.Image(title=coordinate_string(**temp_params), cmap="RdBu_r", clim=(0.6, 1.4))
)

(layout * lorentz_vlines(0, scale=1e-9)).opts(
    opts.Image(xlabel=layout[0].kdims[0].label, ylabel=layout[0].kdims[1].label),
    opts.VLine(line_dash="dotted", color="white"),
).cols(1)

In [ ]:
line_plots = [
    restacked_norm_1[var]
    .sel(**temp_params|{"period": 170})
    .squeeze()
    .hvplot(x="wavelength")
    for var in ["A"]
]

layout = hv.Layout(line_plots).opts(
    opts.Curve(title=coordinate_string(**temp_params|{"period": 170}), ylim=(0.9, 1.4))
)

(layout * lorentz_vlines(0, scale=1e-9) * hv.HLine(1)).opts(
    opts.Curve(xlabel=layout[0].kdims[0].label),
    opts.VLine(line_dash="dotted", color=yellow),
    opts.HLine(line_dash="dotted", color=blue),
).cols(1)

### Number of periods plots for thesis

In [ ]:
temp_params = {
    "theta": 0,
    "period": 200,
    "excitonic_layer_thickness": 30,
}

In [ ]:
N_plots = [
    restacked_ds[var]
    .sel(**temp_params)
    .squeeze()
    .hvplot.quadmesh(x="wavelength", y="num_periods")
    for var in ["R"]
]

layout = hv.Layout(N_plots).opts(
    opts.QuadMesh(title=coordinate_string(**temp_params), cmap="viridis", clim=(0, 1))
)

(layout * lorentz_vlines(0, scale=1e-9)).opts(
    opts.QuadMesh(xlabel=layout[0].kdims[0].label, ylabel=layout[0].kdims[1].label),
    opts.VLine(line_dash="dotted", color="white"),
).cols(1)

In [ ]:
N_plots = [
    restacked_norm_1[var]
    .sel(**temp_params)
    .squeeze()
    .hvplot.quadmesh(x="wavelength", y="num_periods")
    for var in ["A"]
]

layout = hv.Layout(N_plots).opts(
    opts.QuadMesh(title=coordinate_string(**temp_params), cmap="RdBu_r", clim=(0.5, 1.5))
)

(layout * lorentz_vlines(0, scale=1e-9)).opts(
    opts.QuadMesh(xlabel=layout[0].kdims[0].label, ylabel=layout[0].kdims[1].label),
    opts.VLine(line_dash="dotted", color="white"),
).cols(1)

## Optimal structures for various parameters

### Investigation and notes

In [ ]:
periods = restacked_ds.period.values[1:-1]
# don't set the maximum period to be lower than the maximum excitonic layer thickness, or it will break the plot!
safe_periods = restacked_ds.period.sel(
    period=slice(restacked_ds.excitonic_layer_thickness.values[-1], None)
).values

kdims = [
    hv.Dimension("variable", values=restacked_ds.data_vars),
    hv.Dimension("dataset", values=["restacked_ds", "restacked_diff_1"]),
    hv.Dimension("optimise", values=["max", "min"]),
    hv.Dimension("lorentz_line", range=(-15, 8), default=0),
    hv.Dimension("window_radius", range=(0, 8), default=0),
    hv.Dimension("theta", range=(0, 86), default=0),
    hv.Dimension("cmap", values=["viridis", "cividis", "inferno", "PRGn", "PuOr_r", "RdBu_r"]),
    hv.Dimension("period_start", values=periods, default=periods[0]),
    hv.Dimension("period_stop", values=safe_periods, default=periods[-1]),
    hv.Dimension("integrate_angle", range=(0, 86), default=0),
]

angle_dmap = hv.DynamicMap(
    partial(
        wrapped_2D_plot,
        extra_plots=["RTA_normal", "RTA_int", "norm_1_normal", "norm_1_int"],
    ),
    kdims=kdims,
).opts(
    opts.Overlay(legend_position="right", legend_opts={"background_fill_alpha": 0.5})
)
# if I'm just looking at normal incidence the extra plots are redundant
dmap = hv.DynamicMap(
    partial(wrapped_2D_plot, extra_plots=["RTA_int", "norm_1_int"], integrate_angle=0),
    kdims=kdims[:9],
).opts(
    opts.Overlay(legend_position="right", legend_opts={"background_fill_alpha": 0.5})
)

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

# there is some sort of horrible bug that makes the title and axis on the second plot sometime disappear, along with one of the vlines
angle_dmap

In [ ]:
# Use this to record interesting key sets
angle_dmap.current_key

### Optimal structures for reflectance

#### Maximising at a specific wavelength

Carniglia and Apfel (1979) showed that the theoretical limit of the reflectance of a multilayer comprising one absorbing and one non-absorbing layer is unity. However, this requires variable layer thicknesses, unlike in our case where the periodicity is strict. As a result, our situation is more like Koppelmann's (1960), with a saturating reflectance depending on the absorption properties at a particular wavelength, although Koppelmann showed this for quarter-wave stacks only and we do not attempt to extend the analytical proof to other periodic multilayers. The gist of the argument is that without careful selection of layer thicknesses to optimise the reflectance with each additional layer pair, a certain amount of light intensity is unavoidably lost to absorption.

Generally, the reflectance near the oscillator resonance peaks at <50 layers (the maximum simulated), while the reflectance far from the resonance peaks at >=50 layers.

High reflectance at a wavelength comes at a cost to transmittance and absorptance locally, but is usually associated with an increase in absorptance at longer wavelengths.

In [ ]:
dmap[(('R', 'restacked_ds', 'max', 0, 0, 0, 'viridis', 10, 450))]

In [ ]:
dmap[('R', 'restacked_ds', 'max', -2, 0, 0, 'viridis', 10, 450)]

In [ ]:
dmap[('R', 'restacked_ds', 'max', 2, 0, 0, 'viridis', 10, 450)]

In [ ]:
dmap[('R', 'restacked_ds', 'max', -5, 0, 0, 'viridis', 10, 450)]

In [ ]:
dmap[('R', 'restacked_ds', 'max', 5, 0, 0, 'viridis', 10, 450)]

In [ ]:
# this calculation is just for the purposes of looking at how reflectance can decrease with period, see above sections
dmap[('R', 'restacked_ds', 'max', -7, 0, 0, 'viridis', 10, 450)]

#### Maximising over a window of 4 linewidths

When optimising for reflectance over a window of wavelengths (in this case a radius of 2 oscillator linewidths), the optimal number of periods is lower than when optimising for only the central wavelength of that window. This is because with more periods the PBG tends to increase in height but decrease in width. Unlike with purely non-absorbing layers, where the width of the PBG starts to increase again when the reflectance is saturated at 100%, the competing increase in absorption with additional layers means the reflectance peaks at a finite number of periods.

It is particularly difficult to attain a high reflectance in the low-RI contrast region immediately on the blue side of the LO resonance, and the peak values attainable on the lower-contrast blue side of the resonance are generally lower than those attainable on the higher-contrast red side.

It's worth noting that, as is always the case when referring to optical parameters over a range of wavelengths or angles, that this is an integration of the *intrinsic* optical properties, and the *extrinsic* value will depend on the spectral and angular distribution of illumination.

In [ ]:
dmap[(('R', 'restacked_ds', 'max', 0, 2, 0, 'viridis', 10, 450))]

In [ ]:
dmap[('R', 'restacked_ds', 'max', -2, 2, 0, 'viridis', 10, 450)]

In [ ]:
dmap[('R', 'restacked_ds', 'max', 2, 2, 0, 'viridis', 10, 450)]

In [ ]:
dmap[('R', 'restacked_ds', 'max', -5, 2, 0, 'viridis', 10, 450)]

In [ ]:
dmap[('R', 'restacked_ds', 'max', 5, 2, 0, 'viridis', 10, 450)]

In [ ]:
wrapped_2D_plot(
    "R",
    "restacked_ds",
    "max",
    lorentz_line=0,
    window_radius=2,
    theta=0,
    cmap="viridis",
    period_start=10,
    period_stop=450,
    integrate_angle=0,
    extra_plots=["RTA_int", "norm_1_int"],
).opts(
    opts.Overlay(legend_position="right", legend_opts={"background_fill_alpha": 0.5})
)

In [ ]:
wrapped_2D_plot(
    "R",
    "restacked_ds",
    "max",
    lorentz_line=-2,
    window_radius=2,
    theta=0,
    cmap="viridis",
    period_start=10,
    period_stop=450,
    integrate_angle=0,
    extra_plots=["RTA_int", "norm_1_int"],
).opts(
    opts.Overlay(legend_position="right", legend_opts={"background_fill_alpha": 0.5})
)

In [ ]:
wrapped_2D_plot(
    "R",
    "restacked_ds",
    "max",
    lorentz_line=2,
    window_radius=2,
    theta=0,
    cmap="viridis",
    period_start=10,
    period_stop=450,
    integrate_angle=0,
    extra_plots=["RTA_int", "norm_1_int"],
).opts(
    opts.Overlay(legend_position="right", legend_opts={"background_fill_alpha": 0.5})
)

In [ ]:
wrapped_2D_plot(
    "R",
    "restacked_ds",
    "max",
    lorentz_line=-5,
    window_radius=2,
    theta=0,
    cmap="viridis",
    period_start=10,
    period_stop=450,
    integrate_angle=0,
    extra_plots=["RTA_int", "norm_1_int"],
).opts(
    opts.Overlay(legend_position="right", legend_opts={"background_fill_alpha": 0.5})
)

In [ ]:
wrapped_2D_plot(
    "R",
    "restacked_ds",
    "max",
    lorentz_line=5,
    window_radius=2,
    theta=0,
    cmap="viridis",
    period_start=10,
    period_stop=450,
    integrate_angle=0,
    extra_plots=["RTA_int", "norm_1_int"],
).opts(
    opts.Overlay(legend_position="right", legend_opts={"background_fill_alpha": 0.5})
)

#### Minimising reflectance

Interestingly, the reflectance at the LO resonance is lower with five layers at 10 nm than a single layer. In this regime, the absorptance is enhanced compared to the CRS at all wavelengths. It is also possible to minimise the reflectance at wavelengths far from the resonance with even higher numbers of thin (10 nm) layers. However, the absolute difference in reflectance with the CRS is generally negligible, and so this is not that relevant for iridoplast-type scenarios. Additionally, the optimal structure degenerates to a pair of thin layers everywhere when a window of radius >=2 linewidths is considered.

In [ ]:
dmap[('R', 'restacked_ds', 'min', 0, 0, 0, 'viridis', 10, 450)]

In [ ]:
dmap[('R', 'restacked_ds', 'min', -8, 0, 0, 'viridis', 10, 450)]

In [ ]:
dmap[('R', 'restacked_ds', 'min', 0, 2, 0, 'viridis', 10, 450)]

In [ ]:
dmap[('R', 'restacked_ds', 'min', -8, 1, 0, 'viridis', 10, 450)]

In [ ]:
dmap[('R', 'restacked_ds', 'min', 4, 0, 0, 'viridis', 10, 450)]

#### Angle behaviour

The period which gives the highest overall reflectance at any given wavelength, when integrated over a angular range, is (not strictly) greater than the optimal period for normal incidence. This is because the PBG blueshifts with increasing $\theta$. The overall reflectance values after integration and normalisation are lower than those at normal incidence. Also, there is less specificity in where the reflectance is - generally there is a 'tail' of reflectance on the blue side of the target region.

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

angle_dmap[('R', 'restacked_ds', 'max', 0, 0, 0, 'viridis', 10, 450, 75)]

In [ ]:
# params that optimise reflectance integrated over theta
plot_RTA_2D(period=250, excitonic_layer_thickness=70, num_periods=50, theta=(0, 75), include=['LOPC'])*lorentz_vlines([0], scale=1e-9).opts(opts.VLine(line_dash='dotted', color="white"))

In [ ]:
# params that optimise reflectance at normal incidence
plot_RTA_2D(period=250, excitonic_layer_thickness=40, num_periods=40, theta=(0, 75), include=['LOPC'])*lorentz_vlines([0], scale=1e-9).opts(opts.VLine(line_dash='dotted', color="white"))

Supporting evidence: the period which gives the optimal reflectance near the resonance at 30 degrees is higher than that for normal incidence. Also notable is that the number of layers is much higher.

In [ ]:
angle_dmap[('R', 'restacked_ds', 'max', 0, 2, 30, 'viridis', 10, 450, 0)]

In [ ]:
angle_dmap[('R', 'restacked_ds', 'max', 0, 2, 60, 'viridis', 10, 450, 0)]

##### Consider the case below, where the PBG at normal incidence is clearly redshifted compared to the wavelength range we are trying to optimise reflectance in (-2<2). As the 2D plot below shows, this is because the reflectance is higher in that region overall taking into account the blueshift with angle.

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

angle_dmap[('R', 'restacked_ds', 'max', 0, 2, 0, 'viridis', 10, 450, 75)]

In [ ]:
# params that optimise reflectance integrated over theta
plot_RTA_2D(period=260, excitonic_layer_thickness=80, num_periods=25, theta=(0, 75), include=['LOPC'])*lorentz_vlines([-2, 2], scale=1e-9).opts(opts.VLine(line_dash='dotted', color="white"))

In [ ]:
# params that optimise reflectance at normal incidence
plot_RTA_2D(period=250, excitonic_layer_thickness=70, num_periods=30, theta=(0, 75), include=['LOPC'])*lorentz_vlines([-2, 2], scale=1e-9).opts(opts.VLine(line_dash='dotted', color="white"))

In [ ]:
# I'm curious - what happens if I try to make this PBG better with more layers? --Ah, it doesn't get better because of the absorption, I see.
plot_RTA_2D(period=250, excitonic_layer_thickness=70, num_periods=50, theta=(0, 75), include=['LOPC'])*lorentz_vlines([-2, 2], scale=1e-9).opts(opts.VLine(line_dash='dotted', color="white"))

In [ ]:
# comparison of the integrated and unintegrated RTA at normal incidence
# params optimise R when integrated
params = dict(
    period=260,
    excitonic_layer_thickness=80,
    num_periods=25,
)

(compare_RTA_normal_vs_integrated(**params) * lorentz_vlines([-2, 2], scale=1e-9)).opts(
    opts.VLine(line_dash="dotted", color=green), opts.Overlay(legend_position="right")
)

In [ ]:
# comparison of the integrated and unintegrated RTA at normal incidence
# params optimise R at normal incidence
params = dict(
    period=250,
    excitonic_layer_thickness=70,
    num_periods=30,
)

(compare_RTA_normal_vs_integrated(**params) * lorentz_vlines([-2, 2], scale=1e-9)).opts(
    opts.VLine(line_dash="dotted", color=green), opts.Overlay(legend_position="right")
)

##### As above, but the range (-4<0). Note how hard it is to get any appreciable reflectance here!

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

angle_dmap[('R', 'restacked_ds', 'max', -2, 2, 0, 'viridis', 10, 450, 75)]

In [ ]:
# params that optimise reflectance integrated over theta
plot_RTA_2D(period=250, excitonic_layer_thickness=80, num_periods=50, theta=(0, 75), include=['LOPC'])*lorentz_vlines([-4, 0], scale=1e-9).opts(opts.VLine(line_dash='dotted', color="white"))

In [ ]:
# params that optimise reflectance at normal incidence
plot_RTA_2D(period=230, excitonic_layer_thickness=80, num_periods=40, theta=(0, 75), include=['LOPC'])*lorentz_vlines([-4, 0], scale=1e-9).opts(opts.VLine(line_dash='dotted', color="white"))

##### As above, but the range (0, 4).

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

dmap[('R', 'restacked_ds', 'max', 2, 2, 0, 'viridis', 10, 450, 75)]

In [ ]:
# params that optimise reflectance integrated over theta
plot_RTA_2D(period=270, excitonic_layer_thickness=100, num_periods=20, theta=(0, 75), include=['LOPC'])*lorentz_vlines([0, 4], scale=1e-9).opts(opts.VLine(line_dash='dotted', color="white"))

In [ ]:
# params that optimise reflectance at normal incidence
plot_RTA_2D(period=260, excitonic_layer_thickness=80, num_periods=26, theta=(0, 75), include=['LOPC'])*lorentz_vlines([0, 4], scale=1e-9).opts(opts.VLine(line_dash='dotted', color="white"))

##### As above, but the range (-7, -3).

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

angle_dmap[('R', 'restacked_ds', 'max', -5, 2, 0, 'viridis', 10, 450, 75)]

In [ ]:
# params that optimise reflectance integrated over theta
plot_RTA_2D(period=220, excitonic_layer_thickness=100, num_periods=40, theta=(0, 75), include=['LOPC'])*lorentz_vlines([-7, -3], scale=1e-9).opts(opts.VLine(line_dash='dotted', color="white"))

In [ ]:
# params that optimise reflectance at normal incidence
plot_RTA_2D(period=210, excitonic_layer_thickness=100, num_periods=50, theta=(0, 75), include=['LOPC'])*lorentz_vlines([-7, -3], scale=1e-9).opts(opts.VLine(line_dash='dotted', color="white"))

##### As above, but the range (3, 7). Much higher potential reflectance here.

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

angle_dmap[('R', 'restacked_ds', 'max', 5, 2, 0, 'viridis', 10, 450, 75)]

In [ ]:
# params that optimise reflectance integrated over theta
plot_RTA_2D(period=290, excitonic_layer_thickness=130, num_periods=21, theta=(0, 75), include=['LOPC'])*lorentz_vlines([3, 7], scale=1e-9).opts(opts.VLine(line_dash='dotted', color="white"))

In [ ]:
# params that optimise reflectance at normal incidence
plot_RTA_2D(period=280, excitonic_layer_thickness=110, num_periods=22, theta=(0, 75), include=['LOPC'])*lorentz_vlines([3, 7], scale=1e-9).opts(opts.VLine(line_dash='dotted', color="white"))

### Optimal structures for transmittance

#### Maximum

The optimum structure for maximal transmittance at a wavelength is obvious (a single thin layer). What is more interesting is the way that the maximal transmittance drops off quickly with total excitonic layer thickness. The capacity of the LOPC to increase transmittance is minimal - see below plot with iridoplast-like parameters. It is at its best with a higher number of thin layers (examine this further when looking in to tunability).

In [ ]:
dmap[('T', 'restacked_ds', 'max', 0, 0, 0, 'cividis', 10, 450)]

In [ ]:
params = {"theta": 0, "excitonic_layer_thickness": 20, "num_periods": 8}
lorentz_line_number = 0
window_radius = 0
optimum = find_optimum_coords(
    da=select_lorentz_line(
        restacked_ds.T.sel(**params, method="nearest"),
        lorentz_line_number,
        window_radius,
    ),
    dim="period",
)
plot_RTA(period=float(optimum.period), **params)

In [ ]:
params = {"theta": 0, "excitonic_layer_thickness": 20, "num_periods": 8}
lorentz_line_number = -5
window_radius = 0
optimum = find_optimum_coords(
    da=select_lorentz_line(
        restacked_ds.T.sel(**params, method="nearest"),
        lorentz_line_number,
        window_radius,
    ),
    dim="period",
)
plot_RTA(period=float(optimum.period), **params)

In [ ]:
params = {"theta": 0, "excitonic_layer_thickness": 100, "num_periods": 8}
lorentz_line_number = 0
window_radius = 0
optimum = find_optimum_coords(
    da=select_lorentz_line(
        restacked_ds.T.sel(**params, method="nearest"),
        lorentz_line_number,
        window_radius,
    ),
    dim="period",
)
plot_RTA(period=float(optimum.period), **params)

In [ ]:
params = {"theta": 0, "excitonic_layer_thickness": 20, "num_periods": 30}
lorentz_line_number = 0
window_radius = 0
optimum = find_optimum_coords(
    da=select_lorentz_line(
        restacked_ds.T.sel(**params, method="nearest"),
        lorentz_line_number,
        window_radius,
    ),
    dim="period",
)
plot_RTA(period=float(optimum.period), **params)

In [ ]:
params = {"theta": 0, "excitonic_layer_thickness": 20, "num_periods": 30}
lorentz_line_number = 5
window_radius = 0
optimum = find_optimum_coords(
    da=select_lorentz_line(
        restacked_ds.T.sel(**params, method="nearest"),
        lorentz_line_number,
        window_radius,
    ),
    dim="period",
)
plot_RTA(period=float(optimum.period), **params).opts(legend_position="right")

In [ ]:
params = {"theta": 0, "excitonic_layer_thickness": 100, "num_periods": 30}
lorentz_line_number = 0
window_radius = 0
optimum = find_optimum_coords(
    da=select_lorentz_line(
        restacked_ds.T.sel(**params, method="nearest"),
        lorentz_line_number,
        window_radius,
    ),
    dim="period",
)
plot_RTA(period=float(optimum.period), **params)

#### Minimum

Where it is possible to, the dominant strategy is to layer up so much material that almost all the light is absorbed. Where it is not possible, the dominant strategy still involves the maximum number of layers, but the thickness of the layers may be reduced to form a higher-quality PBG. This can especially be the case when considering a wider window. Generally, the LOPC has the capacity to reduce transmittance relative to the CRS.

In [ ]:
dmap[('T', 'restacked_ds', 'min', 0, 0, 0, 'cividis', 10, 450)]

In [ ]:
dmap[('T', 'restacked_ds', 'min', 0, 2, 0, 'cividis', 10, 450)]

In [ ]:
dmap[('T', 'restacked_ds', 'min', -5, 0, 0, 'cividis', 10, 450)]

In [ ]:
dmap[('T', 'restacked_ds', 'min', -5, 2, 0, 'cividis', 10, 450)]

In [ ]:
dmap[('T', 'restacked_ds', 'min', 5, 0, 0, 'cividis', 10, 450)]

In [ ]:
dmap[('T', 'restacked_ds', 'min', 5, 2, 0, 'cividis', 10, 450)]

In [ ]:
# limit the total excitonic thickness so that absorption doesn't saturate
temp_ds = restacked_ds.drop_sel(excitonic_layer_thickness=0).sel(excitonic_layer_thickness=slice(None, 20), num_periods=slice(None, 20))
wrapped_2D_plot('T', temp_ds, 'min', 0, 0, 0, 'cividis', 10, 450, dim="period")

#### Angle behaviour

##### As expected, the optimal structure for minimising transmittance near the resonance doesn't really change at higher angles, because absorption is isotropic.

In [ ]:
angle_dmap[('T', 'restacked_ds', 'min', 0, 2, 30, 'cividis', 10, 450, 0)]

Far from the resonance, the reflectance plays a role. As expected, the optimal period is higher than at normal incidence.

In [ ]:
angle_dmap[('T', 'restacked_ds', 'min', -5, 2, 30, 'cividis', 10, 450, 0)]

In [ ]:
angle_dmap[('T', 'restacked_ds', 'min', 5, 2, 30, 'cividis', 10, 450, 0)]

##### When integrating...

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

angle_dmap[('T', 'restacked_ds', 'min', 0, 2, 0, 'cividis', 10, 450, 75)]

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

angle_dmap[('T', 'restacked_ds', 'min', -5, 2, 0, 'cividis', 10, 450, 75)]

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

angle_dmap[('T', 'restacked_ds', 'min', 5, 2, 0, 'cividis', 10, 450, 75)]

### Optimal structures for absorptance

#### Maximum

If it is possible to just pile up layers to approach 100% absorptance, that is the dominant strategy. There is a secondary effect in minimising reflectance.

If it is not possible, then slow light enhancement can play a role in increasing absorptance, both at the LO resonance and further away. Additionally, the advantage of the photonic effect can be competitive with just increasing layer thickness (see the plot at -8 Lorentz linewidths, where high absorptance is possible at an excitonic layer thickness of 100 nm as well as 150 nm). However, photonic effects play a smaller role when considering larger integration windows, because the range of the slow light absorption enhancement is limited.

Interestingly, when maximising the absorptance near the resonance, the PBG may be located quite far away (~100 nm) from the target wavelength for enhancement.

In [ ]:
dmap[('A', 'restacked_ds', 'max', 0, 0, 0, 'inferno', 10, 450)]

In [ ]:
dmap[('A', 'restacked_ds', 'max', 0, 4, 0, 'inferno', 10, 450)]

In [ ]:
# limit the total excitonic thickness so that absorption doesn't saturate
temp_ds = restacked_ds.drop_sel(excitonic_layer_thickness=0).sel(excitonic_layer_thickness=slice(None, 20), num_periods=slice(None, 20))
wrapped_2D_plot('A', temp_ds, 'max', 0, 0, 0, 'inferno', 10, 450, dim="period", extra_plots=["RTA_int", "norm_1_int"])

In [ ]:
dmap[('A', 'restacked_ds', 'max', -8, 0, 0, 'inferno', 10, 450)]

In [ ]:
# limit the total excitonic thickness so that absorption doesn't saturate
temp_ds = restacked_ds.drop_sel(excitonic_layer_thickness=0).sel(excitonic_layer_thickness=slice(None, 140), num_periods=slice(None, None))
wrapped_2D_plot('A', temp_ds, 'max', -8, 0, 0, 'inferno', 10, 450, dim="period", extra_plots=["RTA_int", "norm_1_int"])

It's interesting that the second-order PBG at 400 nm is more effective at enhancing absorption than the first-order one at 190 nm.

In [ ]:
# limit the total excitonic thickness so that absorption doesn't saturate
temp_ds = restacked_ds.drop_sel(excitonic_layer_thickness=0).sel(excitonic_layer_thickness=slice(None, 140), num_periods=slice(None, None))
wrapped_2D_plot('A', temp_ds, 'max', -8, 0, 0, 'inferno', 10, 300, dim="period", extra_plots=["RTA_int", "norm_1_int"])

In [ ]:
dmap[('A', 'restacked_ds', 'max', 8, 0, 0, 'inferno', 10, 450)]

In [ ]:
dmap[('A', 'restacked_ds', 'max', -8, 2, 0, 'inferno', 10, 450)]

In [ ]:
dmap[('A', 'restacked_ds', 'max', 6, 2, 0, 'inferno', 10, 450)]

#### Minimum

Obviously, the minimal absorptance comes with the smallest structures. There isn't much interesting to say about this.

In [ ]:
dmap[('A', 'restacked_ds', 'min', 0, 0, 0, 'inferno', 10, 450)]

#### Angle behaviour

##### At higher angles, the optimal period is higher. The PBG can reduce absorptance at normal incidence while increasing it at 30 degrees.

In [ ]:
angle_dmap[('A', 'restacked_ds', 'max', 0, 2, 30, 'inferno', 10, 450, 0)]

In [ ]:
angle_dmap[('A', 'restacked_ds', 'max', -5, 2, 30, 'inferno', 10, 450, 0)]

In [ ]:
angle_dmap[('A', 'restacked_ds', 'max', 6, 2, 30, 'inferno', 10, 450, 0)]

##### When integrating, the capacity for photonic effects to meaningfully enhance absorptance overall is very low, because of their localised nature.

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

angle_dmap[('A', 'restacked_ds', 'max', 0, 2, 0, 'inferno', 10, 450, 75)]

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

angle_dmap[('A', 'restacked_ds', 'max', -5, 2, 0, 'inferno', 10, 450, 75)]

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

angle_dmap[('A', 'restacked_ds', 'max', 6, 2, 0, 'inferno', 10, 450, 75)]

### General note on per-oscillator metrics

When the metric increases in absolute terms with layer thickness (e.g. reflectance, absorptance), I expect at the high-thickness limit for there to be an inverse relationship between that metric on a per-oscillator basis and the total number of oscillators (equivalent up to normalisation with total excitonic layer thickness). The per-oscillator metric will also be low at low layer thicknesses, so the optimum is expected at intermediate values.

What affects the position of the optimum in parameter space, and how, is an open question,

### Optimal structures for reflectance per oscillator

#### Maximum

Reflectance per oscillator is kind of as expected. At low total excitonic layer thickness, there is little reflectance, and at high total excitonic layer thickness the reflectance saturates while the number of oscillators increases without bound, so the peak in reflectance per oscillator is at intermediate values of excitonic layer thickness and number of layers. The interesting thing is that for the LO resonance wavelength, the optimum in this parameter ($16 \times 20\ \text{nm} = 320\ \text{nm}$) is in a similar range to the real iridoplast structure.

The behaviour around the LO resonance is assymmetric; there is a shift to significantly higher total excitonic layer thickness at the -2 Lorentz line ($26 \times 40\ \text{nm} = 1040\ \text{nm}$), but not at the +2 Lorentz line ($14 \times 30\ \text{nm} = 420\ \text{nm}$). Generally, the maximum achievable reflectance per oscillator is lower on the blue (low contrast) side of the resonance than on the red (high contrast) side, which holds true up to at least 8 linewidths away. The total excitonic layer thickness that gives the maximal reflectance per oscillator tends to increase with distance from the LO resonance wavelength.

**It would be cool to plot the maximum R_per_oscillator and corresponding total excitonic layer thickness for each Lorentz line.**

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

dmap[('R_per_oscillator', 'restacked_ds', 'max', 0, 0, 0, 'viridis', 10, 450)]

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

dmap[('R_per_oscillator', 'restacked_ds', 'max', -2, 0, 0, 'viridis', 10, 450)]

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

dmap[('R_per_oscillator', 'restacked_ds', 'max', 2, 0, 0, 'viridis', 10, 450)]

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

dmap[('R_per_oscillator', 'restacked_ds', 'max', -5, 0, 0, 'viridis', 10, 450)]

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

dmap[('R_per_oscillator', 'restacked_ds', 'max', 5, 0, 0, 'viridis', 10, 450)]

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

dmap[('R_per_oscillator', 'restacked_ds', 'max', -8, 0, 0, 'viridis', 10, 450)]

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

dmap[('R_per_oscillator', 'restacked_ds', 'max', 8, 0, 0, 'viridis', 10, 450)]

#### Maximum over a window

Taking a larger integration window pushes the optimum towards higher layer thicknesses but lower numbers of layers, leading to a wider and flatter PBG. The optimal PBG can't cover more than about 10 linewidths on the red (high contrast) side or 6 linewidths on the blue (low contrast) side.

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

dmap[('R_per_oscillator', 'restacked_ds', 'max', 0, 2, 0, 'viridis', 10, 450)]

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

dmap[('R_per_oscillator', 'restacked_ds', 'max', 0, 5, 0, 'viridis', 10, 450)]

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

dmap[('R_per_oscillator', 'restacked_ds', 'max', 0, 8, 0, 'viridis', 10, 450)]

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# dmap[('R_per_oscillator', 'restacked_ds', 'max', -7, 3, 0, 'viridis', 10, 450)]

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# dmap[('R_per_oscillator', 'restacked_ds', 'max', 3, 5, 0, 'viridis', 10, 450)]

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

dmap[('R_per_oscillator', 'restacked_ds', 'max', -5, 2, 0, 'viridis', 10, 450)]

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

dmap[('R_per_oscillator', 'restacked_ds', 'max', 5, 2, 0, 'viridis', 10, 450)]

#### Minimum

Obviously, minimising the reflectance per oscillator produces uninteresting structures that typically have low layer thickness and high numbers of layers.

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

dmap[('R_per_oscillator', 'restacked_ds', 'min', 0, 0, 0, 'viridis', 10, 450)]

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

dmap[('R_per_oscillator', 'restacked_ds', 'min', -5, 0, 0, 'viridis', 10, 450)]

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

dmap[('R_per_oscillator', 'restacked_ds', 'min', 5, 0, 0, 'viridis', 10, 450)]

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

dmap[('R_per_oscillator', 'restacked_ds', 'min', 0, 2, 0, 'viridis', 10, 450)]

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

dmap[('R_per_oscillator', 'restacked_ds', 'min', -5, 2, 0, 'viridis', 10, 450)]

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

dmap[('R_per_oscillator', 'restacked_ds', 'min', 5, 2, 0, 'viridis', 10, 450)]

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

angle_dmap[('R_per_oscillator', 'restacked_ds', 'min', 0, 2, 0, 'viridis', 10, 450, 75)]

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

angle_dmap[('R_per_oscillator', 'restacked_ds', 'min', -5, 2, 0, 'viridis', 10, 450, 75)]

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

angle_dmap[('R_per_oscillator', 'restacked_ds', 'min', 5, 2, 0, 'viridis', 10, 450, 75)]

#### Angle behaviour

##### At 30 degrees

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

angle_dmap[('R_per_oscillator', 'restacked_ds', 'max', 0, 2, 30, 'viridis', 10, 450, 0)]

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

angle_dmap[('R_per_oscillator', 'restacked_ds', 'max', -5, 2, 30, 'viridis', 10, 450, 0)]

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

angle_dmap[('R_per_oscillator', 'restacked_ds', 'max', 5, 2, 30, 'viridis', 10, 450, 0)]

##### Integrating (angle only)

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

angle_dmap[('R_per_oscillator', 'restacked_ds', 'max', 0, 0, 0, 'viridis', 10, 450, 75)]

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

angle_dmap[('R_per_oscillator', 'restacked_ds', 'max', -5, 0, 0, 'viridis', 10, 450, 75)]

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

angle_dmap[('R_per_oscillator', 'restacked_ds', 'max', 5, 0, 0, 'viridis', 10, 450, 75)]

##### Integrating (angle and wavelength)

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

angle_dmap[('R_per_oscillator', 'restacked_ds', 'max', 0, 2, 0, 'viridis', 10, 450, 75)]

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

angle_dmap[('R_per_oscillator', 'restacked_ds', 'max', -5, 2, 0, 'viridis', 10, 450, 75)]

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

angle_dmap[('R_per_oscillator', 'restacked_ds', 'max', 5, 2, 0, 'viridis', 10, 450, 75)]

##### Integrating (angle and *wide* wavelength)

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

angle_dmap[('R_per_oscillator', 'restacked_ds', 'max', 0, 8, 0, 'viridis', 10, 450, 75)]

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

angle_dmap[('R_per_oscillator', 'restacked_ds', 'max', -5, 8, 0, 'viridis', 10, 450, 75)]

### Optimal structures for transmittance per oscillator

#### Maximum

Obviously, the maximum transmittance per oscillator occurs when there is the minimum possible amount of absorbing material present.

#### Minimum

Obviously, the minimum transmittance per oscillator occurs when there is lots of absorbing material present. However, even near the resonance, photonic effects can play a role with a wider window. Far from the resonance, photonic effects dominate over intrinsic absorptance.

What is not entirely obvious is that minimising transmittance per oscillator is *not* the same as maximising the sum of (reflectance per oscillator) and (absorptance per oscillator). The latter will tend to produce optima in the intermediate region. I might get onto doing calculations of this at some point, but it's not as useful as it may appear because in a real situation, you would need to know how to weight the relative importance of these two metrics.

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

dmap[('T_per_oscillator', 'restacked_ds', 'min', 0, 6, 0, 'cividis', 10, 450)]

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

dmap[('T_per_oscillator', 'restacked_ds', 'min', -8, 0, 0, 'cividis', 10, 450)]

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

dmap[('T_per_oscillator', 'restacked_ds', 'min', -8, 6, 0, 'cividis', 10, 450)]

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

dmap[('T_per_oscillator', 'restacked_ds', 'min', 4, 4, 0, 'cividis', 10, 450)]

### Optimal structures for absorptance per oscillator

#### Maximum

Near the LO resonance, the maximum absorptance per oscillator comes with almost the minimum amount of absorbing material - photonic effects aren't helpful beyond the slight absorption increase that comes from having two layers rather than one. However, further from the resonance (at least -4 linewidths below or 2 above) there is the possibility of improving absorptance per oscillator using the photonic structure.

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

dmap[('A_per_oscillator', 'restacked_ds', 'max', 0, 0, 0, 'inferno', 10, 450)]

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

dmap[('A_per_oscillator', 'restacked_ds', 'max', -4, 0, 0, 'inferno', 10, 450)]

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

dmap[('A_per_oscillator', 'restacked_ds', 'max', -8, 0, 0, 'inferno', 10, 450)]

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

dmap[('A_per_oscillator', 'restacked_ds', 'max', 2, 0, 0, 'inferno', 10, 450)]

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

dmap[('A_per_oscillator', 'restacked_ds', 'max', 4, 0, 0, 'inferno', 10, 450)]

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

dmap[('A_per_oscillator', 'restacked_ds', 'max', 8, 0, 0, 'inferno', 10, 450)]

#### Maximum over a window

The absorptance per oscillator is still not improved over window centred on the resonance, even when the window includes wavelengths where a photonic enhancement is possible. Further from the resonance, widening the window generally moves the optimum to higher layer thicknesses and lower numbers of periods, similarly to reflectance per oscillator. This is related to the idea of widening the range of photonic effects at the expense of their magnitude.

The photonic enhancement 'washes out' faster on the blue side of the resonance, because the region of absorption enhancement due to the slow light effect doesn't extend very far with low RI contrast. On the red side, the photonic enhancement is evident over wider windows. However, when considering regions very far from the LO resonance, where the intrinsic absorption of the CRS is very low, it is possible for high reflectance to coexist with a (marginally) higher absorptance per oscillator and some degree of absorption enhancement - see plot at -10 linewidths over a window of radius 5.

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

# this would be better if the window size was 8
# dmap[('A_per_oscillator', 'restacked_ds', 'max', 0, 4, 0, 'inferno', 10, 450)]

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

dmap[('A_per_oscillator', 'restacked_ds', 'max', -4, 1, 0, 'inferno', 10, 450)]

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

dmap[('A_per_oscillator', 'restacked_ds', 'max', -4, 2, 0, 'inferno', 10, 450)]

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

dmap[('A_per_oscillator', 'restacked_ds', 'max', -5, 2, 0, 'inferno', 10, 450)]

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

dmap[('A_per_oscillator', 'restacked_ds', 'max', -8, 1, 0, 'inferno', 10, 450)]

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

dmap[('A_per_oscillator', 'restacked_ds', 'max', -10, 5, 0, 'inferno', 10, 450)]

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

dmap[('A_per_oscillator', 'restacked_ds', 'max', 4, 3, 0, 'inferno', 10, 450)]

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

dmap[('A_per_oscillator', 'restacked_ds', 'max', 5, 2, 0, 'inferno', 10, 450)]

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

dmap[('A_per_oscillator', 'restacked_ds', 'max', 6, 2, 0, 'inferno', 10, 450)]

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

dmap[('A_per_oscillator', 'restacked_ds', 'max', 7, 1, 0, 'inferno', 10, 450)]

#### Minimum

To minimise the absorptance per oscillator, the strategy is to maximise the amount of material and lay a PBG over the target wavelength range. The reduction in absorptance caused by the PBG can be more significant than the reduction in oscillator number from having thicker layers, but the number of layers is always high.

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

dmap[('A_per_oscillator', 'restacked_ds', 'min', 0, 2, 0, 'inferno', 10, 450)]

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

dmap[('A_per_oscillator', 'restacked_ds', 'min', 4, 0, 0, 'inferno', 10, 450)]

#### Angle behaviour

##### At 30 degrees

The optimal period is higher than at normal incidence.

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

angle_dmap[('A_per_oscillator', 'restacked_ds', 'max', 0, 2, 30, 'inferno', 10, 450, 0)]

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

angle_dmap[('A_per_oscillator', 'restacked_ds', 'max', -5, 2, 30, 'inferno', 10, 450, 0)]

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

angle_dmap[('A_per_oscillator', 'restacked_ds', 'max', 5, 2, 30, 'inferno', 10, 450, 0)]

##### Integrating

The photonic advantage disappears because it is localised to a particular angle/wavelength range.

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

angle_dmap[('A_per_oscillator', 'restacked_ds', 'max', 0, 2, 0, 'inferno', 10, 450, 75)]

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

angle_dmap[('A_per_oscillator', 'restacked_ds', 'max', -5, 2, 0, 'inferno', 10, 450, 75)]

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

angle_dmap[('A_per_oscillator', 'restacked_ds', 'max', 5, 2, 0, 'inferno', 10, 450, 75)]

## Enhancement factors

### Investigation

In [ ]:
# Plot absorptance enhancement factor as a function of wavelength and passive layer thickness for a choice of excitonic layer thickness, theta, and number of periods

sel = norm_1.drop_sel(excitonic_layer_thickness=0).interactive(loc="left")

xs_1D = {
    "excitonic_layer_thickness": pnw.DiscreteSlider,
    "theta": pnw.DiscreteSlider,
    "num_periods": pnw.DiscreteSlider,
    "passive_layer_thickness": pnw.DiscreteSlider,
}

sel_1 = sel.sel(**xs_1D).squeeze()

# Get a handle on the widgets
widgets = {k: w for k, w in zip(xs_1D, sel_1.widgets())}

xs_2D = {k: w for k, w in widgets.items() if k not in {"passive_layer_thickness"}}
sel_2 = sel.sel(**xs_2D).squeeze()

(
    sel_1.A.hvplot(x="wavelength", ylim=(0.5, 1.5))
    * hv.HLine(1).opts(opts.HLine(line_dash="dotted"))
    + sel_2.A.hvplot(x="wavelength", y="passive_layer_thickness", clim=(0.7, 1.3), cmap="RdBu_r")
    # * hv.DynamicMap(lambda y: hv.HLine(y), streams={"y": widgets["passive_layer_thickness"].param.value})
).cols(1)

In [ ]:
# Plot absorptance enhancement factor as a function of wavelength and *period* for a choice of excitonic layer thickness, theta, and number of periods

sel = restacked_norm_1.drop_sel(excitonic_layer_thickness=0).interactive(loc="left")

xs_1D = {
    "excitonic_layer_thickness": pnw.DiscreteSlider,
    "theta": pnw.DiscreteSlider,
    "num_periods": pnw.DiscreteSlider,
    "period": pnw.DiscreteSlider,
}

sel_1 = sel.sel(**xs_1D).squeeze()

# Get a handle on the widgets
widgets = {k: w for k, w in zip(xs_1D, sel_1.widgets())}

xs_2D = {k: w for k, w in widgets.items() if k not in {"period"}}
sel_2 = sel.sel(**xs_2D).squeeze()

(
    sel_1.A.hvplot(x="wavelength", ylim=(0.5, 1.5))
    * hv.HLine(1).opts(opts.HLine(line_dash="dotted"))
    + sel_2.A.dropna("period").hvplot(x="wavelength", y="period", clim=(0.7, 1.3), cmap="RdBu_r")
).cols(1)

In [ ]:
# Plot absorptance enhancement factor as a function of wavelength and *number of periods* for a choice of excitonic layer thickness, theta, and period

sel = restacked_norm_1.drop_sel(excitonic_layer_thickness=0).interactive(loc="left")

xs_1D = {
    "excitonic_layer_thickness": pnw.DiscreteSlider,
    "theta": pnw.DiscreteSlider,
    "num_periods": pnw.DiscreteSlider,
    "period": pnw.DiscreteSlider,
}

sel_1 = sel.sel(**xs_1D).squeeze()

# Get a handle on the widgets
widgets = {k: w for k, w in zip(xs_1D, sel_1.widgets())}

xs_2D = {k: w for k, w in widgets.items() if k not in {"num_periods"}}
sel_2 = sel.sel(**xs_2D).squeeze()

(
    sel_1.A.hvplot(x="wavelength", ylim=(0.5, 1.5))
    * hv.HLine(1).opts(opts.HLine(line_dash="dotted"))
    + sel_2.A.hvplot.quadmesh(x="wavelength", y="num_periods", clim=(0.3, 1.7), cmap="RdBu_r")
).cols(1)

In [ ]:
with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    crs_1_like_ds = (
        crs_1#.drop("passive_layer_thickness")
        .expand_dims(passive_layer_thickness=ds.passive_layer_thickness)
        .sel(total_excitonic_thickness=ds.total_excitonic_thickness)
    )

In [ ]:
crs_1_like_ds = assign_derived_attrs(
    crs_1_like_ds,
    absorption=False,
    unpolarised=False,
    period=True,
    total_excitonic_thickness=False,
    total_passive_thickness=False,
    total_thickness=False,
    N_tot=False,
)

In [ ]:
with dask.config.set(**{"array.slicing.split_large_chunks": False}):
    restacked_crs_1 = (
        crs_1_like_ds.stack(
            multiperiod=["passive_layer_thickness", "excitonic_layer_thickness"]
        )
        .set_index(multiperiod=["period", "excitonic_layer_thickness"])
        .unstack()
    )

In [ ]:
# Workaround to the bug below: pull the widget values directly
widget_vals = {k: v.value for k, v in widgets.items()}

curve_1 = (
    restacked_ds.sel(**widget_vals).squeeze().A.hvplot(x="wavelength", ylim=(0, 1), label="A", group_label="LOPC")
)

curve_11 = (
    restacked_ds.sel(**widget_vals).squeeze().R.hvplot(x="wavelength", ylim=(0, 1), label="R", group_label="LOPC")
)

curve_2 = (
    restacked_crs_1.sel(**widget_vals).squeeze().A.hvplot(x="wavelength", ylim=(0, 1), label="A", group_label="CRS")
).opts(line_dash="dashed")

curve_21 = (
    restacked_crs_1.sel(**widget_vals).squeeze().R.hvplot(x="wavelength", ylim=(0, 1), label="R", group_label="CRS")
).opts(line_dash="dashed")

curve_3 = (
    (restacked_ds.sel(**widget_vals) / restacked_crs_1.sel(**widget_vals))
    .squeeze()
    .A.hvplot(x="wavelength", ylim=(0.5, 1.5))
)

((curve_1 * curve_11 *curve_2 * curve_21) + curve_3 * hv.HLine(1).opts(line_dash="dashed")).cols(1).opts(
    shared_axes=False
)

### Optimisation

In [ ]:
def wrapped_2D_ef_plot(
    variable,
    dataset,
    ref,
    optimise="max",
    lorentz_line=0,
    window_radius=0,
    theta=0,
    cmap="viridis",
    period_start=None,
    period_stop=None,
    integrate_angle: float = None,
    extra_plots=["RTA_normal", "RTA_int", "norm_1_normal", "norm_1_int"],
    dim=None,  # automatically assign if dataset recognised
    restack_ef=None,
    common_dim=None,
):
    plots = []

    # detect string labels
    if str(dataset) == "ds":
        # the drop_sel is important for avoiding the most common degenerate cases
        dataset = ds.drop_sel({"excitonic_layer_thickness": 0})

    if str(ref) == "crs_1":
        ref = crs_1
        common_dim = "total_excitonic_thickness"

    if str(dataset) == "frs_1":
        ref = frs_1
        common_dim = "total_thickness"
    
    match dim:  # if restack is the default value, automatically set it
        case "period":
            restack_ef = True if restack_ef is None else restack_ef
        case "passive_layer_thickness":
            restack_ef = False if restack_ef is None else restack_ef
    integration_dims = []
    vline_locs = [0]

    da = dataset[variable]
    ra = ref[variable]

    da = select_lorentz_line(da, lorentz_line=lorentz_line, window_radius=window_radius)
    ra = select_lorentz_line(ra, lorentz_line=lorentz_line, window_radius=window_radius)

    if window_radius == 0:
        wavelength = float(da.wavelength)
        title = f"{optimise.capitalize()}imum {variable} enhancement factor at {wavelength:.0f} nm"
        if lorentz_line != 0:  # don't put two lines over the resonance
            vline_locs.append(lorentz_line)
    else:
        wavelength_start = float(da.wavelength[0])
        wavelength_stop = float(da.wavelength[-1])
        title = f"{optimise.capitalize()}imum integrated {variable} enhancement factor between {wavelength_start:.0f} and {wavelength_stop:.0f} nm"
        vline_locs.append(lorentz_line - window_radius)
        vline_locs.append(lorentz_line + window_radius)
        integration_dims.append("wavelength")

    if integrate_angle:  # integrate angle should be a float
        th = (theta, integrate_angle)
        da = da.sel(theta=slice(*th))
        ra = ra.sel(theta=slice(*th))
        integration_dims.append("theta")
    else:  # this includes if integrate_angle==0, which is hacky but fine
        da = da.sel(theta=theta, method="nearest")
        ra = ra.sel(theta=theta, method="nearest")
        th = theta

    if integration_dims:  # if the list isn't empty
        da = integrate_da(da, integration_dims, weighting=1, normalisation=None)
        ra = integrate_da(ra, integration_dims, weighting=1, normalisation=None)

    norm = enhancement_factor(ds=da, ref=ra, common_dim=common_dim, method="groupby")

    if (
        window_radius == 0 and not integrate_angle
    ):  # if the precalculated norm_1 dataset will suffice
        norm = norm_1[variable].drop_sel({"excitonic_layer_thickness": 0})  # then use it instead
        norm = select_lorentz_line(norm, lorentz_line=lorentz_line, window_radius=window_radius)
        norm = norm.sel(theta=theta, method="nearest")

    match restack_ef:
        case True:
            norm = restack_plt_to_period(norm)

    if period_start < period_stop:
        norm = norm.sel(period=slice(period_start, period_stop))
    else:  # otherwise no data is selected and everything breaks
        norm = norm.sel(period=slice(period_start, None))

    plot_1, optimum_coords = plot_optimum_over_dim(
        norm,
        dim=dim,
        x="excitonic_layer_thickness",
        y="num_periods",
        optimise=optimise,
    )
    ############################################# code above is fresh, below is stale
    P = float(optimum_coords["period"])
    t = float(optimum_coords["excitonic_layer_thickness"])
    N = float(optimum_coords["num_periods"])
    # try:  # this should work if not integrating over theta
    #     th = float(optimum_coords["theta"])
    # except:  # probably the problem is that theta doesn't exist because I integrated over it already
    #     th = (theta, integrate_angle)
    lorentz_lines = lorentz_vlines(vline_locs, scale=1e-9, mode="wavelength").opts(
        opts.VLine(line_color=green, line_dash="dotted"),
    )

    # give the resonance line a special colour
    lorentz_lines.VLine.I.opts(opts.VLine(line_color=yellow))

    plot_1.opts(
        opts.QuadMesh(cmap=cmap),
        opts.Points(color="red"),
        opts.Overlay(title=f"{title}\nOptimal period: {P:.0f}"),
    )

    plots.append(plot_1)

    if "RTA_normal" in extra_plots:  # plot RTA at theta=0
        new_plot = plot_RTA(
            period=P, excitonic_layer_thickness=t, num_periods=N, theta=0
        )
        new_plot *= lorentz_lines

        plots.append(new_plot)

    if "RTA_int" in extra_plots:  # plot RTA at theta OR integrating over theta
        new_plot = plot_RTA(
            period=P, excitonic_layer_thickness=t, num_periods=N, theta=th
        )
        new_plot *= lorentz_lines
        plots.append(new_plot)

    if "norm_1_normal" in extra_plots:  # plot enhancement factor at theta=0
        sel = {
            "period": P,
            "excitonic_layer_thickness": t,
            "num_periods": N,
            "theta": 0,
        }
        new_plot = (
            plot_ef(variable="A", dataset=restacked_norm_1, sel=sel) * lorentz_lines
        ).opts(opts.Overlay(legend_position="bottom_right"))
        plots.append(new_plot)

    if (
        "norm_1_int" in extra_plots
    ):  # plot enhancement factor at theta OR integrating over theta
        try:  # this should work if not integrating over theta
            sel = {
                "period": P,
                "excitonic_layer_thickness": t,
                "num_periods": N,
                "theta": th,
            }
            new_plot = (
                plot_ef(variable="A", dataset=restacked_norm_1, sel=sel) * lorentz_lines
            ).opts(opts.Overlay(legend_position="bottom_right"))
        except:  # if integrating, we need to do the integral *before* normalising
            ds_int = sel_or_integrate(ds, dim="theta", val=th)
            crs_1_int = sel_or_integrate(crs_1, dim="theta", val=th)
            norm = enhancement_factor(
                ds_int,
                ref=crs_1_int,
                common_dim="total_excitonic_thickness",
                method="groupby",
            )
            restacked_norm = restack_plt_to_period(norm)

            sel = {"period": P, "excitonic_layer_thickness": t, "num_periods": N}
            new_plot = (
                plot_ef(variable="A", dataset=restacked_norm, sel=sel) * lorentz_lines
            ).opts(opts.Overlay(legend_position="bottom_right"))
        plots.append(new_plot)

    return hv.Layout(plots).cols(1)

#### Tests

This should give exactly the same output as an equivalent call I could make using `wrapped_2D_plot` and `restacked_norm_1`.

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

layout = wrapped_2D_ef_plot(
    "As",
    ds.drop_sel(excitonic_layer_thickness=0),
    crs_1,
    optimise="max",
    lorentz_line=-3,
    window_radius=0,
    theta=0,
    cmap="RdBu_r",
    period_start=0,
    period_stop=1000,
    integrate_angle=0,
    extra_plots=["RTA_normal", "RTA_int", "norm_1_normal", "norm_1_int"
                ],
    dim="period",  # automatically assign if dataset recognised
    restack_ef=None,
    common_dim="total_excitonic_thickness",
)

layout

Now doing an angle integral, the results diverge.excitonic_layer_thickness

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

layout = wrapped_2D_ef_plot(
    "As",
    ds.drop_sel(excitonic_layer_thickness=0),
    crs_1,
    optimise="max",
    lorentz_line=-3,
    window_radius=0,
    theta=0,
    cmap="RdBu_r",
    period_start=0,
    period_stop=1000,
    integrate_angle=50,
    extra_plots=["RTA_normal", "RTA_int", "norm_1_normal", "norm_1_int"
                ],
    dim="period",  # automatically assign if dataset recognised
    restack_ef=None,
    common_dim="total_excitonic_thickness",
)

layout

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

layout = wrapped_2D_plot(
    "As",
    restacked_norm_1.drop_sel(excitonic_layer_thickness=0),
    optimise="max",
    lorentz_line=-3,
    window_radius=0,
    theta=0,
    cmap="RdBu_r",
    period_start=0,
    period_stop=1000,
    integrate_angle=50,
    extra_plots=["RTA_normal", "RTA_int", "norm_1_normal", "norm_1_int"
                ],
    dim="period",  # automatically assign if dataset recognised
    # restack_ef=None,
    # common_dim="total_excitonic_thickness",
)

layout

Integrating over wavelength too, the divergence is greater.

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

layout = wrapped_2D_ef_plot(
    "As",
    ds.drop_sel(excitonic_layer_thickness=0),
    crs_1,
    optimise="max",
    lorentz_line=-3,
    window_radius=2,
    theta=0,
    cmap="RdBu_r",
    period_start=0,
    period_stop=1000,
    integrate_angle=50,
    extra_plots=["RTA_normal", "RTA_int", "norm_1_normal", "norm_1_int"
                ],
    dim="period",  # automatically assign if dataset recognised
    restack_ef=None,
    common_dim="total_excitonic_thickness",
)

layout

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

layout = wrapped_2D_plot(
    "As",
    restacked_norm_1.drop_sel(excitonic_layer_thickness=0),
    optimise="max",
    lorentz_line=-3,
    window_radius=2,
    theta=0,
    cmap="RdBu_r",
    period_start=0,
    period_stop=1000,
    integrate_angle=50,
    extra_plots=["RTA_normal", "RTA_int", "norm_1_normal", "norm_1_int"
                ],
    dim="period",  # automatically assign if dataset recognised
    # restack_ef=None,
    # common_dim="total_excitonic_thickness",
)

layout

#### Serious business

In [ ]:
periods = restacked_ds.period.values[1:-1]
# don't set the maximum period to be lower than the maximum excitonic layer thickness, or it will break the plot!
safe_periods = restacked_ds.period.sel(
    period=slice(restacked_ds.excitonic_layer_thickness.values[-1], None)
).values

kdims = [
    hv.Dimension("variable", values=restacked_norm_1.data_vars),
    hv.Dimension("dataset", values=["ds"]),
    hv.Dimension("ref", values=["crs_1", "frs_1"]),
    hv.Dimension("optimise", values=["max", "min"]),
    hv.Dimension("lorentz_line", range=(-15, 8), default=0),
    hv.Dimension("window_radius", range=(0, 8), default=0),
    hv.Dimension("theta", range=(0, 86), default=0),
    hv.Dimension("cmap", values=["viridis", "cividis", "inferno", "PRGn", "PuOr_r", "RdBu_r"]),
    hv.Dimension("period_start", values=periods, default=periods[0]),
    hv.Dimension("period_stop", values=safe_periods, default=periods[-1]),
    hv.Dimension("integrate_angle", range=(0, 86), default=0),
]


angle_ef_dmap = hv.DynamicMap(
    partial(
        wrapped_2D_ef_plot,
        extra_plots=["RTA_normal", "RTA_int", "norm_1_normal", "norm_1_int"],
        dim="period",
    ),
    kdims=kdims,
).opts(
    opts.Overlay(legend_position="right", legend_opts={"background_fill_alpha": 0.5})
)
# if I'm just looking at normal incidence the extra plots are redundant
ef_dmap = hv.DynamicMap(
    partial(
        wrapped_2D_ef_plot,
        integrate_angle=0,
        extra_plots=["RTA_int", "norm_1_int"],
        dim="period",
    ),
    kdims=kdims[:10],
).opts(
    opts.Overlay(legend_position="right", legend_opts={"background_fill_alpha": 0.5})
)

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

# there is some sort of horrible bug that makes the title and axis on the second plot sometime disappear, along with one of the vlines
angle_ef_dmap

In [ ]:
# Use this to record interesting key sets
angle_ef_dmap.current_key

### Reflectance enhancement factor

The plots of max and min reflectance enhancement factor look quite odd, but ultimately are not that interesting from a design perspective.

In [ ]:
# This is just the maximum reflectance structure again
ef_dmap[('R', 'ds', 'crs_1', 'max', 0, 0, 0, 'PRGn', 10, 460)]

The band at 110-120 nm is probably due to two things:
1. The reflectance of the CRS getting very low as the total excitonic thickness becomes larger than the wavelength of light, leading to incoherent reflection;
2. That particular excitonic layer thickness makes creating a reflectance-suppressing photonic effect difficult.

In [ ]:
ef_dmap[('R', 'ds', 'crs_1', 'min', 0, 0, 0, 'PRGn', 10, 460)]

In [ ]:
# why is minimum R enhancement factor so high when excitonic layer thickness is 110 nm?
wrapped_2D_plot('R', restacked_norm_1.sel(excitonic_layer_thickness=slice(110, 120), num_periods=slice(8, None)), 'min', 0, 0, 0, 'viridis', 10, 450, dim="period")

### Transmittance enhancement factor

#### Maximum

The result at the resonance is basically meaningless. The maximal values far from the resonance are very small.

In [ ]:
ef_dmap[('T', 'ds', 'crs_1', 'max', 0, 0, 0, 'PuOr_r', 10, 460)]

In [ ]:
ef_dmap[('T', 'ds', 'crs_1', 'max', -2, 0, 0, 'PuOr_r', 10, 460)]

In [ ]:
ef_dmap[('T', 'ds', 'crs_1', 'max', 2, 0, 0, 'PuOr_r', 10, 460)]

In [ ]:
ef_dmap[('T', 'ds', 'crs_1', 'max', -5, 0, 0, 'PuOr_r', 10, 460)]

In [ ]:
ef_dmap[('T', 'ds', 'crs_1', 'max', 5, 0, 0, 'PuOr_r', 10, 460)]

#### Minimum

The minimum transmittance enhancement factor at the resonance is surprisingly high. This tells us that the light intensity that is diverted to reflectance comes almost entirely at the cost of absorptance, no matter the parameters. (Really a diff would be better than a norm here.)

In [ ]:
ef_dmap[('T', 'ds', 'crs_1', 'min', 0, 0, 0, 'PuOr_r', 10, 460)]

In [ ]:
ef_dmap[('T', 'ds', 'crs_1', 'min', -2, 0, 0, 'PuOr_r', 10, 460)]

In [ ]:
ef_dmap[('T', 'ds', 'crs_1', 'min', 2, 0, 0, 'PuOr_r', 10, 460)]

In [ ]:
ef_dmap[('T', 'ds', 'crs_1', 'min', -5, 0, 0, 'PuOr_r', 10, 460)]

In [ ]:
ef_dmap[('T', 'ds', 'crs_1', 'min', 5, 0, 0, 'PuOr_r', 10, 460)]

### Absorptance enhancement factor

#### Optimal structure for maximising absorptance enhancement near the resonance

The maximal absorptance enhancement near the LO resonance (between the resonance and -1 linewidth from it) is confined to a narrow band around a total excitonic thickness of roughly 100 nm. The maximum possible enhancement is modest and also not precisely targeted (there is a greater enhancement at longer wavelengths). Furthermore, it is not accompanied by a strong reflectance band, because of the low number of layers.

In [ ]:
ef_dmap[('A', 'ds', 'crs_1', 'max', 0, 0, 0, 'inferno', 10, 450)].opts(opts.QuadMesh(clim=(1, None)), clone=True)

In [ ]:
ef_dmap[('A', 'ds', 'crs_1', 'max', 0, 2, 0, 'inferno', 10, 450)].opts(opts.QuadMesh(clim=(1, None)), clone=True)

In [ ]:
ef_dmap[('A', 'ds', 'crs_1', 'max', 0, 5, 0, 'inferno', 10, 450)].opts(opts.QuadMesh(clim=(1, None)), clone=True)

#### Optimal structure for maximising absorption enhancement far from the resonance

Beyond a target wavelength of -2 linewidths below or 1 above the resonance, the optimum starts to move, first towards higher numbers of periods, then thicker layers. Bands of local optima in the 2D plot start to appear, as certain sets of parameters have higher potential. (**Why?**)

As we move further from the resonance and towards regions of lower intrinsic absorptance, the maximum attainable wavelength specific absorptance enhancement increases and the optimal structures that produce these become larger. These structures also have higher reflectance - although local optima with lower reflectance also come into play.

These effects are more pronounced on the red (high-contrast) side of the resonance than the blue side.

In [ ]:
ef_dmap[('A', 'ds', 'crs_1', 'max', -2, 0, 0, 'inferno', 10, 450)].opts(opts.QuadMesh(clim=(1, None)), clone=True)

In [ ]:
ef_dmap[('A', 'ds', 'crs_1', 'max', -5, 0, 0, 'inferno', 10, 450)].opts(opts.QuadMesh(clim=(1, None)), clone=True)

In [ ]:
ef_dmap[('A', 'ds', 'crs_1', 'max', -5, 2, 0, 'inferno', 10, 450)].opts(opts.QuadMesh(clim=(1, None)), clone=True)

In [ ]:
ef_dmap[('A', 'ds', 'crs_1', 'max', -8, 0, 0, 'inferno', 10, 450)].opts(opts.QuadMesh(clim=(1, None)), clone=True)

In [ ]:
ef_dmap[('A', 'ds', 'crs_1', 'max', 2, 0, 0, 'inferno', 10, 450)].opts(opts.QuadMesh(clim=(1, None)), clone=True)
# should do a windowed version of this

In [ ]:
# example of a local optimum with lower reflectance
wrapped_2D_plot('A', restacked_norm_1.sel(excitonic_layer_thickness=slice(20, None)), 'max', 2, 0, 0, 'inferno', 10, 450, dim="period")

In [ ]:
ef_dmap[('A', 'ds', 'crs_1', 'max', 5, 0, 0, 'inferno', 10, 450)].opts(opts.QuadMesh(clim=(1, None)), clone=True)

In [ ]:
ef_dmap[('A', 'ds', 'crs_1', 'max', 5, 2, 0, 'inferno', 10, 450)].opts(opts.QuadMesh(clim=(1, None)), clone=True)

In [ ]:
ef_dmap[('A', 'ds', 'crs_1', 'max', 8, 0, 0, 'inferno', 10, 450)].opts(opts.QuadMesh(clim=(1, None)), clone=True)

#### Optimal structure for minimising absorptance enhancement

Near the resonance, the structures that minimise absorptance enhancement are almost identical to those maximising reflectance, which tells us that reducing absorptance by enhancing transmittance is not really a thing.

The strategy to get the minimal absorptance enhancement far from the resonance is to saturate the absorptance of the CRS, then add a large reflectance band on top. It looks similar to the optimal structure for reflectance. However, we also see the emergence on local minima at lower total excitonic layer thicknesses.

Again, sometimes the second-order PBG is more effective than the first-order one.

In [ ]:
ef_dmap[('A', 'ds', 'crs_1', 'min', 0, 0, 0, 'inferno', 10, 450)].opts(opts.QuadMesh(clim=(None, 1)), clone=True)

In [ ]:
ef_dmap[('A', 'ds', 'crs_1', 'min', 0, 2, 0, 'inferno', 10, 450)].opts(opts.QuadMesh(clim=(None, 1)), clone=True)

In [ ]:
ef_dmap[('A', 'ds', 'crs_1', 'min', 0, 5, 0, 'inferno', 10, 450)].opts(opts.QuadMesh(clim=(None, 1)), clone=True)

In [ ]:
# Compare this to structure for maximal reflectance at this wavelength
ef_dmap[('A', 'ds', 'crs_1', 'min', -2, 0, 0, 'inferno', 10, 450)].opts(opts.QuadMesh(clim=(None, 1)), clone=True)

In [ ]:
# This is exactly the structure for maximal reflectance at this wavelength
ef_dmap[('A', 'ds', 'crs_1', 'min', 1, 0, 0, 'inferno', 10, 450)].opts(opts.QuadMesh(clim=(None, 1)), clone=True)

In [ ]:
# This is exactly the structure for maximal reflectance at this wavelength - but note the local minimum at lower thicknesses!
ef_dmap[('A', 'ds', 'crs_1', 'min', -4, 0, 0, 'inferno', 10, 450)].opts(opts.QuadMesh(clim=(None, 1)), clone=True)

In [ ]:
ef_dmap[('A', 'ds', 'crs_1', 'min', -8, 0, 0, 'inferno', 10, 450)].opts(opts.QuadMesh(clim=(None, 1)), clone=True)

In [ ]:
# Note the difference when we exclude the second-order PBG
ef_dmap[('A', 'ds', 'crs_1', 'min', -8, 0, 0, 'inferno', 10, 400)].opts(opts.QuadMesh(clim=(None, 1)), clone=True)

In [ ]:
# This is similar to the structure for maximal reflectance at this wavelength
ef_dmap[('A', 'ds', 'crs_1', 'min', 8, 0, 0, 'inferno', 10, 450)].opts(opts.QuadMesh(clim=(None, 1)), clone=True)

In [ ]:
ef_dmap[('A', 'ds', 'crs_1', 'min', -5, 0, 0, 'inferno', 10, 450)].opts(opts.QuadMesh(clim=(None, 1)), clone=True)

In [ ]:
ef_dmap[('A', 'ds', 'crs_1', 'min', -5, 2, 0, 'inferno', 10, 450)].opts(opts.QuadMesh(clim=(None, 1)), clone=True)

In [ ]:
ef_dmap[('A', 'ds', 'crs_1', 'min', -5, 5, 0, 'inferno', 10, 450)].opts(opts.QuadMesh(clim=(None, 1)), clone=True)

In [ ]:
ef_dmap[('A', 'ds', 'crs_1', 'min', 5, 0, 0, 'inferno', 10, 450)].opts(opts.QuadMesh(clim=(None, 1)), clone=True)

In [ ]:
ef_dmap[('A', 'ds', 'crs_1', 'min', 5, 2, 0, 'inferno', 10, 450)].opts(opts.QuadMesh(clim=(None, 1)), clone=True)

In [ ]:
# don't trust this - integral extends off the edge of the plot
ef_dmap[('A', 'ds', 'crs_1', 'min', 5, 5, 0, 'inferno', 10, 450)].opts(opts.QuadMesh(clim=(None, 1)), clone=True)

#### Angle behaviour

##### At 30 degrees - higher period, otherwise pretty similar to normal incidence

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

angle_ef_dmap[('A', 'ds', 'crs_1', 'max', 0, 2, 30, 'inferno', 10, 450, 0)].opts(opts.QuadMesh(clim=(1, None)), clone=True)

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

angle_ef_dmap[('A', 'ds', 'crs_1', 'max', -5, 2, 30, 'inferno', 10, 450, 0)].opts(opts.QuadMesh(clim=(1, None)), clone=True)

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

angle_ef_dmap[('A', 'ds', 'crs_1', 'max', 5, 2, 30, 'inferno', 10, 450, 0)].opts(opts.QuadMesh(clim=(1, None)), clone=True)

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

angle_ef_dmap[('A', 'ds', 'crs_1', 'min', 0, 2, 30, 'inferno', 10, 450, 0)].opts(opts.QuadMesh(clim=(None, 1)), clone=True)

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

angle_ef_dmap[('A', 'ds', 'crs_1', 'min', -5, 2, 30, 'inferno', 10, 450, 0)].opts(opts.QuadMesh(clim=(None, 1)), clone=True)

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

angle_ef_dmap[('A', 'ds', 'crs_1', 'min', 5, 2, 30, 'inferno', 10, 450, 0)].opts(opts.QuadMesh(clim=(None, 1)), clone=True)

##### Integrating (max)

At the resonance, the structure is actually very similar to the case at normal incidence - because the absorption enhancement isn't really coming from a photonic effect, but from the incoherent scattering. The period is slightly higher. The number of layers is higher but the total excitonic thickness is nearly the same. When increasing the wavelength window, the only difference is to increase the number of layers further (thus strengthening the optical path length enhancement at further wavelengths).

Far from resonance, the optimal number of layers and thickness are both much lower after an angle integral, indicating that the slow light enhancement becomes less important and the incoherent scattering more important.

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

angle_ef_dmap[('A', 'ds', 'crs_1', 'max', 0, 2, 0, 'inferno', 10, 450, 75)].opts(opts.QuadMesh(clim=(1, None)), clone=True)

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

angle_ef_dmap[('A', 'ds', 'crs_1', 'max', 0, 5, 0, 'inferno', 10, 450, 75)].opts(opts.QuadMesh(clim=(1, None)), clone=True)

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

angle_ef_dmap[('A', 'ds', 'crs_1', 'max', -5, 2, 0, 'inferno', 10, 450, 75)].opts(opts.QuadMesh(clim=(1, None)), clone=True)

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

angle_ef_dmap[('A', 'ds', 'crs_1', 'max', 5, 2, 0, 'inferno', 10, 450, 75)].opts(opts.QuadMesh(clim=(1, None)), clone=True)

##### Integrating (min)

At resonance, the optimal parameters to minimise the angle-integrated absorptance enhancement factor are basically the same as when not integrating over angle. The period and layer thickness are slightly higher. Far from resonance, again the periods are slightly higher and the excitonic layer thicknesses somewhat higher.

What is quite notable is that when integrating over angle, it becomes very difficult to actually obtain an absorptance enhancement factor <1 for low total excitonic thickness. In other words, a small iridoplast-like structure can’t be built such that it has less absorption than its CRS.

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

angle_ef_dmap[('A', 'ds', 'crs_1', 'min', 0, 2, 0, 'inferno', 10, 450, 75)].opts(opts.QuadMesh(clim=(None, 1)), clone=True)

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

angle_ef_dmap[('A', 'ds', 'crs_1', 'min', 0, 5, 0, 'inferno', 10, 450, 75)].opts(opts.QuadMesh(clim=(None, 1)), clone=True)

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

angle_ef_dmap[('A', 'ds', 'crs_1', 'min', -5, 2, 0, 'inferno', 10, 450, 75)].opts(opts.QuadMesh(clim=(None, 1)), clone=True)

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

angle_ef_dmap[('A', 'ds', 'crs_1', 'min', 5, 2, 0, 'inferno', 10, 450, 75)].opts(opts.QuadMesh(clim=(None, 1)), clone=True)

## Difference factors

### Investigation

In [ ]:
# Plot difference factors as a function of wavelength and passive layer thickness for a choice of excitonic layer thickness, theta, and number of periods

sel = diff_1.interactive(loc="left")

variable = pnw.Select(options=list(diff_1.data_vars))

xs_1D = {
    "excitonic_layer_thickness": pnw.DiscreteSlider,
    "theta": pnw.DiscreteSlider,
    "num_periods": pnw.DiscreteSlider,
    "passive_layer_thickness": pnw.DiscreteSlider,
}

sel_1 = sel.sel(**xs_1D).squeeze()

# Get a handle on the widgets
widgets = {k: w for k, w in zip(xs_1D, sel_1.widgets())}

xs_2D = {k: w for k, w in widgets.items() if k not in {"passive_layer_thickness"}}
sel_2 = sel.sel(**xs_2D).squeeze()

(
    sel_1[variable].hvplot(x="wavelength", ylim=(-1, 1))
    * hv.HLine(0).opts(opts.HLine(line_dash="dotted"))
    + sel_2[variable].hvplot(x="wavelength", y="passive_layer_thickness", clim=(-1, 1), cmap="RdBu_r")
    # * hv.DynamicMap(lambda y: hv.HLine(y), streams={"y": widgets["passive_layer_thickness"].param.value})
).cols(1)

In [ ]:
# Plot difference factors as a function of wavelength and *period* for a choice of excitonic layer thickness, theta, and number of periods

sel = restacked_diff_1.interactive(loc="left")

variable = pnw.Select(options=list(restacked_diff_1.data_vars))

xs_1D = {
    "excitonic_layer_thickness": pnw.DiscreteSlider,
    "theta": pnw.DiscreteSlider,
    "num_periods": pnw.DiscreteSlider,
    "period": pnw.DiscreteSlider,
}

sel_1 = sel.sel(**xs_1D).squeeze()

# Get a handle on the widgets
widgets = {k: w for k, w in zip(xs_1D, sel_1.widgets())}

xs_2D = {k: w for k, w in widgets.items() if k not in {"period"}}
sel_2 = sel.sel(**xs_2D).squeeze()

(
    sel_1[variable].hvplot(x="wavelength", ylim=(-1, 1))
    * hv.HLine(0).opts(opts.HLine(line_dash="dotted"))
    + sel_2[variable].hvplot(x="wavelength", y="period", clim=(-1, 1), cmap="RdBu_r")
    # * hv.DynamicMap(lambda y: hv.HLine(y), streams={"y": widgets["passive_layer_thickness"].param.value})
).cols(1)

In [ ]:
# Plot difference factors as a function of wavelength and *number of periods* for a choice of excitonic layer thickness, theta, and period

sel = restacked_diff_1.interactive(loc="left")

variable = pnw.Select(options=list(restacked_diff_1.data_vars))

xs_1D = {
    "excitonic_layer_thickness": pnw.DiscreteSlider,
    "theta": pnw.DiscreteSlider,
    "num_periods": pnw.DiscreteSlider,
    "period": pnw.DiscreteSlider,
}

sel_1 = sel.sel(**xs_1D).squeeze()

# Get a handle on the widgets
widgets = {k: w for k, w in zip(xs_1D, sel_1.widgets())}

xs_2D = {k: w for k, w in widgets.items() if k not in {"num_periods"}}
sel_2 = sel.sel(**xs_2D).squeeze()

(
    sel_1[variable].hvplot(x="wavelength", ylim=(-1, 1))
    * hv.HLine(0).opts(opts.HLine(line_dash="dotted"))
    + sel_2[variable].hvplot(kind="quadmesh", x="wavelength", y="num_periods", clim=(-1, 1), cmap="RdBu_r")
    # * hv.DynamicMap(lambda y: hv.HLine(y), streams={"y": widgets["passive_layer_thickness"].param.value})
).cols(1)

In [ ]:
with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    crs_1_like_ds = (
        crs_1#.drop("passive_layer_thickness")
        .expand_dims(passive_layer_thickness=ds.passive_layer_thickness)
        .sel(total_excitonic_thickness=ds.total_excitonic_thickness)
    )

In [ ]:
crs_1_like_ds = assign_derived_attrs(
    crs_1_like_ds,
    absorption=False,
    unpolarised=False,
    period=True,
    total_excitonic_thickness=False,
    total_passive_thickness=False,
    total_thickness=False,
    N_tot=False,
)

In [ ]:
with dask.config.set(**{"array.slicing.split_large_chunks": False}):
    restacked_crs_1 = (
        crs_1_like_ds.stack(
            multiperiod=["passive_layer_thickness", "excitonic_layer_thickness"]
        )
        .set_index(multiperiod=["period", "excitonic_layer_thickness"])
        .unstack()
    )

In [ ]:
# Workaround to the bug below: pull the widget values directly
widget_vals = {k: v.value for k, v in widgets.items()}

curve_1 = (
    restacked_ds.sel(**widget_vals).squeeze().A.hvplot(x="wavelength", ylim=(0, 1), label="A", group_label="LOPC")
)

curve_11 = (
    restacked_ds.sel(**widget_vals).squeeze().R.hvplot(x="wavelength", ylim=(0, 1), label="R", group_label="LOPC")
)

curve_2 = (
    restacked_crs_1.sel(**widget_vals).squeeze().A.hvplot(x="wavelength", ylim=(0, 1), label="A", group_label="CRS")
).opts(line_dash="dashed")

curve_21 = (
    restacked_crs_1.sel(**widget_vals).squeeze().R.hvplot(x="wavelength", ylim=(0, 1), label="R", group_label="CRS")
).opts(line_dash="dashed")

curve_3 = (
    (restacked_ds.sel(**widget_vals) / restacked_crs_1.sel(**widget_vals))
    .squeeze()
    .A.hvplot(x="wavelength", ylim=(0.5, 1.5))
)

((curve_1 * curve_11 *curve_2 * curve_21) + curve_3 * hv.HLine(1).opts(line_dash="dashed")).cols(1).opts(
    shared_axes=False
)

### Optimisation

Can just reuse the dmaps from the RTA section, because difference factors integrate perfectly well.

In [ ]:
# Use this to record interesting key sets
angle_dmap.current_key

### Transmittance difference factor

#### Max (near)

##### Normal incidence

At normal incidence and on resonance, optimal structure for maximising absorptance difference factor is similar to for absorptance enhancement factor. As the window width increases, the optimal total excitonic layer thickness also increases. The relevance of the slow light enhancement decreases relative to that of path length enhancement (see 0-5-0-0).

In [ ]:
dmap[('T', 'restacked_diff_1', 'max', 0, 0, 0, 'cividis', 10, 450)]

In [ ]:
dmap[('T', 'restacked_diff_1', 'max', 0, 2, 0, 'cividis', 10, 450)]

In [ ]:
dmap[('T', 'restacked_diff_1', 'max', 0, 5, 0, 'cividis', 10, 450)]

##### 30 degrees

Higher angle = higher period, otherwise structure is similar.

In [ ]:
angle_dmap[('T', 'restacked_diff_1', 'max', 0, 0, 30, 'cividis', 10, 450, 0)]

In [ ]:
angle_dmap[('T', 'restacked_diff_1', 'max', 0, 2, 30, 'cividis', 10, 450, 0)]

In [ ]:
angle_dmap[('T', 'restacked_diff_1', 'max', 0, 5, 30, 'cividis', 10, 450, 0)]

##### Integrated 0-75 degrees

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

angle_dmap[('T', 'restacked_diff_1', 'max', 0, 0, 0, 'cividis', 10, 450, 75)]

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

angle_dmap[('T', 'restacked_diff_1', 'max', 0, 2, 0, 'cividis', 10, 450, 75)]

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

angle_dmap[('T', 'restacked_diff_1', 'max', 0, 5, 0, 'cividis', 10, 450, 75)]

#### Max (far)

##### Normal incidence

In [ ]:
dmap[('T', 'restacked_diff_1', 'max', -5, 0, 0, 'cividis', 10, 450)]

In [ ]:
dmap[('T', 'restacked_diff_1', 'max', -5, 2, 0, 'cividis', 10, 450)]

In [ ]:
dmap[('T', 'restacked_diff_1', 'max', 5, 0, 0, 'cividis', 10, 450)]

In [ ]:
dmap[('T', 'restacked_diff_1', 'max', 5, 2, 0, 'cividis', 10, 450)]

##### 30 degrees

In [ ]:
angle_dmap[('T', 'restacked_diff_1', 'max', -5, 0, 30, 'cividis', 10, 450, 0)]

In [ ]:
angle_dmap[('T', 'restacked_diff_1', 'max', -5, 2, 30, 'cividis', 10, 450, 0)]

In [ ]:
angle_dmap[('T', 'restacked_diff_1', 'max', 5, 0, 30, 'cividis', 10, 450, 0)]

In [ ]:
angle_dmap[('T', 'restacked_diff_1', 'max', 5, 2, 30, 'cividis', 10, 450, 0)]

##### Integrated 0-75 degrees

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

angle_dmap[('T', 'restacked_diff_1', 'max', -5, 0, 0, 'cividis', 10, 450, 75)]

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

angle_dmap[('T', 'restacked_diff_1', 'max', -5, 2, 0, 'cividis', 10, 450, 75)]

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

angle_dmap[('T', 'restacked_diff_1', 'max', 5, 0, 0, 'cividis', 10, 450, 75)]

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

angle_dmap[('T', 'restacked_diff_1', 'max', 5, 2, 0, 'cividis', 10, 450, 75)]

#### Min (near)

##### Normal incidence

In [ ]:
dmap[('T', 'restacked_diff_1', 'min', 0, 0, 0, 'cividis', 10, 450)]

In [ ]:
dmap[('T', 'restacked_diff_1', 'min', 0, 2, 0, 'cividis', 10, 450)]

In [ ]:
dmap[('T', 'restacked_diff_1', 'min', 0, 5, 0, 'cividis', 10, 450)]

##### 30 degrees

In [ ]:
angle_dmap[('T', 'restacked_diff_1', 'min', 0, 0, 30, 'cividis', 10, 450, 0)]

In [ ]:
angle_dmap[('T', 'restacked_diff_1', 'min', 0, 2, 30, 'cividis', 10, 450, 0)]

In [ ]:
angle_dmap[('T', 'restacked_diff_1', 'min', 0, 5, 30, 'cividis', 10, 450, 0)]

##### Integrated 0-75 degrees

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

angle_dmap[('T', 'restacked_diff_1', 'min', 0, 0, 0, 'cividis', 10, 450, 75)]

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

angle_dmap[('T', 'restacked_diff_1', 'min', 0, 2, 0, 'cividis', 10, 450, 75)]

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

angle_dmap[('T', 'restacked_diff_1', 'min', 0, 5, 0, 'cividis', 10, 450, 75)]

#### Min (far)

##### Normal incidence

In [ ]:
dmap[('T', 'restacked_diff_1', 'min', -5, 0, 0, 'cividis', 10, 450)]

In [ ]:
dmap[('T', 'restacked_diff_1', 'min', -5, 2, 0, 'cividis', 10, 450)]

In [ ]:
dmap[('T', 'restacked_diff_1', 'min', 5, 0, 0, 'cividis', 10, 450)]

In [ ]:
dmap[('T', 'restacked_diff_1', 'min', 5, 2, 0, 'cividis', 10, 450)]

##### 30 degrees

In [ ]:
angle_dmap[('T', 'restacked_diff_1', 'min', -5, 0, 30, 'cividis', 10, 450, 0)]

In [ ]:
angle_dmap[('T', 'restacked_diff_1', 'min', -5, 2, 30, 'cividis', 10, 450, 0)]

In [ ]:
angle_dmap[('T', 'restacked_diff_1', 'min', 5, 0, 30, 'cividis', 10, 450, 0)]

In [ ]:
angle_dmap[('T', 'restacked_diff_1', 'min', 5, 2, 30, 'cividis', 10, 450, 0)]

##### Integrated 0-75 degrees

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

angle_dmap[('T', 'restacked_diff_1', 'min', -5, 0, 0, 'cividis', 10, 450, 75)]

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

angle_dmap[('T', 'restacked_diff_1', 'min', -5, 2, 0, 'cividis', 10, 450, 75)]

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

angle_dmap[('T', 'restacked_diff_1', 'min', 5, 0, 0, 'cividis', 10, 450, 75)]

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

angle_dmap[('T', 'restacked_diff_1', 'min', 5, 2, 0, 'cividis', 10, 450, 75)]

### Absorptance difference factor

#### Max (near)

##### Normal incidence

At normal incidence and on resonance, optimal structure for maximising absorptance difference factor is similar to for absorptance enhancement factor. As the window width increases, the optimal total excitonic layer thickness also increases. The relevance of the slow light enhancement decreases relative to that of path length enhancement (see 0-5-0-0).

In [ ]:
dmap[('A', 'restacked_diff_1', 'max', 0, 0, 0, 'inferno', 10, 450)]

In [ ]:
dmap[('A', 'restacked_diff_1', 'max', 0, 2, 0, 'inferno', 10, 450)]

In [ ]:
dmap[('A', 'restacked_diff_1', 'max', 0, 5, 0, 'inferno', 10, 450)]

##### 30 degrees

Higher angle = higher period, otherwise structure is similar.

In [ ]:
angle_dmap[('A', 'restacked_diff_1', 'max', 0, 0, 30, 'inferno', 10, 450, 0)]

In [ ]:
angle_dmap[('A', 'restacked_diff_1', 'max', 0, 2, 30, 'inferno', 10, 450, 0)]

In [ ]:
angle_dmap[('A', 'restacked_diff_1', 'max', 0, 5, 30, 'inferno', 10, 450, 0)]

##### Integrated 0-75 degrees

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

angle_dmap[('A', 'restacked_diff_1', 'max', 0, 0, 0, 'inferno', 10, 450, 75)]

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

angle_dmap[('A', 'restacked_diff_1', 'max', 0, 2, 0, 'inferno', 10, 450, 75)]

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

angle_dmap[('A', 'restacked_diff_1', 'max', 0, 5, 0, 'inferno', 10, 450, 75)]

#### Max (far)

##### Normal incidence

In [ ]:
dmap[('A', 'restacked_diff_1', 'max', -5, 0, 0, 'inferno', 10, 450)]

In [ ]:
dmap[('A', 'restacked_diff_1', 'max', -5, 2, 0, 'inferno', 10, 450)]

In [ ]:
dmap[('A', 'restacked_diff_1', 'max', 5, 0, 0, 'inferno', 10, 450)]

In [ ]:
dmap[('A', 'restacked_diff_1', 'max', 5, 2, 0, 'inferno', 10, 450)]

##### 30 degrees

In [ ]:
angle_dmap[('A', 'restacked_diff_1', 'max', -5, 0, 30, 'inferno', 10, 450, 0)]

In [ ]:
angle_dmap[('A', 'restacked_diff_1', 'max', -5, 2, 30, 'inferno', 10, 450, 0)]

In [ ]:
angle_dmap[('A', 'restacked_diff_1', 'max', 5, 0, 30, 'inferno', 10, 450, 0)]

In [ ]:
angle_dmap[('A', 'restacked_diff_1', 'max', 5, 2, 30, 'inferno', 10, 450, 0)]

##### Integrated 0-75 degrees

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

angle_dmap[('A', 'restacked_diff_1', 'max', -5, 0, 0, 'inferno', 10, 450, 75)]

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

angle_dmap[('A', 'restacked_diff_1', 'max', -5, 2, 0, 'inferno', 10, 450, 75)]

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

angle_dmap[('A', 'restacked_diff_1', 'max', 5, 0, 0, 'inferno', 10, 450, 75)]

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

angle_dmap[('A', 'restacked_diff_1', 'max', 5, 2, 0, 'inferno', 10, 450, 75)]

#### Min (near)

##### Normal incidence

In [ ]:
dmap[('A', 'restacked_diff_1', 'min', 0, 0, 0, 'inferno', 10, 450)]

In [ ]:
dmap[('A', 'restacked_diff_1', 'min', 0, 2, 0, 'inferno', 10, 450)]

In [ ]:
dmap[('A', 'restacked_diff_1', 'min', 0, 5, 0, 'inferno', 10, 450)]

##### 30 degrees

In [ ]:
angle_dmap[('A', 'restacked_diff_1', 'min', 0, 0, 30, 'inferno', 10, 450, 0)]

In [ ]:
angle_dmap[('A', 'restacked_diff_1', 'min', 0, 2, 30, 'inferno', 10, 450, 0)]

In [ ]:
angle_dmap[('A', 'restacked_diff_1', 'min', 0, 5, 30, 'inferno', 10, 450, 0)]

##### Integrated 0-75 degrees

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

angle_dmap[('A', 'restacked_diff_1', 'min', 0, 0, 0, 'inferno', 10, 450, 75)]

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

angle_dmap[('A', 'restacked_diff_1', 'min', 0, 2, 0, 'inferno', 10, 450, 75)]

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

angle_dmap[('A', 'restacked_diff_1', 'min', 0, 5, 0, 'inferno', 10, 450, 75)]

#### Min (far)

##### Normal incidence

In [ ]:
dmap[('A', 'restacked_diff_1', 'min', -5, 0, 0, 'inferno', 10, 450)]

In [ ]:
dmap[('A', 'restacked_diff_1', 'min', -5, 2, 0, 'inferno', 10, 450)]

In [ ]:
dmap[('A', 'restacked_diff_1', 'min', 5, 0, 0, 'inferno', 10, 450)]

In [ ]:
dmap[('A', 'restacked_diff_1', 'min', 5, 2, 0, 'inferno', 10, 450)]

##### 30 degrees

In [ ]:
angle_dmap[('A', 'restacked_diff_1', 'min', -5, 0, 30, 'inferno', 10, 450, 0)]

In [ ]:
angle_dmap[('A', 'restacked_diff_1', 'min', -5, 2, 30, 'inferno', 10, 450, 0)]

In [ ]:
angle_dmap[('A', 'restacked_diff_1', 'min', 5, 0, 30, 'inferno', 10, 450, 0)]

In [ ]:
angle_dmap[('A', 'restacked_diff_1', 'min', 5, 2, 30, 'inferno', 10, 450, 0)]

##### Integrated 0-75 degrees

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

angle_dmap[('A', 'restacked_diff_1', 'min', -5, 0, 0, 'inferno', 10, 450, 75)]

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

angle_dmap[('A', 'restacked_diff_1', 'min', -5, 2, 0, 'inferno', 10, 450, 75)]

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

angle_dmap[('A', 'restacked_diff_1', 'min', 5, 0, 0, 'inferno', 10, 450, 75)]

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

angle_dmap[('A', 'restacked_diff_1', 'min', 5, 2, 0, 'inferno', 10, 450, 75)]

## Total absorptance

Gonna try to define a dataset where the integration is done automatically.

Nope, can't be done. Instead what's needed is to redefine the integral.

In [ ]:
#

sel = ds.interactive(loc="left")

variable = pnw.Select(options=list(ds.data_vars))

xs_1D = {
    "excitonic_layer_thickness": pnw.DiscreteSlider,
    "theta": pnw.DiscreteSlider,
    "num_periods": pnw.DiscreteSlider,
    "passive_layer_thickness": pnw.DiscreteSlider,
}

sel_1 = sel.sel(**xs_1D).squeeze()[variable]

# Get a handle on the widgets
widgets = {k: w for k, w in zip(xs_1D, sel_1.widgets())}

# xs_2D = {k: w for k, w in widgets.items() if k not in {"passive_layer_thickness"}}
# sel_2 = sel.sel(**xs_2D).squeeze()

(
    sel_1.hvplot(x="wavelength")
    # * hv.HLine(0).opts(opts.HLine(line_dash="dotted"))
    # + sel_2[variable].hvplot(x="wavelength", y="passive_layer_thickness", clim=(-1, 1), cmap="RdBu_r")
    # * hv.DynamicMap(lambda y: hv.HLine(y), streams={"y": widgets["passive_layer_thickness"].param.value})
)#.cols(1)

In [ ]:
ds.rolling(wavelength=2).construct("window").sel(window=1).sel(theta=0, passive_layer_thickness=180, num_periods=6, excitonic_layer_thickness=20, method="nearest").Rs.load()

In [ ]:
ds.sel(theta=0).integrate('theta')

In [ ]:
xs_1D = {
    "excitonic_layer_thickness": pnw.DiscreteSlider,
    "theta": pnw.DiscreteSlider,
    "num_periods": pnw.DiscreteSlider,
    "passive_layer_thickness": pnw.DiscreteSlider,
}

variable = pnw.Select(options=list(ds.data_vars))
# window_size = pnw.DiscreteSlider(options=list(range(1, 257)))

ds.interactive(loc="left")[variable].sel(**xs_1D).squeeze().rolling({'wavelength': 10}, center=True).construct("window").integrate('window').hvplot.line()

### Investigation

Plot total absorption enhancement factor under an arbitrary spectrum.

##### Plot total absorptance enhancement factor as a function of period.

In [ ]:
ymin1 = float(ds_flat_spectrum.A.drop_sel(excitonic_layer_thickness=0).sel(theta=0).min())
ymax1 = float(ds_flat_spectrum.A.drop_sel(excitonic_layer_thickness=0).sel(theta=0).max())

In [ ]:
ymin2 = float(norm_flat_spectrum.A.drop_sel(excitonic_layer_thickness=0).sel(theta=0).min())
ymax2 = float(norm_flat_spectrum.A.drop_sel(excitonic_layer_thickness=0).sel(theta=0).max())

In [ ]:
norm_flat_spectrum.drop_sel(excitonic_layer_thickness=0).squeeze().A.hvplot(x="passive_layer_thickness", y="excitonic_layer_thickness", clim=(0.8, 1.2), cmap='RdBu_r')

In [ ]:
(
    restacked_norm_flat_spectrum.drop_sel(excitonic_layer_thickness=0)
    .squeeze()
    .A.hvplot(x="period", y="excitonic_layer_thickness", clim=(0.8, 1.2), cmap="RdBu_r")
    .opts(clabel="γ_tot (flat spectrum)", shared_axes=False)
    + restacked_ds_flat_spectrum.drop_sel(excitonic_layer_thickness=0)
    .squeeze()
    .A.hvplot(x="period", y="excitonic_layer_thickness", clim=(0, 1), cmap="inferno")
    .opts(clabel="A_tot", shared_axes=False)
).cols(1)

##### Define some convenient datasets for plotting.

In [ ]:
crs_1_fs_like_ds = crs_1_flat_spectrum.sel(
    total_excitonic_thickness=ds.total_excitonic_thickness
).assign(passive_layer_thickness = ds_flat_spectrum.passive_layer_thickness)

In [ ]:
restacked_crs_1_fs_like_ds = (
    assign_derived_attrs(
        crs_1_fs_like_ds,
        absorption=False,
        unpolarised=False,
        period=True,
        total_excitonic_thickness=False,
        total_passive_thickness=False,
        total_thickness=False,
        N_tot=False,
        per_oscillator=False,
    )
    .stack(multiperiod=["passive_layer_thickness", "excitonic_layer_thickness"])
    .set_index(multiperiod=["period", "excitonic_layer_thickness"])
    .unstack()
)

In [ ]:
diff_1_flat_spectrum = ds_flat_spectrum - crs_1_fs_like_ds

In [ ]:
restacked_diff_1_flat_spectrum = (
    diff_1_flat_spectrum.stack(
        multiperiod=["passive_layer_thickness", "excitonic_layer_thickness"]
    )
    .set_index(multiperiod=["period", "excitonic_layer_thickness"])
    .unstack()
)

##### Plot the total absorptance and absolute difference in total absorptance

In [ ]:
restacked_ds_flat_spectrum[unpolarised_RTA].squeeze().hvplot(
    x="period", ylim=(0, 1)
)

In [ ]:
crs_1[unpolarised_RTA].squeeze().hvplot(x="wavelength", ylim=(0,1))*lorentz_vlines(0, scale=1e-9).opts(line_dash='dotted', color=yellow)

In [ ]:
(
    crs_1_flat_spectrum[unpolarised_RTA]
    .squeeze()
    .hvplot(
        x="total_excitonic_thickness",
        xlim=(0, 3500),
        ylim=(0, 1),
    )
    * hv.VLine(120, label="6 periods of 20 nm").opts(line_dash="dotted")
    * hv.VLine(3000, label="50 periods of 60 nm").opts(line_dash="dotted")
)

In [ ]:
(restacked_diff_1_flat_spectrum)[unpolarised_RTA].squeeze().hvplot(
    x="period",
)*hv.HLine(0).opts(line_dash='dotted')

##### Plot total absorption enhancement factor as a function of excitonic layer thickness and number of layers.

In [ ]:
norm_flat_spectrum.drop_sel(excitonic_layer_thickness=0).squeeze().A.hvplot(kind="quadmesh", x="excitonic_layer_thickness", y="num_periods", clim=(0.8, 1.2), cmap='RdBu_r')

In [ ]:
restacked_norm_flat_spectrum.drop_sel(excitonic_layer_thickness=0).squeeze().A.hvplot(kind="quadmesh", x="excitonic_layer_thickness", y="num_periods", clim=(0.8, 1.2), cmap='RdBu_r')

##### Plot total absorption enhancement factor as a function of passive layer thickness/period and number of layers.

In [ ]:
norm_flat_spectrum.drop_sel(excitonic_layer_thickness=0).squeeze().A.hvplot(kind="quadmesh", x="passive_layer_thickness", y="num_periods", clim=(0.8, 1.2), cmap='RdBu_r')

In [ ]:
restacked_norm_flat_spectrum.drop_sel(excitonic_layer_thickness=0).squeeze().A.hvplot(kind="quadmesh", x="period", y="num_periods", clim=(0.8, 1.2), cmap='RdBu_r')

##### A prototype of a dashboard

In [ ]:
all_widgets = {
    "excitonic_layer_thickness": pnw.DiscreteSlider,
    "theta": pnw.DiscreteSlider,
    "num_periods": pnw.DiscreteSlider,
    "passive_layer_thickness": pnw.DiscreteSlider,
}

x = "num_periods"

xs = {k: v for k, v in all_widgets.items() if k is not x}

RTA_selector = pnw.Select(options=unpolarised_RTA, name='RTA')

sel_1 = ds_flat_spectrum.interactive[RTA_selector].sel(**xs).squeeze()

sel_2 = ref_flat_spectrum

In [ ]:
sel_1.hvplot(x="num_periods", ylim=(0,1))

### Optimisation

##### Find maximum gamma_tot

In [ ]:
sel = {'excitonic_layer_thickness': slice(None,None), 'period': slice(None,None), 'num_periods': slice(None, None), 'theta': slice(0,0)}
da = restacked_norm_flat_spectrum['A'].drop_sel(excitonic_layer_thickness=0).sel(sel)
optimum = da.where(da==da.max(), drop=True)
print(optimum.values)
print(optimum.coords)
P = optimum.period.values[0]
t = optimum.excitonic_layer_thickness.values[0]
N = optimum.num_periods.values[0]

In [ ]:
(
    restacked_ds["R"]
    .sel(period=P, excitonic_layer_thickness=t, num_periods=N, theta=0)
    .squeeze()
    .hvplot(x="wavelength", label="Reflectance (LOPC)")
    * restacked_ds["A"]
    .sel(period=P, excitonic_layer_thickness=t, num_periods=N, theta=0)
    .squeeze()
    .hvplot(x="wavelength", label="Absorptance (LOPC)")
    * crs_1["R"]
    .sel(total_excitonic_thickness=t * N, theta=0)
    .squeeze()
    .hvplot(x="wavelength", label="Reflectance (CRS)")
    .opts(opts.Curve(color=blue, line_dash="dashed"))
    * crs_1["A"]
    .sel(total_excitonic_thickness=t * N, theta=0)
    .squeeze()
    .hvplot(x="wavelength", label="Absorptance (CRS)")
    .opts(opts.Curve(color=red, line_dash="dashed"))
    * resonance_line
).opts(
    ylim=(0, 1),
    ylabel="Intensity",
    title=f"period: {P} nm\nexcitonic_layer_thickness: {t} nm\nnum_periods: {N}",
)

##### Find maximum Delta_tot

In [ ]:
sel = {'excitonic_layer_thickness': slice(None,None), 'period': slice(None,None), 'num_periods': slice(10,10), 'theta': slice(0,0)}
da = restacked_diff_1_flat_spectrum['A'].sel(sel)
optimum = da.where(da==da.max(), drop=True)
print(optimum.values)
print(optimum.coords)
P = optimum.period.values[0]
t = optimum.excitonic_layer_thickness.values[0]
N = optimum.num_periods.values[0]

In [ ]:
(
    restacked_ds["R"]
    .sel(period=P, excitonic_layer_thickness=t, num_periods=N, theta=0)
    .squeeze()
    .hvplot(x="wavelength", label="Reflectance (LOPC)")
    * restacked_ds["A"]
    .sel(period=P, excitonic_layer_thickness=t, num_periods=N, theta=0)
    .squeeze()
    .hvplot(x="wavelength", label="Absorptance (LOPC)")
    * crs_1["R"]
    .sel(total_excitonic_thickness=t * N, theta=0)
    .squeeze()
    .hvplot(x="wavelength", label="Reflectance (CRS)")
    .opts(opts.Curve(color=blue, line_dash='dashed'))
    * crs_1["A"]
    .sel(total_excitonic_thickness=t * N, theta=0)
    .squeeze()
    .hvplot(x="wavelength", label="Absorptance (CRS)")
    .opts(opts.Curve(color=red, line_dash='dashed'))
    *resonance_line
).opts(
    ylim=(0, 1),
    ylabel="Intensity",
    title=f"period: {P} nm\nexcitonic_layer_thickness: {t} nm\nnum_periods: {N}",
)

##### Find maximum... Delta_absorption_per_oscillator_tot?

In [ ]:
sel = {'excitonic_layer_thickness': slice(None,None), 'period': slice(None,None), 'num_periods': slice(None, None), 'theta': slice(0,0)}
da = restacked_diff_1_flat_spectrum['A_per_oscillator'].drop_sel(excitonic_layer_thickness=0).sel(sel)
optimum = da.where(da==da.max(), drop=True)
print(optimum.values)
print(optimum.coords)
P = optimum.period.values[0]
t = optimum.excitonic_layer_thickness.values[0]
N = optimum.num_periods.values[0]

In [ ]:
(
    restacked_ds["R"]
    .sel(period=P, excitonic_layer_thickness=t, num_periods=N, theta=0)
    .squeeze()
    .hvplot(x="wavelength", label="Reflectance (LOPC)")
    * restacked_ds["A"]
    .sel(period=P, excitonic_layer_thickness=t, num_periods=N, theta=0)
    .squeeze()
    .hvplot(x="wavelength", label="Absorptance (LOPC)")
    * crs_1["R"]
    .sel(total_excitonic_thickness=t * N, theta=0)
    .squeeze()
    .hvplot(x="wavelength", label="Reflectance (CRS)")
    .opts(opts.Curve(color=blue, line_dash='dashed'))
    * crs_1["A"]
    .sel(total_excitonic_thickness=t * N, theta=0)
    .squeeze()
    .hvplot(x="wavelength", label="Absorptance (CRS)")
    .opts(opts.Curve(color=red, line_dash='dashed'))
    *resonance_line
).opts(ylim=(0, 1), ylabel="Intensity")

## High angles

At high angles the reflectance of the slab gets high (as is well-known) while that of the LOPC stays low (for certain thicknesses).
As a result, absorptance can be much higher.

In [ ]:
(crs_1['R'].squeeze().hvplot(x="wavelength", y="theta", clim=(0,1), cmap="viridis")#*resonance_line
 + crs_1['T'].squeeze().hvplot(x="wavelength", y="theta", clim=(0,1), cmap="cividis")#*resonance_line
 + crs_1['A'].squeeze().hvplot(x="wavelength", y="theta", clim=(0,1), cmap="inferno")#*resonance_line
).cols(1)

In [ ]:
plot_R?

In [ ]:
plot_R(variable="R", dataset=restacked_ds.sel(period=300, excitonic_layer_thickness=30, num_periods=50).squeeze(), x="wavelength", y="theta")

In [ ]:
plot_R(variable="Rs", dataset=restacked_ds.sel(period=300, excitonic_layer_thickness=30, num_periods=50).squeeze(), x="wavelength", y="theta")

In [ ]:
plot_R(variable="Rp", dataset=restacked_ds.sel(period=200, excitonic_layer_thickness=30, num_periods=50).squeeze(), x="wavelength", y="theta")

## Optimal structure dataset/table

In [ ]:
def cross_section(
    dataset,
    variable=None,
    lorentz_line=0,
    window_radius=0,
    theta=0,
    integrate_angle=None,
    normalisation=1
):
    da = dataset[variable] if variable is not None else dataset
    integration_dims = []

    if not integrate_angle:
        da = da.sel(theta=theta, method="nearest")
    else:  # integrate_angle must be a float, so that (theta, integrate_angle) is a slice syntax
        da = da.sel(theta=slice(theta, integrate_angle))
        # da = integrate_da(da, "theta", weighting=1, normalisation=1)
        integration_dims.append("theta")

    da = select_lorentz_line(da, lorentz_line=lorentz_line, window_radius=window_radius)

    if window_radius != 0:
        # da = integrate_da(da, "wavelength", weighting=1, normalisation=1)
        integration_dims.append("wavelength")
    
    if integration_dims:  # if the list isn't empty
        da = integrate_da(da, integration_dims, weighting=1, normalisation=normalisation)
    
    return da

In [ ]:
def select_period(da, period_start=None, period_stop=None):
    period_start = float(da.period.min().values) if period_start is None else period_start
    period_stop = float(da.period.max().values) if period_stop is None else period_stop
    
    if period_start < period_stop:
        da = da.sel(period=slice(period_start, period_stop))
    else:  # otherwise no data is selected and everything breaks
        da = da.sel(period=slice(period_start, None))
        
    return da

In [ ]:
def opt_over_dim(
    dataset,
    ref=None,
    common_dim=None,
    variable=None,
    optimise="max",
    lorentz_line=0,
    window_radius=0,
    theta=0,
    period_start=None,
    period_stop=None,
    integrate_angle=None,
    dim=...,
    load=False,
):
    if ref is not None:
        normalisation = None
    else:
        normalisation = 1
        
    da = cross_section(
        dataset,
        variable=variable,
        lorentz_line=lorentz_line,
        window_radius=window_radius,
        theta=theta,
        integrate_angle=integrate_angle,
        normalisation=normalisation
    )
    
    if ref is not None:
        match ref:
            case "crs_1":
                ref = crs_1
                common_dim = "total_excitonic_thickness"
                norm_method = "groupby"
            case "frs_1":
                ref = frs_1
                common_dim = "total_thickness"
                norm_method = "vectorized_indexing"
            case _:
                if common_dim is None:
                    raise ValueError("ref was given but not common_dim")
                norm_method = "vectorized_indexing"
        ra = cross_section(
            ref,
            variable=variable,
            lorentz_line=lorentz_line,
            window_radius=window_radius,
            theta=theta,
            integrate_angle=integrate_angle,
            normalisation=None
        )
        da = enhancement_factor(ds=da, ref=ra, common_dim=common_dim, method=norm_method)
        da = restack_plt_to_period(da)

    da = select_period(da, period_start=period_start, period_stop=period_stop)
    
    if load:
        da = da.load()
        
    if optimise == "max":
        optimised_da = da.max(dim=dim)
    elif optimise == "min":
        optimised_da = da.min(dim=dim)
    else:
        raise ValueError(f"optimise must be 'max' or 'min, not {optimise}")
    
    return optimised_da

In [ ]:
(opt_over_dim(
    restacked_ds,
    dim="period",
    window_radius=1,
    load=True
).squeeze(drop=True)
# .drop_vars(
#     ["frequency",
#      "theta",
# #      "passive_RI",
# #      "incident_medium_RI",
# #      "exit_medium_RI",
# #      "N",
# #      "permittivity",
# #      "lorentz_resonance_wavelength",
# #      "lorentz_linewidth",
# #      "remove_last_layer",
#      "wavelength"
#     ]
# )
)

In [ ]:
def optimal_indices(
    dataset,
    variable=None,
    optimise="max",
    lorentz_line=0,
    window_radius=0,
    theta=0,
    period_start=None,
    period_stop=None,
    integrate_angle=None,
    dim=...,
    load=True,
):
    da = cross_section(
        dataset,
        variable=variable,
        lorentz_line=lorentz_line,
        window_radius=window_radius,
        theta=theta,
        integrate_angle=integrate_angle
    )

    da = select_period(da, period_start=period_start, period_stop=period_stop)
    
    if load:
        da = da.load()
    
    if optimise == "max":
        optimised_da = da.argmax(dim=dim)
    elif optimise == "min":
        optimised_da = da.argmin(dim=dim)
    else:
        raise ValueError(f"optimise must be 'max' or 'min, not {optimise}")
    
    return optimised_da

In [ ]:
def optimal_parameters(
    dataset,
    ref=None,
    common_dim=None,
    variable=None,
    optimise="max",
    lorentz_line=0,
    window_radius=0,
    theta=0,
    period_start=None,
    period_stop=None,
    integrate_angle=None,
    dim=...,
    load=True,
):
    if ref is not None:
        normalisation = None
    else:
        normalisation = 1
        
    da = cross_section(
        dataset,
        variable=variable,
        lorentz_line=lorentz_line,
        window_radius=window_radius,
        theta=theta,
        integrate_angle=integrate_angle,
        normalisation=normalisation
    )
    
    if ref is not None:
        match ref:
            case "crs_1":
                ref = crs_1
                common_dim = "total_excitonic_thickness"
                norm_method = "groupby"
            case "frs_1":
                ref = frs_1
                common_dim = "total_thickness"
                norm_method = "vectorized_indexing"
            case _:
                if common_dim is None:
                    raise ValueError("ref was given but not common_dim")
                norm_method = "vectorized_indexing"
        ra = cross_section(
            ref,
            variable=variable,
            lorentz_line=lorentz_line,
            window_radius=window_radius,
            theta=theta,
            integrate_angle=integrate_angle,
            normalisation=None
        )
        da = enhancement_factor(ds=da, ref=ra, common_dim=common_dim, method=norm_method)
        da = restack_plt_to_period(da)

    da = select_period(da, period_start=period_start, period_stop=period_stop)
    
    if load:
        da = da.load()
    
    optimised_da = find_optimum_coords(da, dim=dim, optimise=optimise, load=False)
    
    return optimised_da

In [ ]:
def optimal_parameters_to_dataframe(
    optimal_params,
    params,
    cast=None,
    store_value=True,
    cast_value=None,
    index=None,
    columns=None,
    dtype=None,
    **additonal_params
):
    cast = (lambda x: x) if cast is None else cast
    to_df = additonal_params|{p: cast(optimal_params[p]) for p in params}
    if store_value:
        cast_value = (lambda x: x) if cast_value is None else cast_value
        to_df|= {"value": cast_value(optimal_params.values)}
    
    df = pd.DataFrame([to_df], index=index, columns=columns, dtype=dtype)
    
    return df

In [ ]:
%%time
foo = optimal_parameters(
    ds.drop_sel(excitonic_layer_thickness=0),
    "crs_1",
    variable="A",
    lorentz_line=5,
    window_radius=2,
    integrate_angle=75
)

In [ ]:
%%time
foo = optimal_parameters(
    restacked_ds.drop_sel(excitonic_layer_thickness=0),
    variable="R",
    lorentz_line=5,
    window_radius=2,
    integrate_angle=75
)

In [ ]:
foo.values

In [ ]:
%%time
df = optimal_parameters_to_dataframe(
    optimal_parameters(restacked_ds, variable="R"),
    params=["excitonic_layer_thickness", "num_periods", "period"],
    cast=int,
    store_value=True,
    cast_value=float,
    lline=0,
)

In [ ]:
df

In [ ]:
opt_params = []
dataset = restacked_ds.drop_sel(excitonic_layer_thickness=0)
period_start = int(dataset.period.min().values)
period_stop = int(dataset.period.max().values)

with warnings.catch_warnings():
    warnings.filterwarnings("ignore")
    for variable in tqdm(unpolarised_vars):
        for maxmin in ["max", "min"]:
            for lorentz_line in [-8, -5, -2, 0, 2, 5, 8]:
                for window_radius in [0, 2, 5]:
                    for theta_range in [(0, 0), (30, 0), (0, 45)]:
                        optimal_params = optimal_parameters(
                            dataset=dataset,
                            variable=variable,
                            optimise=maxmin,
                            lorentz_line=lorentz_line,
                            window_radius=window_radius,
                            theta=theta_range[0],
                            integrate_angle=theta_range[1],
                            period_start=period_start,
                            period_stop=period_stop,
                            dim=...,
                            load=True
                        )
                        opt_params.append(
                            optimal_parameters_to_dataframe(
                                optimal_params,
                                params=["excitonic_layer_thickness", "num_periods", "period"],
                                cast=int,
                                store_value=True,
                                cast_value=float,
                                variable=variable,
                                maxmin=maxmin,
                                lorentz_line=lorentz_line,
                                window_radius=window_radius,
                                theta_start=theta_range[0],
                                theta_stop=theta_range[1],
                                period_start=period_start,
                                period_stop=period_stop,
                            )
                        )

In [ ]:
df = pd.concat(opt_params, ignore_index=True)

df.to_csv(f"data/run_{run_number}/optimal_LOPC.csv", index=False, mode="w")
# df.to_csv(f"data/run_{run_number}/optimal_LOPC.csv", index=False, mode="a", header=False)

# pd.read_csv(f"data/run_{run_number}/optimal_LOPC.csv").equals(df)

In [ ]:
opt_params = []
dataset = restacked_diff_1.drop_sel(excitonic_layer_thickness=0)
period_start = int(dataset.period.min().values)
period_stop = int(dataset.period.max().values)

with warnings.catch_warnings():
    warnings.filterwarnings("ignore")
    for variable in tqdm(unpolarised_RTA):
        for maxmin in ["max", "min"]:
            for lorentz_line in [-8, -5, -2, 0, 2, 5, 8]:
                for window_radius in [0, 2, 5]:
                    for theta_range in [(0, 0), (30, 0), (0, 45)]:
                        optimal_params = optimal_parameters(
                            dataset=dataset,
                            variable=variable,
                            optimise=maxmin,
                            lorentz_line=lorentz_line,
                            window_radius=window_radius,
                            theta=theta_range[0],
                            integrate_angle=theta_range[1],
                            period_start=period_start,
                            period_stop=period_stop,
                            dim=...,
                            load=True
                        )
                        opt_params.append(
                            optimal_parameters_to_dataframe(
                                optimal_params,
                                params=["excitonic_layer_thickness", "num_periods", "period"],
                                cast=int,
                                store_value=True,
                                cast_value=float,
                                variable=variable,
                                maxmin=maxmin,
                                lorentz_line=lorentz_line,
                                window_radius=window_radius,
                                theta_start=theta_range[0],
                                theta_stop=theta_range[1],
                                period_start=period_start,
                                period_stop=period_stop,
                            )
                        )

In [ ]:
df = pd.concat(opt_params, ignore_index=True)

df.to_csv(f"data/run_{run_number}/optimal_diff_1.csv", index=False, mode="w")
# df.to_csv(f"data/run_{run_number}/optimal_diff_1.csv", index=False, mode="a", header=False)

# pd.read_csv(f"data/run_{run_number}/optimal_diff_1.csv").equals(df)

In [ ]:
opt_params = []
dataset = restacked_diff_2.drop_sel(excitonic_layer_thickness=0)
period_start = int(dataset.period.min().values)
period_stop = int(dataset.period.max().values)

with warnings.catch_warnings():
    warnings.filterwarnings("ignore")
    for variable in tqdm(unpolarised_RTA):
        for maxmin in ["max", "min"]:
            for lorentz_line in [-8, -5, -2, 0, 2, 5, 8]:
                for window_radius in [0, 2, 5]:
                    for theta_range in [(0, 0), (30, 0), (0, 45)]:
                        optimal_params = optimal_parameters(
                            dataset=dataset,
                            variable=variable,
                            optimise=maxmin,
                            lorentz_line=lorentz_line,
                            window_radius=window_radius,
                            theta=theta_range[0],
                            integrate_angle=theta_range[1],
                            period_start=period_start,
                            period_stop=period_stop,
                            dim=...,
                            load=True
                        )
                        opt_params.append(
                            optimal_parameters_to_dataframe(
                                optimal_params,
                                params=["excitonic_layer_thickness", "num_periods", "period"],
                                cast=int,
                                store_value=True,
                                cast_value=float,
                                variable=variable,
                                maxmin=maxmin,
                                lorentz_line=lorentz_line,
                                window_radius=window_radius,
                                theta_start=theta_range[0],
                                theta_stop=theta_range[1],
                                period_start=period_start,
                                period_stop=period_stop,
                            )
                        )

In [ ]:
df = pd.concat(opt_params, ignore_index=True)

df.to_csv(f"data/run_{run_number}/optimal_diff_2.csv", index=False, mode="w")
# df.to_csv(f"data/run_{run_number}/optimal_diff_2.csv", index=False, mode="a", header=False)

# pd.read_csv(f"data/run_{run_number}/optimal_diff_2.csv").equals(df)

In [ ]:
opt_params = []
dataset = ds.drop_sel(excitonic_layer_thickness=0)
period_start = int(dataset.period.min().values)
period_stop = int(dataset.period.max().values)
ref = "crs_1"

with warnings.catch_warnings():
    warnings.filterwarnings("ignore")
    for variable in tqdm(unpolarised_RTA):
        for maxmin in ["max", "min"]:
            for lorentz_line in [-8, -5, -2, 0, 2, 5, 8]:
                for window_radius in [0, 2, 5]:
                    for theta_range in [(0, 0), (30, 0), (0, 45)]:
                        optimal_params = optimal_parameters(
                            dataset=dataset,
                            ref=ref,
                            variable=variable,
                            optimise=maxmin,
                            lorentz_line=lorentz_line,
                            window_radius=window_radius,
                            theta=theta_range[0],
                            integrate_angle=theta_range[1],
                            period_start=period_start,
                            period_stop=period_stop,
                            dim=...,
                            load=True
                        )
                        opt_params.append(
                            optimal_parameters_to_dataframe(
                                optimal_params,
                                params=["excitonic_layer_thickness", "num_periods", "period"],
                                cast=int,
                                store_value=True,
                                cast_value=float,
                                variable=variable,
                                maxmin=maxmin,
                                lorentz_line=lorentz_line,
                                window_radius=window_radius,
                                theta_start=theta_range[0],
                                theta_stop=theta_range[1],
                                period_start=period_start,
                                period_stop=period_stop,
                            )
                        )

In [ ]:
df = pd.concat(opt_params, ignore_index=True)

df.to_csv(f"data/run_{run_number}/optimal_norm_1.csv", index=False, mode="w")
# df.to_csv(f"data/run_{run_number}/optimal_norm_1.csv", index=False, mode="a", header=False)

In [ ]:
opt_params = []
dataset = ds.drop_sel(excitonic_layer_thickness=0)
period_start = int(dataset.period.min().values)
period_stop = int(dataset.period.max().values)
ref = "frs_1"

with warnings.catch_warnings():
    warnings.filterwarnings("ignore")
    for variable in tqdm(unpolarised_RTA):
        for maxmin in ["max", "min"]:
            for lorentz_line in [-8, -5, -2, 0, 2, 5, 8]:
                for window_radius in [0, 2, 5]:
                    for theta_range in [(0, 0), (30, 0), (0, 45)]:
                        optimal_params = optimal_parameters(
                            dataset=dataset,
                            ref=ref,
                            variable=variable,
                            optimise=maxmin,
                            lorentz_line=lorentz_line,
                            window_radius=window_radius,
                            theta=theta_range[0],
                            integrate_angle=theta_range[1],
                            period_start=period_start,
                            period_stop=period_stop,
                            dim=...,
                            load=True
                        )
                        opt_params.append(
                            optimal_parameters_to_dataframe(
                                optimal_params,
                                params=["excitonic_layer_thickness", "num_periods", "period"],
                                cast=int,
                                store_value=True,
                                cast_value=float,
                                variable=variable,
                                maxmin=maxmin,
                                lorentz_line=lorentz_line,
                                window_radius=window_radius,
                                theta_start=theta_range[0],
                                theta_stop=theta_range[1],
                                period_start=period_start,
                                period_stop=period_stop,
                            )
                        )

In [ ]:
df = pd.concat(opt_params, ignore_index=True)

df.to_csv(f"data/run_{run_number}/optimal_norm_2.csv", index=False, mode="w")
# df.to_csv(f"data/run_{run_number}/optimal_norm_2.csv", index=False, mode="a", header=False)

### Recording datasets optimised over period

In [ ]:
def opt_over_period(
    dataset,
    ref=None,
    common_dim=None,
    variable=None,
    optimise='max',
    lorentz_line=0,
    window_radius=0,
    theta_start=0,
    theta_stop=None,
    period_start=None,
    period_stop=None,
    load=False,
):
    opt_ds = opt_over_dim(
        dataset,
        ref=ref,
        common_dim=common_dim,
        variable=variable,
        optimise=optimise,
        lorentz_line=lorentz_line,
        window_radius=window_radius,
        theta=theta_start,
        period_start=period_start,
        period_stop=period_stop,
        integrate_angle=theta_stop,
        dim="period",
        load=load,
    )

    opt_ds = opt_ds.squeeze(drop=True).drop_vars([
        "frequency",
         "theta",
         "wavelength"
        ], errors="ignore"
    )

    return opt_ds

#### norm_1

In [ ]:
dataset = ds.drop_sel(excitonic_layer_thickness=0)

In [ ]:
resources = {
    "dataset": dataset,
    "variable": unpolarised_RTA,
}

In [ ]:
constants = {
    "ref": "crs_1",
    "period_start": int(dataset.period.min().values),
    "period_stop": int(dataset.period.max().values),
}

In [ ]:
combos = {
    "optimise": ["max", "min"],
    # "lorentz_line": [-5, 0, 5],
    "lorentz_line": [-8, -2, 2, 8],
    "window_radius": [0, 2, 5],
}

In [ ]:
cases = [
    {"theta_start": 0, "theta_stop": 0},
    {"theta_start": 30, "theta_stop": 0},
    {"theta_start": 0, "theta_stop": 45},
]

In [ ]:
r = xyz.Runner(opt_over_period, var_names=None, constants=constants, resources=resources)

In [ ]:
# r.run_cases([{"optimise": "max", "lorentz_line": -5, "window_radius": 5, "theta_start": 0, "theta_stop": 45}])

In [ ]:
h = xyz.Harvester(runner=r, data_name=f"data/run_{run_number}/opt_over_period_norm_1.nc")

In [ ]:
h.harvest_combos(combos=combos, cases=cases)

#### LOPC

In [ ]:
dataset = restacked_ds.drop_sel(excitonic_layer_thickness=0)

In [ ]:
resources = {
    "dataset": dataset,
    "variable": unpolarised_vars,
    "load": True,
}

In [ ]:
constants = {
    "period_start": int(dataset.period.min().values),
    "period_stop": int(dataset.period.max().values),
}

In [ ]:
combos = {
    "optimise": ["max", "min"],
    "lorentz_line": [-8, -5, -2, 0, 2, 5, 8],
    "window_radius": [0, 2, 5],
    "theta_start": [0, 15, 30],
    "theta_stop": [0, 45],
}

In [ ]:
r = xyz.Runner(opt_over_period, var_names=None, constants=constants, resources=resources)

In [ ]:
%%time
r.run_cases([{"optimise": "max", "lorentz_line": -5, "window_radius": 5, "theta_start": 0, "theta_stop": 45}]).compute()

In [ ]:
h = xyz.Harvester(runner=r, data_name=f"data/run_{run_number}/opt_over_period_LOPC.nc")

In [ ]:
h.harvest_combos(combos=combos)

#### diff_1

In [ ]:
dataset = restacked_diff_1.drop_sel(excitonic_layer_thickness=0)

In [ ]:
resources = {
    "dataset": dataset,
    "variable": unpolarised_RTA,
    "load": False,
}

In [ ]:
constants = {
    "period_start": int(dataset.period.min().values),
    "period_stop": int(dataset.period.max().values),
}

In [ ]:
combos = {
    "optimise": ["max", "min"],
    # "lorentz_line": [-5, 0, 5],
    "lorentz_line": [-8, -2, 2, 8],
    "window_radius": [0, 2, 5],
    "theta_start": [0, 15, 30],
    "theta_stop": [0, 45],
}

In [ ]:
r = xyz.Runner(opt_over_period, var_names=None, constants=constants, resources=resources)

In [ ]:
# %%time
# r.run_cases([{"optimise": "max", "lorentz_line": -5, "window_radius": 5, "theta_start": 0, "theta_stop": 45}]).compute()

In [ ]:
h = xyz.Harvester(runner=r, data_name=f"data/run_{run_number}/opt_over_period_diff_1.nc")

In [ ]:
h.harvest_combos(combos=combos)

# Tunability

## Investigation

In [ ]:
xs_1D = {
    "excitonic_layer_thickness": pnw.DiscreteSlider,
    "theta": pnw.DiscreteSlider,
    "num_periods": pnw.DiscreteSlider,
    "passive_layer_thickness": pnw.DiscreteSlider,
}

ds_sel = ds[s_polarised_RTA].interactive(loc="left").sel(**xs_1D).squeeze(drop=True)

In [ ]:
# Get a handle on the widgets
widgets = {k: w for k, w in zip(xs_1D, ds_sel.widgets())}

In [ ]:
rolling = ds[s_polarised_RTA].interactive(loc="left").rolling(passive_layer_thickness=3, center=True)
# rolling = ds.rolling(passive_layer_thickness=3, center=True)

In [ ]:
ptp = rolling.reduce(np.ptp)

In [ ]:
ptp_sel = ptp.sel(**widgets).squeeze(drop=True)

In [ ]:
{k: v.value for k, v in widgets.items()}

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

(ptp_sel.hvplot(x="frequency") + ds_sel.hvplot(x="frequency")).cols(1)

In [ ]:
widgets_plt = {k: v for k, v in widgets.items() if k != "passive_layer_thickness"}

In [ ]:
ptp_sel_plt = ptp.sel(**widgets_plt).squeeze(drop=True)

In [ ]:
ds_sel_plt = ds[s_polarised_RTA].interactive(loc="left").sel(**widgets_plt).squeeze(drop=True)

In [ ]:
temp_plots = []
for var in s_polarised_RTA:
    temp_plots.append(plot_var(variable=var, dataset=ptp_sel_plt, kind="quadmesh", x="frequency", y="passive_layer_thickness", label=None))
    temp_plots.append(plot_var(variable=var, dataset=ds_sel_plt, kind="quadmesh", x="frequency", y="passive_layer_thickness", label=None))

In [ ]:
for p in temp_plots:
    display(p)

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

# I'm not sure why this raises KeyError - must be a bug with .interactive API
# (hv.Layout(temp_plots))#.cols(2) 

In [ ]:
ptp = pre_process_for_plots(ptp)

In [ ]:
widgets_th = {k: v for k, v in widgets.items() if k != "theta"}

In [ ]:
ptp_sel_th = ptp.sel(**widgets_th).squeeze(drop=True)

In [ ]:
ds_sel_th = ds[s_polarised_RTA].interactive(loc="left").sel(**widgets_th).squeeze(drop=True)

In [ ]:
temp_plots = []
for var in s_polarised_RTA:
    temp_plots.append(plot_var(variable=var, dataset=ptp_sel_th, kind="quadmesh", x="wavelength", y="theta", label=None))
    temp_plots.append(plot_var(variable=var, dataset=ds_sel_th, kind="quadmesh", x="wavelength", y="theta", label=None))

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

# I'm not sure why this raises KeyError - must be a bug with .interactive API
(hv.Layout(temp_plots))#.cols(2) 

In [ ]:
for p in temp_plots:
    display(p)

## Plots

Plots I want:
- 1D plots of tunability in any given dimension as a function of any given dimension, grouped by all the others
- 2D plots of tunability in any given dimension as a function of any two given dimension, grouped by all the others
- Plots of the optimal degree of tunability, etc.

Notes:
- The most important dimension to be tunable in is period/passive layer thickness
- The most important dimension to plot by is frequency/wavelength
- The second most important dimension for both is probably angle theta
- Multi-dimensional tunability is possible (via multi-dimensional rolling) but I'm not sure it's desirable - maybe plt and theta together?
- Tunability of an integrated FOM has to be calculated *after* the integration - integrating tunability doesn't really make sense
- Tunability = peak-to-peak over a window of fixed size, for me. There's cleverer stuff you can do with exponentially weighted rolling windows or whatever but no compelling reason to
- `.hvplot(..., groupby=[...])` will not work because the array is too big - have to use `.interactive` API or function to select array before DynamicMap call

Interesting things to investigate:
* Tunability (period) in all the FOMs...
    * Including integrated in wavelength and angle!
* ... as a function of all the other parameters.
* Does tunability of integrated FOM correlate with tunability of raw FOM?
* Double-band in tunability of R
* Hot spots in tunability of A
* Optimal tunability, and if optimal params for tunability relate to optimal params for FOMs in a consistent way
* Size of tuning window is itself a new parameter to consider

### Helper functions

In [ ]:
def nanptp(a, axis=None):
    """
    Essentially equivalent to numpy.ptp, but ignores nan values in array.
    Credit to yliapis's rejected PR: https://github.com/numpy/numpy/pull/13220/commits/f83ec2fbacaa78717518fe2af4a51d824f421eed
    """
    amin = np.nanmin(a, axis)
    amax = np.nanmax(a, axis)
    res = amax - amin
    return res

In [ ]:
def nanquotient(a, axis=None):
    """
    Similar to nanptp, but taking the ratio rather than difference between the max and min.
    """
    amin = np.nanmin(a, axis)
    amax = np.nanmax(a, axis)
    res = amax / amin
    return res

In [ ]:
def ptp(ds, dim=None, min_periods=1, center=True, mode="difference", **window_kwargs):
    match mode:
        case "difference":
            ptp_func = nanptp
        case "quotient":
            ptp_func = nanquotient
        case _:
            raise NotImplementedError(
                f'argument "mode" must be "difference" or "quotient", not {mode}'
            )
    
    rolling = ds.rolling(dim=dim, min_periods=min_periods, center=center, **window_kwargs)
    ptp = rolling.reduce(ptp_func)
    return ptp

##### Tests

In [ ]:
dummy_da = xr.DataArray(np.arange(0, 20).reshape(10, 2), coords={'dummy': np.arange(10, 20), 'dummy2': ['a', 'b']})

dummy_ds = xr.Dataset(data_vars={'var1': dummy_da})

In [ ]:
dummy_ds

In [ ]:
dummy_ds.rolling(dummy=4, center=True).reduce(np.max)#.shift(dummy=-3)

In [ ]:
dummy_ds.rolling(dummy=2, min_periods=None
                ).construct('window').reduce(nanptp, 'window')

In [ ]:
dummy_ds.rolling(dummy=3, min_periods=1).reduce(nanptp)

In [ ]:
dummy_ds.rolling(dummy=3, dummy2=2).construct({'dummy': 'window', 'dummy2': 'window2'}).reduce(np.max, dim=['window', 'window2'])

In [ ]:
ptp(dummy_ds.pad(dummy=1), min_periods=1, dummy=3)

In [ ]:
ds.pad(passive_layer_thickness=1)

In [ ]:
ptp(dummy_ds, center=True, dummy=4)

In [ ]:
ptp(ds.R, center=True, theta=5)

### plot_tunability_RTA?

In [ ]:
def plot_tunability(
    dataset,
    coords: dict,
    title="",
    plot_vars=["R", "T", "A"],
    ptp_dim=None,  # parameter which designates which dim(s?) we're looking at tunability over
    ptp_window=None,  # parameter for how wide the tunability window is
    ptp_mode="difference",  # difference or quotient tunability
    ptp_kwargs=None,
    label_override=None,
    label_append=None,
    **plot_func_args,
):
    label_field = None  # for debugging (LaTeX labels don't seem to render properly)
    label_append = "" if label_append is None else label_append

    if not {**plot_func_args}:  # if no plot_func_args were passed
        plot_func_args |= {"x": "wavelength"}  # default behaviour of plot_RTA

    plot_funcs = {
        plot_var: partial(plot_func, label_field=label_field, **plot_func_args)
        for plot_var, plot_func in zip(plot_vars, plot_vars_to_funcs(plot_vars))
    }

    label = f" (Tunability)" if label_override is None else label_override
    label += label_append
    # lopc_sel = sel_or_integrate(lopc_sel, "theta", theta, normalisation=1)  # this happens to ds before coming in

    # calculate peak-to-peak tunability
    if ptp_kwargs is None:
        ptp_kwargs = {}
    if (
        dim not in ptp_kwargs
    ):  # if for some reason you explicitly passed dim, can't pass window_kwargs
        ptp_kwargs |= {ptp_dim: ptp_window}
    ptp_kwargs |= {"mode": ptp_mode}
    ds_ptp = ptp(dataset, **ptp_kwargs)

    ds_sel = ds_ptp.sel(**coords).squeeze()

    plots = {
        plot_var: plot_func(dataset=ds_sel, label_append=label)
        for plot_var, plot_func in plot_funcs.items()
    }

    return plots

In [ ]:
# def RTA_curve_opts(title=None):
#     if title is None:
#         title = ""
#     return [
#         opts.Curve(
#             ylim=(0, 1),
#             ylabel="Intensity",
#             title=f"{title}",
#         ),
#     ]

In [ ]:
# a more flexible implementation of the above
RTA_curve_opts = partial(opts.Curve, ylim=(0,1), ylabel="Intensity")

In [ ]:
def RTA_2D_opts(title=None):
    if title is None:
        title = ""
    return [
        opts.Image(
            clim=(0, 1),
            clabel="Intensity",
            title=f"{title}",
        ),
        opts.QuadMesh(
            clim=(0, 1),
            clabel="Intensity",
            title=f"{title}",
        ),
        opts.Layout(
            title=f"{title}",
        ),
    ]

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero and all-Nan slice warnings

theta = (0, 75)

coords = {"period": 210,
         "excitonic_layer_thickness": 30,
         "num_periods": 10,}

test_dataset = sel_or_integrate(restacked_ds, "theta", theta, normalisation=1)

plots = plot_tunability(test_dataset, coords, ptp_dim="period", ptp_window=3, label_override="")
test_overlay = hv.Overlay(plots.values()).opts(RTA_curve_opts(title=coordinate_string(**coords) + f"\ntheta: {theta}"))

In [ ]:
plots

In [ ]:
test_overlay

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero and all-Nan slice warnings

theta = (0, 45)

coords = {"period": 210,
         "excitonic_layer_thickness": 30,
         "num_periods": 10,}

test_dataset = sel_or_integrate(restacked_ds, "theta", theta, normalisation=1)

plots = plot_tunability(test_dataset, coords, ptp_dim="period", ptp_window=3, ptp_mode="quotient", label_override="")
test_overlay = hv.Overlay(plots.values()).opts(RTA_curve_opts(title=(coordinate_string(**coords) + f"\ntheta: {theta}"), ylim=(1, None)))

In [ ]:
plots

In [ ]:
test_overlay

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero and all-Nan slice warnings

(
    plot_RTA(**coords, theta=theta, plot_vars=["R"], include=["LOPC"]).opts(
        opts.Curve(line_dash="dashed", ylim=(0,1)#hooks=[plot_secondary]
                  )
    )
) * (
    plot_tunability(test_dataset, coords, plot_vars=["A"], ptp_dim="period", ptp_window=3, ptp_mode="quotient", label_override=" Tunability")["A"].opts(
        opts.Curve(#ylim=(1,None),
            hooks=[plot_secondary],
                   apply_ranges=False
        )
    )
)

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero and all-Nan slice warnings

(
    plot_RTA(**coords, theta=theta, plot_vars=["R"], include=["LOPC"]).opts(
        opts.Curve(line_dash="dashed", ylim=(None,None))  # hooks=[plot_secondary]
    )
) * (
    hv.Overlay(
        plot_tunability(
            test_dataset,
            coords,
            plot_vars=["R", "T", "A"],
            ptp_dim="period",
            ptp_window=3,
            ptp_mode="difference",
            label_override=" Tunability",
        ).values()
    ).opts(opts.Curve(ylim=(None,None)))
).opts(opts.Curve(ylim=(None,None)))

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero and all-Nan slice warnings

coords = {"period": 210,
          "theta": 0,
          "wavelength": float(ds.wavelength[128])
         #"excitonic_layer_thickness": 30,
         #"num_periods": 10,
         }

test_dataset = restacked_ds

plots = plot_tunability(test_dataset, coords, ptp_dim="period", ptp_window=3, label_override="", x="excitonic_layer_thickness", y="num_periods", kind="quadmesh")
test_layout = hv.Layout(plots.values()).opts(RTA_2D_opts(coordinate_string(**coords)))

In [ ]:
plots

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero and all-Nan slice warnings

test_layout.cols(1)

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero and all-Nan slice warnings

coords = {"period": 250,
         "excitonic_layer_thickness": 20,
         "num_periods": 6,
         "theta": 0
         }

test_dataset = restacked_ds

plots = plot_tunability(test_dataset, coords, ptp_dim="period", ptp_window=9, x="wavelength")
test_overlay = hv.Overlay(plots.values()).opts(RTA_curve_opts(title=coordinate_string(**coords)))

In [ ]:
(
    test_overlay.opts(opts.Curve(line_dash="dashed"))
    * plot_RTA(**coords, include=["LOPC"])
    * lorentz_vlines(0, scale=1e-9)
).opts(opts.VLine(color=yellow, line_dash="dotted"), opts.Overlay(legend_position="right"))

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero and all-Nan slice warnings

coords = {"period": 250,
         "excitonic_layer_thickness": 20,
         "num_periods": 30,
         "theta": 0
         }

plots = plot_tunability(restacked_ds, coords, ptp_dim="period", ptp_window=9, ptp_mode="quotient", x="wavelength")
more_plots = plot_tunability(restacked_norm_1, coords, ptp_dim="period", ptp_window=9, ptp_mode="quotient", x="wavelength")

test_overlay = hv.Overlay(list(plots.values())) * (
    hv.Overlay(list(more_plots.values())).opts(opts.Curve(line_dash="dashed"))
).opts(RTA_curve_opts(ylim=(None, None), title=coordinate_string(**coords)))
test_overlay

### Wrapper and dmap

In [ ]:
def wrapped_2D_tunability_plot(
    variable,
    dataset,
    optimise="max",
    lorentz_line=0,
    window_radius=0,
    theta=0,
    cmap="viridis",
    period_start=None,
    period_stop=None,
    integrate_angle=None,
    extra_plots=["RTA_normal", "RTA_int", "norm_1_normal", "norm_1_int"],
    dim=None,  # automatically assign if dataset recognised
    ptp_dim=None,  # parameter which designates which dim(s?) we're looking at tunability over
    ptp_window=None,  # parameter for how wide the tunability window is
    ptp_kwargs=None,
):
    plots = []

    if str(dataset) == "restacked_ds":
        # the drop_sel is important for avoiding the most common degenerate cases
        dataset = restacked_ds.drop_sel({"excitonic_layer_thickness": 0})
        dataset = dataset.drop_sel({"period": 0})
        dim = "period"

    if str(dataset) == "restacked_norm_1":
        # the drop_sel is important for avoiding the most common degenerate cases
        dataset = restacked_norm_1.drop_sel({"excitonic_layer_thickness": 0})
        dataset = dataset.drop_sel({"period": 0})
        dim = "period"

    if str(dataset) == "restacked_diff_1":
        # the drop_sel is important for avoiding the most common degenerate cases
        dataset = restacked_diff_1.drop_sel({"excitonic_layer_thickness": 0})
        dataset = dataset.drop_sel({"period": 0})
        dim = "period"

    if str(dataset) == "restacked_ds_flat_spectrum":
        # the drop_sel is important for avoiding the most common degenerate cases
        dataset = restacked_ds_flat_spectrum.drop_sel({"excitonic_layer_thickness": 0})
        dataset = dataset.drop_sel({"period": 0})
        dim = "period"
        window_radius = None  # circumvent wavelength selection later

    if str(dataset) == "restacked_norm_flat_spectrum":
        # the drop_sel is important for avoiding the most common degenerate cases
        dataset = restacked_norm_flat_spectrum.drop_sel(
            {"excitonic_layer_thickness": 0}
        )
        dataset = dataset.drop_sel({"period": 0})
        dim = "period"
        window_radius = None  # circumvent wavelength selection later

    da = dataset[variable]

    # integration needs to happen BEFORE tunability is calculated to be meaningful
    # but selection of angle (if not integrating) needs to happen AFTER ptp because
    # theta is a valid tunabilty dimension

    if (
        integrate_angle
    ):  # integrate_angle must be a float, so that (theta, integrate_angle) is a slice syntax
        if ptp_dim == "theta":
            raise ValueError(
                "Can't calculate ptp over theta and integrate over theta simultaneously"
            )
        da = da.sel(theta=slice(theta, integrate_angle))
        da = integrate_da(da, "theta", weighting=1, normalisation=1)

    if period_start < period_stop:
        da = da.sel(period=slice(period_start, period_stop))
    else:  # otherwise no data is selected and everything breaks
        da = da.sel(period=slice(period_start, None))

    vline_locs = [0]  # where the vlines go

    if (
        window_radius is not None
    ):  # a hack to make this function work with pre-integrated datasets
        da = select_lorentz_line(
            da, lorentz_line=lorentz_line, window_radius=window_radius
        )
        if window_radius == 0:
            wavelength = float(da.wavelength)
            title = (
                f"{optimise.capitalize()}imum tunability in {ptp_dim} with window radius {ptp_window}\n"
                f"of {variable} at {wavelength:.0f} nm"
            )
            if lorentz_line != 0:  # don't put two lines over the resonance
                vline_locs.append(lorentz_line)
        else:
            wavelength_start = float(da.wavelength[0])
            wavelength_stop = float(da.wavelength[-1])
            # make it easier to compare values
            da = integrate_da(
                da, "wavelength", weighting=1, normalisation=1
            )  # replaces the below two lines
            # da = normalise_over_dim(da, "wavelength", 1)
            # da = da.integrate("wavelength")
            title = (
                f"{optimise.capitalize()}imum tunability in {ptp_dim} with window radius {ptp_window}\n"
                f"of integrated {variable} between {wavelength_start:.0f} and {wavelength_stop:.0f} nm"
            )
            vline_locs.append(lorentz_line - window_radius)
            vline_locs.append(lorentz_line + window_radius)
    elif window_radius is None:
        title = (
            f"{optimise.capitalize()}imum tunability in {ptp_dim} with window radius {ptp_window}\n"
            f"of {variable} integrated over the whole spectrum"
        )
        if lorentz_line != 0:  # don't put two lines over the resonance
            vline_locs.append(
                lorentz_line
            )  # this line is just a guide for the eye, it doesn't signify anything

    # calculate peak-to-peak tunability
    if ptp_kwargs is None:
        ptp_kwargs = {}
    if (
        dim not in ptp_kwargs
    ):  # if for some reason you explicitly passed dim, can't pass window_kwargs
        ptp_kwargs |= {ptp_dim: ptp_window}
    da = ptp(da, **ptp_kwargs)

    # it's safe to slice in theta now
    if not integrate_angle:
        da = da.sel(theta=theta, method="nearest")

    plot_1, optimum_coords = plot_optimum_over_dim(
        da,
        dim=dim,
        x="excitonic_layer_thickness",
        y="num_periods",
        optimise=optimise,
    )

    P = float(optimum_coords["period"])
    t = float(optimum_coords["excitonic_layer_thickness"])
    N = float(optimum_coords["num_periods"])
    try:  # this should work if not integrating over theta
        th = float(optimum_coords["theta"])
    except:  # probably the problem is that theta doesn't exist because I integrated over it already
        th = (theta, integrate_angle)
    lorentz_lines = lorentz_vlines(vline_locs, scale=1e-9, mode="wavelength").opts(
        opts.VLine(line_color=green, line_dash="dotted"),
    )

    # give the resonance line a special colour
    lorentz_lines.VLine.I.opts(opts.VLine(line_color=yellow))

    plot_1.opts(
        opts.QuadMesh(cmap=cmap),
        opts.Points(color="red"),
        opts.Overlay(title=f"{title}\nOptimal period: {P:.0f}"),
    )

    plots.append(plot_1)

    if "RTA_normal" in extra_plots:  # plot RTA at theta=0
        new_plot = plot_RTA(
            period=P, excitonic_layer_thickness=t, num_periods=N, theta=0
        )
        new_plot *= lorentz_lines

        plots.append(new_plot)

    if "RTA_int" in extra_plots:  # plot RTA at theta OR integrating over theta
        new_plot = plot_RTA(
            period=P, excitonic_layer_thickness=t, num_periods=N, theta=th
        )
        new_plot *= lorentz_lines
        plots.append(new_plot)

    if "norm_1_normal" in extra_plots:  # plot enhancement factor at theta=0
        sel = {
            "period": P,
            "excitonic_layer_thickness": t,
            "num_periods": N,
            "theta": 0,
        }
        new_plot = plot_ef(variable="A", dataset=restacked_norm_1, sel=sel)
        new_plot *= lorentz_lines

        plots.append(new_plot)

    if (
        "norm_1_int" in extra_plots
    ):  # plot enhancement factor at theta OR integrating over theta
        try:  # this should work if not integrating over theta
            sel = {
                "period": P,
                "excitonic_layer_thickness": t,
                "num_periods": N,
                "theta": th,
            }
            new_plot = plot_ef(variable="A", dataset=restacked_norm_1, sel=sel)
            new_plot *= lorentz_lines
        except:  # if integrating, we need to do the integral *before* normalising
            ds_int = sel_or_integrate(ds, dim="theta", val=th)
            crs_1_int = sel_or_integrate(crs_1, dim="theta", val=th)
            norm = enhancement_factor(
                ds_int,
                ref=crs_1_int,
                common_dim="total_excitonic_thickness",
                method="groupby",
            )
            restacked_norm = restack_plt_to_period(norm)
            # replaces the lines below
            #             # this should all get separated out into its own function
            #             crs_1_like_ds = crs_1.sel(
            #                 total_excitonic_thickness=ds.total_excitonic_thickness
            #             )

            #             ds_int = sel_or_integrate(ds, dim="theta", val=th)
            #             crs_1_int = sel_or_integrate(crs_1_like_ds, dim="theta", val=th)
            #             norm = ds_int / crs_1_int
            #             restacked_norm = norm.stack(multiperiod=['passive_layer_thickness', 'excitonic_layer_thickness']).set_index(multiperiod=['period', 'excitonic_layer_thickness']).unstack()

            sel = {"period": P, "excitonic_layer_thickness": t, "num_periods": N}
            new_plot = plot_ef(variable="A", dataset=restacked_norm, sel=sel)
            new_plot *= lorentz_lines
            sel["theta"] = th
            new_plot = new_plot.opts(opts.Overlay(title=f"{coordinate_string(**sel)}"))
        plots.append(new_plot)

    if "RTA_ptp" in extra_plots:
        coords = {
            "period": P,
            "excitonic_layer_thickness": t,
            "num_periods": N,
        }
        dataset = sel_or_integrate(restacked_ds, dim="theta", val=th, normalisation=1)

        dict_of_plots = plot_tunability(
            dataset,
            coords=coords,
            ptp_dim=ptp_dim,
            ptp_window=ptp_window,
            ptp_kwargs=ptp_kwargs,
            x="wavelength",
        )
        new_plot = hv.Overlay(dict_of_plots.values()).opts(
            RTA_curve_opts(title=coordinate_string(**coords | {"theta": th}))
        )
        plots.append(new_plot)

    return hv.Layout(plots).cols(1)

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero and all-Nan slice warnings

wrapped_2D_tunability_plot(
    variable="A_per_oscillator",
    dataset="restacked_ds",
    optimise="max",
    lorentz_line=-5,
    window_radius=0,
    theta=0,
    cmap="inferno",
    period_start=0,
    period_stop=0,
    integrate_angle=30,
    extra_plots=["RTA_normal", "RTA_int", "RTA_ptp"],
    # dim=None,  # automatically assign if dataset recognised
    ptp_dim="period",  # parameter which designates which dim(s?) we're looking at tunability over
    ptp_window=3,  # parameter for how wide the tunability window is
    ptp_kwargs={"center": False}
)

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero and all-Nan slice warnings

coords = {"period": 250,
         "excitonic_layer_thickness": 30,
         "num_periods": 30,
         "theta": 0
         }

In [ ]:
test_dataset = restacked_ds

plots = plot_tunability(test_dataset, coords, ptp_dim="period", ptp_window=3, x="wavelength")
test_overlay = hv.Overlay(plots.values()).opts(RTA_curve_opts(title=coordinate_string(**coords)))

In [ ]:
test_overlay.opts(opts.Curve(line_dash="dashed"))*plot_RTA(**coords, include=["LOPC"])*lorentz_vlines(0, scale=1e-9).opts(opts.VLine(color=yellow, line_dash="dotted"))

In [ ]:
plot_RTA(**coords|{"period":240}, include=["LOPC"])*lorentz_vlines(0, scale=1e-9).opts(opts.VLine(color=yellow, line_dash="dotted"))

In [ ]:
plot_RTA(**coords|{"period":260}, include=["LOPC"])*lorentz_vlines(0, scale=1e-9).opts(opts.VLine(color=yellow, line_dash="dotted"))

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero and all-Nan slice warnings

coords = {"period": 250,
          "wavelength": float(ds.wavelength[127]),
         "theta": 0
         }

test_dataset = restacked_ds

plots = plot_tunability(test_dataset, coords, ptp_dim="period", ptp_window=3, x="excitonic_layer_thickness", y="num_periods", kind="quadmesh")
test_layout = hv.Layout(plots.values()).opts(RTA_2D_opts(coordinate_string(**coords)))

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero and all-Nan slice warnings

test_layout

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero and all-Nan slice warnings

wrapped_2D_tunability_plot(
    variable="A",
    dataset="restacked_ds",
    optimise="max",
    lorentz_line=0,
    window_radius=2,
    theta=0,
    cmap="viridis",
    period_start=0,
    period_stop=0,
    integrate_angle=None,
    extra_plots=["RTA_int"],
    # dim=None,  # automatically assign if dataset recognised
    ptp_dim="period",  # parameter which designates which dim(s?) we're looking at tunability over
    ptp_window=3,  # parameter for how wide the tunability window is
    ptp_kwargs={"center": False}
)

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero and all-Nan slice warnings

wrapped_2D_tunability_plot(
    variable="A",
    dataset="restacked_ds",
    optimise="max",
    lorentz_line=0,
    window_radius=2,
    theta=0,
    cmap="viridis",
    period_start=0,
    period_stop=0,
    integrate_angle=30,
    extra_plots=["RTA_normal", "RTA_int"],
    # dim=None,  # automatically assign if dataset recognised
    ptp_dim="period",  # parameter which designates which dim(s?) we're looking at tunability over
    ptp_window=3,  # parameter for how wide the tunability window is
    ptp_kwargs={"center": False}
)

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero and all-Nan slice warnings

foo = wrapped_2D_tunability_plot(
    variable="A",
    dataset="restacked_norm_1",
    optimise="max",
    lorentz_line=0,
    window_radius=0,
    theta=0,
    cmap="inferno",
    period_start=0,
    period_stop=0,
    integrate_angle=None,
    extra_plots=["RTA_int", "norm_1_int"],
    # dim=None,  # automatically assign if dataset recognised
    ptp_dim="period",  # parameter which designates which dim(s?) we're looking at tunability over
    ptp_window=3,  # parameter for how wide the tunability window is
    ptp_kwargs={"center": False}
)
foo

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero and all-Nan slice warnings

coords = {"period": 250,
         "excitonic_layer_thickness": 30,
         "num_periods": 27,
         "theta": 0
         }

test_dataset = restacked_ds

plots = plot_tunability(test_dataset, coords, ptp_dim="period", ptp_window=3, x="wavelength")
test_overlay = hv.Overlay(plots.values()).opts(RTA_curve_opts(title=coordinate_string(**coords)))

In [ ]:
test_overlay.opts(opts.Curve(line_dash="dashed"))*plot_RTA(**coords, include=["LOPC"])*lorentz_vlines(0, scale=1e-9).opts(opts.VLine(color=yellow, line_dash="dotted"))

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero and all-Nan slice warnings

coords = {"period": 250,
         "excitonic_layer_thickness": 30,
         "num_periods": 27,
         "theta": 0
         }

test_dataset = restacked_norm_1

plots = plot_tunability(test_dataset, coords, ptp_dim="period", ptp_window=3, x="wavelength", plot_vars=["A"])
test_overlay = hv.Overlay(plots.values())

In [ ]:
(
    plot_ef(variable="A", dataset=restacked_norm_1, sel=coords)
    * plot_ef(variable="A", dataset=restacked_norm_1, sel=coords|{"period": 240}).opts(opts.Curve(line_dash="dotdash"))
    * plot_ef(variable="A", dataset=restacked_norm_1, sel=coords|{"period": 260}).opts(opts.Curve(line_dash="dashdot"))
).opts(show_legend=False, ylabel=r"\(\gamma_A\)")

In [ ]:
test_overlay.opts(opts.Curve(line_dash="dashed"))*plot_ef(**coords, include=["LOPC"])*lorentz_vlines(0, scale=1e-9).opts(opts.VLine(color=yellow, line_dash="dotted"))

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero and all-Nan slice warnings

coords = {"excitonic_layer_thickness": 30, "num_periods": 27, "theta": 0}

test_dataset = restacked_norm_1.sel(period=slice(30, 340))

plots = plot_tunability(
    test_dataset,
    coords,
    ptp_dim="period",
    ptp_window=3,
    x="wavelength",
    y="period",
    plot_vars=["A"],
)
test_layout = hv.Layout(plots.values())

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero and all-Nan slice warnings

test_layout.opts(opts.Image(clim=(None,None)))

In [ ]:
test_overlay.opts(opts.Curve(line_dash="dashed"))*plot_ef(**coords, include=["LOPC"])*lorentz_vlines(0, scale=1e-9).opts(opts.VLine(color=yellow, line_dash="dotted"))

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero and all-Nan slice warnings

wrapped_2D_tunability_plot(
    variable="R_per_oscillator",
    dataset="restacked_ds",
    optimise="max",
    lorentz_line=2,
    window_radius=0,
    theta=0,
    cmap="viridis",
    period_start=0,
    period_stop=0,
    integrate_angle=None,
    extra_plots=["RTA_int"],
    # dim=None,  # automatically assign if dataset recognised
    ptp_dim="period",  # parameter which designates which dim(s?) we're looking at tunability over
    ptp_window=9,  # parameter for how wide the tunability window is
    ptp_kwargs={"center": True}
)

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero and all-Nan slice warnings

coords = {"period": 290,
         "excitonic_layer_thickness": 40,
         "num_periods": 11,
         "theta": 0
         }

test_dataset = restacked_ds

plots = plot_tunability(test_dataset, coords, ptp_dim="period", ptp_window=9, x="wavelength")
test_overlay = hv.Overlay(plots.values()).opts(RTA_curve_opts(title=coordinate_string(**coords)))

In [ ]:
test_overlay.opts(opts.Curve(line_dash="dashed"))*plot_RTA(**coords, include=["LOPC"])*lorentz_vlines([0, 2], scale=1e-9).opts(opts.VLine(color=yellow, line_dash="dotted"))

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero and all-Nan slice warnings

plot_vars = ["R_per_oscillator", "T_per_oscillator", "A_per_oscillator"]
plots = plot_tunability(test_dataset, coords, ptp_dim="period", ptp_window=9, x="wavelength", plot_vars=plot_vars)
test_overlay = hv.Overlay(plots.values()).opts(RTA_curve_opts(title=coordinate_string(**coords)))

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero and all-Nan slice warnings

(
    test_overlay.opts(opts.Curve(line_dash="dashed"))
    * plot_RTA(**coords, plot_vars=plot_vars, include=["LOPC"])
    * lorentz_vlines([0, 2], scale=1e-9)
).opts(opts.VLine(color=yellow, line_dash="dotted"), opts.Curve(ylim=(None, None)))

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero and all-Nan slice warnings

wrapped_2D_tunability_plot(
    variable="T",
    dataset="restacked_ds",
    optimise="max",
    lorentz_line=2,
    window_radius=2,
    theta=0,
    cmap="cividis",
    period_start=0,
    period_stop=0,
    integrate_angle=None,
    extra_plots=["RTA_int"],
    # dim=None,  # automatically assign if dataset recognised
    ptp_dim="period",  # parameter which designates which dim(s?) we're looking at tunability over
    ptp_window=5,  # parameter for how wide the tunability window is
    ptp_kwargs={"center": True}
)

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero and all-Nan slice warnings

dataset = restacked_norm_flat_spectrum.drop_sel(excitonic_layer_thickness=0, period=0)

wrapped_2D_tunability_plot(
    variable="A",
    dataset=dataset,
    optimise="max",
    lorentz_line=0,
    window_radius=None,
    theta=0,
    cmap="inferno",
    period_start=0,
    period_stop=0,
    integrate_angle=None,
    extra_plots=["RTA_int", "norm_1_int"],
    dim="period",  # automatically assign if dataset recognised
    ptp_dim="period",  # parameter which designates which dim(s?) we're looking at tunability over
    ptp_window=5,  # parameter for how wide the tunability window is
    ptp_kwargs={"center": True}
)

In [ ]:
coords = {
    "excitonic_layer_thickness": 60,
    "num_periods": 7,
    "theta": 0,
}

dataset.sel(**coords).squeeze().A.hvplot(x="period")*(hv.HLine(1).opts(line_dash="dotted"))

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero and all-Nan slice warnings

dataset = restacked_ds_flat_spectrum.drop_sel(excitonic_layer_thickness=0, period=0)

wrapped_2D_tunability_plot(
    variable="T",
    dataset=dataset,
    optimise="max",
    lorentz_line=2,
    window_radius=None,
    theta=0,
    cmap="cividis",
    period_start=0,
    period_stop=0,
    integrate_angle=None,
    extra_plots=["RTA_int", "norm_1_int"],
    dim="period",  # automatically assign if dataset recognised
    ptp_dim="period",  # parameter which designates which dim(s?) we're looking at tunability over
    ptp_window=5,  # parameter for how wide the tunability window is
    ptp_kwargs={"center": True}
)

In [ ]:
coords = {
    "excitonic_layer_thickness": 80,
    "num_periods": 40,
    "theta": 0,
}

dataset.sel(**coords).squeeze().T.hvplot(x="period")

In [ ]:
def wrapped_2D_plot(
    variable,
    dataset,
    optimise="max",
    lorentz_line=0,
    window_radius=0,
    theta=0,
    cmap="viridis",
    period_start=None,
    period_stop=None,
    integrate_angle=None,
    extra_plots=["RTA_normal", "RTA_int", "norm_1_normal", "norm_1_int"],
    dim=None,  # automatically assign if dataset recognised
):
    plots = []

    if str(dataset) == "restacked_ds":
        # the drop_sel is important for avoiding the most common degenerate cases
        dataset = restacked_ds.drop_sel({"excitonic_layer_thickness": 0})
        dim = "period"

    if str(dataset) == "restacked_norm_1":
        # the drop_sel is important for avoiding the most common degenerate cases
        dataset = restacked_norm_1.drop_sel({"excitonic_layer_thickness": 0})
        dim = "period"

    if str(dataset) == "restacked_diff_1":
        # the drop_sel is important for avoiding the most common degenerate cases
        dataset = restacked_diff_1.drop_sel({"excitonic_layer_thickness": 0})
        dim = "period"

    da = dataset[variable]

    if not integrate_angle:
        da = da.sel(theta=theta, method="nearest")
    else:  # integrate_angle must be a float, so that (theta, integrate_angle) is a slice syntax
        da = da.sel(theta=slice(theta, integrate_angle))
        da = integrate_da(da, "theta", weighting=1, normalisation=1)

    if period_start < period_stop:
        da = da.sel(period=slice(period_start, period_stop))
    else:  # otherwise no data is selected and everything breaks
        da = da.sel(period=slice(period_start, None))
    da = select_lorentz_line(da, lorentz_line=lorentz_line, window_radius=window_radius)

    vline_locs = [0]

    if window_radius == 0:
        wavelength = float(da.wavelength)
        title = f"{optimise.capitalize()}imum {variable} at {wavelength:.0f} nm"
        if lorentz_line != 0:  # don't put two lines over the resonance
            vline_locs.append(lorentz_line)
    else:
        wavelength_start = float(da.wavelength[0])
        wavelength_stop = float(da.wavelength[-1])
        # make it easier to compare values
        da = integrate_da(
            da, "wavelength", weighting=1, normalisation=1
        )  # replaces the below two lines
        # da = normalise_over_dim(da, "wavelength", 1)
        # da = da.integrate("wavelength")
        title = f"{optimise.capitalize()}imum integrated {variable} between {wavelength_start:.0f} and {wavelength_stop:.0f} nm"
        vline_locs.append(lorentz_line - window_radius)
        vline_locs.append(lorentz_line + window_radius)

    plot_1, optimum_coords = plot_optimum_over_dim(
        da,
        dim=dim,
        x="excitonic_layer_thickness",
        y="num_periods",
        optimise=optimise,
    )

    P = float(optimum_coords["period"])
    t = float(optimum_coords["excitonic_layer_thickness"])
    N = float(optimum_coords["num_periods"])
    try:  # this should work if not integrating over theta
        th = float(optimum_coords["theta"])
    except:  # probably the problem is that theta doesn't exist because I integrated over it already
        th = (theta, integrate_angle)
    lorentz_lines = lorentz_vlines(vline_locs, scale=1e-9, mode="wavelength").opts(
        opts.VLine(line_color=green, line_dash="dotted"),
    )

    # give the resonance line a special colour
    lorentz_lines.VLine.I.opts(opts.VLine(line_color=yellow))

    plot_1.opts(
        opts.QuadMesh(cmap=cmap),
        opts.Points(color="red"),
        opts.Overlay(title=f"{title}\nOptimal period: {P:.0f}"),
    )

    plots.append(plot_1)

    if "RTA_normal" in extra_plots:  # plot RTA at theta=0
        new_plot = plot_RTA(
            period=P, excitonic_layer_thickness=t, num_periods=N, theta=0
        )
        new_plot *= lorentz_lines

        plots.append(new_plot)

    if "RTA_int" in extra_plots:  # plot RTA at theta OR integrating over theta
        new_plot = plot_RTA(
            period=P, excitonic_layer_thickness=t, num_periods=N, theta=th
        )
        new_plot *= lorentz_lines
        plots.append(new_plot)

    if "norm_1_normal" in extra_plots:  # plot enhancement factor at theta=0
        sel = {
            "period": P,
            "excitonic_layer_thickness": t,
            "num_periods": N,
            "theta": 0,
        }
        new_plot = plot_ef(variable="A", dataset=restacked_norm_1, sel=sel)
        new_plot *= lorentz_lines

        plots.append(new_plot)

    if (
        "norm_1_int" in extra_plots
    ):  # plot enhancement factor at theta OR integrating over theta
        try:  # this should work if not integrating over theta
            sel = {
                "period": P,
                "excitonic_layer_thickness": t,
                "num_periods": N,
                "theta": th,
            }
            new_plot = plot_ef(variable="A", dataset=restacked_norm_1, sel=sel)
            new_plot *= lorentz_lines
        except:  # if integrating, we need to do the integral *before* normalising
            ds_int = sel_or_integrate(ds, dim="theta", val=th)
            crs_1_int = sel_or_integrate(crs_1, dim="theta", val=th)
            norm = enhancement_factor(
                ds_int,
                ref=crs_1_int,
                common_dim="total_excitonic_thickness",
                method="groupby",
            )
            restacked_norm = restack_plt_to_period(norm)
            # replaces the lines below
            #             # this should all get separated out into its own function
            #             crs_1_like_ds = crs_1.sel(
            #                 total_excitonic_thickness=ds.total_excitonic_thickness
            #             )

            #             ds_int = sel_or_integrate(ds, dim="theta", val=th)
            #             crs_1_int = sel_or_integrate(crs_1_like_ds, dim="theta", val=th)
            #             norm = ds_int / crs_1_int
            #             restacked_norm = norm.stack(multiperiod=['passive_layer_thickness', 'excitonic_layer_thickness']).set_index(multiperiod=['period', 'excitonic_layer_thickness']).unstack()

            sel = {"period": P, "excitonic_layer_thickness": t, "num_periods": N}
            new_plot = plot_ef(variable="A", dataset=restacked_norm, sel=sel)
            new_plot *= lorentz_lines
            sel["theta"] = th
            new_plot = new_plot.opts(opts.Overlay(title=f"{coordinate_string(**sel)}"))
        plots.append(new_plot)

    return hv.Layout(plots).cols(1)

In [ ]:
def wrapped_2D_ef_plot(
    variable,
    dataset,
    ref,
    optimise="max",
    lorentz_line=0,
    window_radius=0,
    theta=0,
    cmap="viridis",
    period_start=None,
    period_stop=None,
    integrate_angle: float = None,
    extra_plots=["RTA_normal", "RTA_int", "norm_1_normal", "norm_1_int"],
    dim=None,  # automatically assign if dataset recognised
    restack_ef=None,
    common_dim=None,
):
    plots = []

    # detect string labels
    if str(dataset) == "ds":
        # the drop_sel is important for avoiding the most common degenerate cases
        dataset = ds.drop_sel({"excitonic_layer_thickness": 0})

    if str(ref) == "crs_1":
        ref = crs_1
        common_dim = "total_excitonic_thickness"

    if str(dataset) == "frs_1":
        ref = frs_1
        common_dim = "total_thickness"
    
    match dim:  # if restack is the default value, automatically set it
        case "period":
            restack_ef = True if restack_ef is None else restack_ef
        case "passive_layer_thickness":
            restack_ef = False if restack_ef is None else restack_ef
    integration_dims = []
    vline_locs = [0]

    da = dataset[variable]
    ra = ref[variable]

    da = select_lorentz_line(da, lorentz_line=lorentz_line, window_radius=window_radius)
    ra = select_lorentz_line(ra, lorentz_line=lorentz_line, window_radius=window_radius)

    if window_radius == 0:
        wavelength = float(da.wavelength)
        title = f"{optimise.capitalize()}imum {variable} enhancement factor at {wavelength:.0f} nm"
        if lorentz_line != 0:  # don't put two lines over the resonance
            vline_locs.append(lorentz_line)
    else:
        wavelength_start = float(da.wavelength[0])
        wavelength_stop = float(da.wavelength[-1])
        title = f"{optimise.capitalize()}imum integrated {variable} enhancement factor between {wavelength_start:.0f} and {wavelength_stop:.0f} nm"
        vline_locs.append(lorentz_line - window_radius)
        vline_locs.append(lorentz_line + window_radius)
        integration_dims.append("wavelength")

    if integrate_angle:  # integrate angle should be a float
        th = (theta, integrate_angle)
        da = da.sel(theta=slice(*th))
        ra = ra.sel(theta=slice(*th))
        integration_dims.append("theta")
    else:  # this includes if integrate_angle==0, which is hacky but fine
        da = da.sel(theta=theta, method="nearest")
        ra = ra.sel(theta=theta, method="nearest")
        th = theta

    if integration_dims:  # if the list isn't empty
        da = integrate_da(da, integration_dims, weighting=1, normalisation=None)
        ra = integrate_da(ra, integration_dims, weighting=1, normalisation=None)

    norm = enhancement_factor(ds=da, ref=ra, common_dim=common_dim, method="groupby")

    if (
        window_radius == 0 and not integrate_angle
    ):  # if the precalculated norm_1 dataset will suffice
        norm = norm_1[variable].drop_sel({"excitonic_layer_thickness": 0})  # then use it instead
        norm = select_lorentz_line(norm, lorentz_line=lorentz_line, window_radius=window_radius)
        norm = norm.sel(theta=theta, method="nearest")

    match restack_ef:
        case True:
            norm = restack_plt_to_period(norm)

    if period_start < period_stop:
        norm = norm.sel(period=slice(period_start, period_stop))
    else:  # otherwise no data is selected and everything breaks
        norm = norm.sel(period=slice(period_start, None))

    plot_1, optimum_coords = plot_optimum_over_dim(
        norm,
        dim=dim,
        x="excitonic_layer_thickness",
        y="num_periods",
        optimise=optimise,
    )
    ############################################# code above is fresh, below is stale
    P = float(optimum_coords["period"])
    t = float(optimum_coords["excitonic_layer_thickness"])
    N = float(optimum_coords["num_periods"])
    # try:  # this should work if not integrating over theta
    #     th = float(optimum_coords["theta"])
    # except:  # probably the problem is that theta doesn't exist because I integrated over it already
    #     th = (theta, integrate_angle)
    lorentz_lines = lorentz_vlines(vline_locs, scale=1e-9, mode="wavelength").opts(
        opts.VLine(line_color=green, line_dash="dotted"),
    )

    # give the resonance line a special colour
    lorentz_lines.VLine.I.opts(opts.VLine(line_color=yellow))

    plot_1.opts(
        opts.QuadMesh(cmap=cmap),
        opts.Points(color="red"),
        opts.Overlay(title=f"{title}\nOptimal period: {P:.0f}"),
    )

    plots.append(plot_1)

    if "RTA_normal" in extra_plots:  # plot RTA at theta=0
        new_plot = plot_RTA(
            period=P, excitonic_layer_thickness=t, num_periods=N, theta=0
        )
        new_plot *= lorentz_lines

        plots.append(new_plot)

    if "RTA_int" in extra_plots:  # plot RTA at theta OR integrating over theta
        new_plot = plot_RTA(
            period=P, excitonic_layer_thickness=t, num_periods=N, theta=th
        )
        new_plot *= lorentz_lines
        plots.append(new_plot)

    if "norm_1_normal" in extra_plots:  # plot enhancement factor at theta=0
        sel = {
            "period": P,
            "excitonic_layer_thickness": t,
            "num_periods": N,
            "theta": 0,
        }
        new_plot = (
            plot_ef(variable="A", dataset=restacked_norm_1, sel=sel) * lorentz_lines
        ).opts(opts.Overlay(legend_position="bottom_right"))
        plots.append(new_plot)

    if (
        "norm_1_int" in extra_plots
    ):  # plot enhancement factor at theta OR integrating over theta
        try:  # this should work if not integrating over theta
            sel = {
                "period": P,
                "excitonic_layer_thickness": t,
                "num_periods": N,
                "theta": th,
            }
            new_plot = (
                plot_ef(variable="A", dataset=restacked_norm_1, sel=sel) * lorentz_lines
            ).opts(opts.Overlay(legend_position="bottom_right"))
        except:  # if integrating, we need to do the integral *before* normalising
            ds_int = sel_or_integrate(ds, dim="theta", val=th)
            crs_1_int = sel_or_integrate(crs_1, dim="theta", val=th)
            norm = enhancement_factor(
                ds_int,
                ref=crs_1_int,
                common_dim="total_excitonic_thickness",
                method="groupby",
            )
            restacked_norm = restack_plt_to_period(norm)

            sel = {"period": P, "excitonic_layer_thickness": t, "num_periods": N}
            new_plot = (
                plot_ef(variable="A", dataset=restacked_norm, sel=sel) * lorentz_lines
            ).opts(opts.Overlay(legend_position="bottom_right"))
        plots.append(new_plot)

    return hv.Layout(plots).cols(1)

# Iridescence differentiates between diffuse and collimated light

In [ ]:
P = 300
elt = 30
N = 10
theta = (0, 45)
fig = plot_R(
    dataset=restacked_ds.sel(
        period=P, excitonic_layer_thickness=elt, num_periods=N
    ).sel(theta=slice(*theta)).squeeze(),
    x="wavelength",
    y="theta",
)
fig.opts(opts.Image(clim=(0,None)))

In [ ]:
fig = compare_RTA_normal_vs_integrated(
    {"theta": 10, "label_override": " (θ = 10)"},
    {"theta": 20, "label_override": " (θ = 20)"},
    {"theta": 30, "label_override": " (θ = 30)"},
    period=P,
    excitonic_layer_thickness=elt,
    num_periods=N,
    plot_vars=["R"],
    include=["LOPC"],
)
fig.opts(opts.Curve(ylim=(0, None)))

In [ ]:
fig = compare_RTA_normal_vs_integrated(period=P, excitonic_layer_thickness=elt, num_periods=N, plot_vars=["R"])
fig.opts(opts.Curve(ylim=(0, None)))

In [ ]:
P = 200
elt = 30
N = 10
theta = (0, 45)
fig = plot_R(
    dataset=restacked_ds.sel(
        period=P, excitonic_layer_thickness=elt, num_periods=N
    ).sel(theta=slice(*theta)).squeeze(),
    x="wavelength",
    y="theta",
)
fig.opts(opts.Image(clim=(0,None)))

In [ ]:
fig = compare_RTA_normal_vs_integrated(
    {"theta": 10, "label_override": " (θ = 10)"},
    {"theta": 20, "label_override": " (θ = 20)"},
    {"theta": 30, "label_override": " (θ = 30)"},
    period=P,
    excitonic_layer_thickness=elt,
    num_periods=N,
    plot_vars=["R"],
    include=["LOPC"],
)
fig.opts(opts.Curve(ylim=(0, None)))

In [ ]:
fig = compare_RTA_normal_vs_integrated(period=P, excitonic_layer_thickness=elt, num_periods=N, plot_vars=["R"])
fig.opts(opts.Curve(ylim=(0, None)))

In [ ]:
N = 30
elt = 10
normal_dataset = integrate_da(
    restacked_ds.sel(num_periods=N, excitonic_layer_thickness=elt)
    .sel(theta=0)
    .squeeze(),
    dim="wavelength",
    normalisation=1,
)
angle_dataset = integrate_da(
    restacked_ds.sel(num_periods=N, excitonic_layer_thickness=elt)
    .sel(theta=slice(0, 45))
    .squeeze(),
    dim=["wavelength", "theta"],
    normalisation=1,
)
(
    compare_RTA(
        {"dataset": angle_dataset, "label_append": " (integrated)"},
        {"dataset": normal_dataset, "label_append": " (θ = 0)"},
        plot_func=plot_R,
        label_field=None,
        x="period",
    )
).redim(period=period_dim).opts(opts.Curve(ylim=(None, None), xlim=(100, 340), title="Integrated by wavelength"))

In [ ]:
(plot_ef(variable="R", dataset=(normal_dataset / angle_dataset), x="period")).redim(
    period=period_dim, R="R (normal / integrated)"
).opts(opts.Curve(xlim=(100, 340)))

In [ ]:
# peak is at Lambda = 180
P = 180
fig = compare_RTA_normal_vs_integrated(period=P, excitonic_layer_thickness=elt, num_periods=N, plot_vars=["R"])
fig.opts(opts.Curve(ylim=(0, None)))

In [ ]:
fig = plot_R(
    dataset=restacked_ds.sel(
        period=P, excitonic_layer_thickness=elt, num_periods=N
    ).sel(theta=slice(*theta_range)).squeeze(),
    x="wavelength",
    y="theta",
)
fig.opts(opts.Image(clim=(0,None)))

In [ ]:
(
    plot_R(dataset=(normal_dataset - angle_dataset), label_field=None, x="period")
    * hv.HLine(0)
).redim(period=period_dim, R="R (normal - integrated)",
).opts(
    opts.Curve(ylim=(None, None), xlim=(100, 340), title=""),
    opts.HLine(line_dash="dotted"),
)

In [ ]:
# second peak is at Lambda = 260
P = 260
fig = compare_RTA_normal_vs_integrated(period=P, excitonic_layer_thickness=elt, num_periods=N, plot_vars=["R"])
fig.opts(opts.Curve(ylim=(0, None)))

In [ ]:
fig = plot_R(
    dataset=restacked_ds.sel(
        period=P, excitonic_layer_thickness=elt, num_periods=N
    ).sel(theta=slice(*theta_range)).squeeze(),
    x="wavelength",
    y="theta",
)
fig.opts(opts.Image(clim=(0,None)))

In [ ]:
lorentz_line = -11
window_radius = 2
N = 10
elt = 30
theta_range = (0, 45)
normal_dataset = integrate_da(
    select_lorentz_line(restacked_ds, lorentz_line, window_radius)
    .sel(num_periods=N, excitonic_layer_thickness=elt)
    .sel(theta=0)
    .squeeze(),
    dim="wavelength",
    normalisation=1,
)
angle_dataset = integrate_da(
    select_lorentz_line(restacked_ds, lorentz_line, window_radius)
    .sel(num_periods=N, excitonic_layer_thickness=elt)
    .sel(theta=slice(*theta_range))
    .squeeze(),
    dim=["wavelength", "theta"],
    normalisation=1,
)
lines = lorentz_vlines(
    [lorentz_line - window_radius, lorentz_line + window_radius], scale=1e-9
).opts(
    opts.VLine(line_color=green, line_dash="dotted"),
)

(
    compare_RTA(
        {"dataset": angle_dataset, "label_append": " (integrated)"},
        {"dataset": normal_dataset, "label_append": " (θ = 0)"},
        plot_func=plot_R,
        label_field=None,
        x="period",
    )
).redim(period=period_dim).opts(opts.Curve(ylim=(None, None), xlim=(100, 340), title="Integrated by wavelength"))

In [ ]:
(plot_ef(variable="R", dataset=(normal_dataset / angle_dataset), x="period")).redim(
    period=period_dim, R="R (normal / integrated)"
).opts(opts.Curve(xlim=(100, 340)))

In [ ]:
# peak is at Lambda = 180 
P = 180
fig = compare_RTA_normal_vs_integrated(period=P, excitonic_layer_thickness=elt, num_periods=N, plot_vars=["R"])*lines
fig.opts(opts.Curve(ylim=(0, None)))

In [ ]:
fig = plot_R(
    dataset=restacked_ds.sel(
        period=P, excitonic_layer_thickness=elt, num_periods=N
    ).sel(theta=slice(*theta_range)).squeeze(),
    x="wavelength",
    y="theta",
)*lines
fig.opts(opts.Image(clim=(0,None)))

In [ ]:
(
    plot_R(dataset=(normal_dataset - angle_dataset), label_field=None, x="period")
    * hv.HLine(0)
).redim(period=period_dim, R="R (normal - integrated)",
).opts(
    opts.Curve(ylim=(None, None), title=""),
    opts.HLine(line_dash="dotted"),
)

In [ ]:
# peak is at Lambda = 190
P = 190
fig = compare_RTA_normal_vs_integrated(period=P, excitonic_layer_thickness=elt, num_periods=N, plot_vars=["R"])*lines
fig.opts(opts.Curve(ylim=(0, None)))

In [ ]:
fig = plot_R(
    dataset=restacked_ds.sel(
        period=P, excitonic_layer_thickness=elt, num_periods=N
    ).sel(theta=slice(*theta_range)).squeeze(),
    x="wavelength",
    y="theta",
)*lines
fig.opts(opts.Image(clim=(0,None)))

# Cleanup

In [ ]:
# hv.archive.export()